In [25]:
# ═══════════════════════════════════════════════════════════════════
# BLOCO 1: SELETOR DE PASTA COM TIMER + MIGRAÇÃO
# Versão: 4.4 - COM MELHORIAS DE OBSERVABILIDADE E CONSISTÊNCIA
# Data: 2025-10-17
# ═══════════════════════════════════════════════════════════════════
# MELHORIAS v4.4:
# ✅ MELHORIA 1: Salvar estado local .bloco_1_state.json
# ✅ MELHORIA 2: Registro de versão do código
# ✅ MELHORIA 3: Validação de dependências
# ═══════════════════════════════════════════════════════════════════

# ═══════════════════════════════════════════════════════════════════
# METADADOS DA VERSÃO
# ═══════════════════════════════════════════════════════════════════

VERSAO_BLOCO1 = '4.4'
DATA_VERSAO = '2025-10-17'
CHANGELOG_V44 = {
    'v4.4': {
        'data': '2025-10-17',
        'melhorias': [
            'Salvar estado local em .bloco_1_state.json',
            'Registro de versão do código',
            'Validação de dependências no início'
        ],
        'compatibilidade': 'v4.3'
    }
}

print("="*70)
print(f" 🔍 PROCESSADOR DE ARQUIVOS DESCONHECIDOS v{VERSAO_BLOCO1}")
print("="*70)
print(f" Versão: {VERSAO_BLOCO1} | Data: {DATA_VERSAO}")
print(" Timer | Migração | Dicionários | Validações | Logs | Estado")
print("="*70)

# ═══════════════════════════════════════════════════════════════════
# MELHORIA 3: VALIDAÇÃO DE DEPENDÊNCIAS
# ═══════════════════════════════════════════════════════════════════

def validar_dependencias():
    """
    Valida que todas as bibliotecas necessárias estão instaladas.

    Evita erros confusos mais tarde na execução.
    """
    print("\n🔍 Validando dependências...")

    dependencias = {
        'pandas': 'pandas',
        'numpy': 'numpy',
        'openpyxl': 'openpyxl',
        'xlrd': 'xlrd',
        'tkinter': 'tkinter (built-in Python)'
    }

    faltando = []
    instaladas = []

    for modulo, nome_pip in dependencias.items():
        try:
            __import__(modulo)
            instaladas.append(f"✅ {modulo}")
        except ImportError:
            faltando.append(nome_pip)

    # Mostrar resultado
    for lib in instaladas:
        print(f"   {lib}")

    if faltando:
        print("\n❌ ERRO: Bibliotecas faltando!")
        print("─" * 70)
        for lib in faltando:
            print(f"   ❌ {lib}")
        print("\n💡 Solução:")
        libs_pip = [lib for lib in faltando if 'built-in' not in lib]
        if libs_pip:
            print(f"   Execute: pip install {' '.join(libs_pip)}")
        print()
        raise ImportError(
            f"Bibliotecas necessárias não instaladas: {', '.join(faltando)}"
        )

    print("✅ Todas as dependências instaladas!\n")

# Validar ANTES de importar
validar_dependencias()

# ═══════════════════════════════════════════════════════════════════
# IMPORTS
# ═══════════════════════════════════════════════════════════════════

import pandas as pd
import numpy as np
import xlrd
import re
import json
import os
import subprocess
import platform
import shutil
from pathlib import Path
from datetime import datetime
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

import tkinter as tk
from tkinter import filedialog, messagebox, ttk

print("✅ Imports carregados")

# ═══════════════════════════════════════════════════════════════════
# CLASSE: LocalizadorDicionario (SISTEMA DE PERSISTÊNCIA GLOBAL)
# ═══════════════════════════════════════════════════════════════════

class LocalizadorDicionario:
    """
    Sistema de localização persistente de dicionários entre sessões.

    Mantém log global em: ~/.processador_dicionario_localizador.json

    Métodos Públicos:
    - obter_dicionario_atual() -> Path  # Para BLOCO 2+
    - obter_pasta_base_atual() -> Path  # Para FileManager
    - obter_timestamp_atual() -> str    # Para recuperar timestamp
    - registrar_mudanca()               # Chamado por BLOCO 1
    """

    LOG_FILE = Path.home() / '.processador_dicionario_localizador.json'

    @classmethod
    def carregar_log(cls):
        """Carrega log global com fallback para encoding"""
        if cls.LOG_FILE.exists():
            for encoding in ['utf-8', 'utf-8-sig', 'latin-1']:
                try:
                    with open(cls.LOG_FILE, 'r', encoding=encoding) as f:
                        return json.load(f)
                except (UnicodeDecodeError, json.JSONDecodeError):
                    continue
        return {
            'versao': '2.1',
            'dicionario_atual': None,
            'pasta_base_atual': None,
            'timestamp': None,
            'historico': []
        }

    @classmethod
    def salvar_log(cls, log):
        """Salva log com backup automático"""
        # Backup do log anterior
        if cls.LOG_FILE.exists():
            backup_file = cls.LOG_FILE.with_suffix('.json.bak')
            shutil.copy2(cls.LOG_FILE, backup_file)

        # Salvar novo log
        with open(cls.LOG_FILE, 'w', encoding='utf-8') as f:
            json.dump(log, f, indent=2, ensure_ascii=False)

    @classmethod
    def obter_dicionario_atual(cls):
        """Retorna Path do dicionário atual (para BLOCO 2+)"""
        log = cls.carregar_log()
        if not log['dicionario_atual']:
            raise FileNotFoundError(
                "❌ Dicionário não encontrado! Execute BLOCO 1."
            )

        dicionario_path = Path(log['dicionario_atual'])
        if not dicionario_path.exists():
            raise FileNotFoundError(
                f"❌ Dicionário não existe: {dicionario_path}"
            )

        return dicionario_path

    @classmethod
    def obter_pasta_base_atual(cls):
        """Retorna Path da pasta base atual (para FileManager)"""
        log = cls.carregar_log()
        if not log['pasta_base_atual']:
            raise FileNotFoundError(
                "❌ Pasta base não encontrada! Execute BLOCO 1."
            )

        pasta_base = Path(log['pasta_base_atual'])
        if not pasta_base.exists():
            raise FileNotFoundError(
                f"❌ Pasta base não existe: {pasta_base}"
            )

        return pasta_base

    @classmethod
    def obter_timestamp_atual(cls):
        """Retorna timestamp da execução atual (para BLOCO 2+)"""
        log = cls.carregar_log()
        if not log.get('timestamp'):
            raise FileNotFoundError(
                "❌ Timestamp não encontrado! Execute BLOCO 1."
            )
        return log['timestamp']

    @classmethod
    def registrar_mudanca(cls, pasta_base, timestamp, dicionario_path=None,
                         migrado_de=None, versao_bloco1=None):
        """
        Registra mudança de localização no log global.

        IMPORTANTE: Agora aceita timestamp e registra no LOG GLOBAL.
        dicionario_path é OPCIONAL - só registra se existir.

        Args:
            pasta_base: Path da pasta container
            timestamp: String timestamp da execução
            dicionario_path: Path do dicionário (opcional)
            migrado_de: Path de onde migrou (opcional)
            versao_bloco1: Versão do BLOCO 1 que criou (opcional)
        """
        log = cls.carregar_log()

        entrada = {
            'timestamp': datetime.now().isoformat(),
            'pasta_base': str(pasta_base),
            'dicionario_path': str(dicionario_path) if dicionario_path else None,
            'timestamp_execucao': timestamp,
            'existe': dicionario_path.exists() if dicionario_path else False,
            'versao_bloco1': versao_bloco1 or 'desconhecida'
        }

        if migrado_de:
            entrada['migrado_de'] = str(migrado_de)

        log['pasta_base_atual'] = str(pasta_base)
        log['timestamp'] = timestamp

        # Só registrar dicionário se ele REALMENTE existir
        if dicionario_path and dicionario_path.exists():
            log['dicionario_atual'] = str(dicionario_path)
        else:
            log['dicionario_atual'] = None

        log['historico'].append(entrada)
        log['ultima_atualizacao'] = datetime.now().isoformat()

        cls.salvar_log(log)

        print(f"\n📍 Localizador atualizado:")
        print(f"   Container: {pasta_base.name}")
        print(f"   Timestamp: {timestamp}")
        print(f"   Versão BLOCO 1: {versao_bloco1 or 'desconhecida'}")
        if dicionario_path and dicionario_path.exists():
            print(f"   Dicionário: {dicionario_path.name}")
        print(f"   Log: {cls.LOG_FILE}")

# ═══════════════════════════════════════════════════════════════════
# CLASSE: SeletorPastaComTimer (GUI COM TIMER)
# ═══════════════════════════════════════════════════════════════════

class SeletorPastaComTimer:
    """Seletor de pasta destino com timer de 10s e memória"""

    CONFIG_FILE = Path.home() / '.processador_last_directory.json'

    def __init__(self):
        self.resultado = {'path': None, 'acao': None}
        self.timeout_ocorreu = False

    def carregar_ultima_pasta(self):
        """Carrega última pasta usada"""
        if self.CONFIG_FILE.exists():
            try:
                with open(self.CONFIG_FILE, 'r', encoding='utf-8') as f:
                    config = json.load(f)
                ultima_pasta = Path(config.get('last_directory', ''))
                if ultima_pasta.exists():
                    return ultima_pasta
            except:
                pass
        return None

    def salvar_escolha(self, pasta):
        """Salva escolha para próxima execução"""
        config = {
            'last_directory': str(pasta),
            'timestamp': datetime.now().isoformat()
        }
        with open(self.CONFIG_FILE, 'w', encoding='utf-8') as f:
            json.dump(config, f, indent=2)

    def validar_pasta_destino(self, pasta):
        """Valida se pasta tem permissões adequadas"""
        pasta = Path(pasta)

        # Verificar permissão de escrita
        if not os.access(pasta, os.W_OK):
            return False, "❌ Sem permissão de escrita"

        # Verificar espaço em disco (mínimo 100MB)
        stat = os.statvfs(pasta) if hasattr(os, 'statvfs') else None
        if stat:
            espaco_livre = stat.f_bavail * stat.f_frsize
            if espaco_livre < 100 * 1024 * 1024:  # 100MB
                return False, (
                    f"❌ Espaço insuficiente "
                    f"({espaco_livre/1024/1024:.1f}MB)"
                )

        return True, "✅ Pasta válida"

    def selecionar_com_timer(self):
        """Exibe GUI com timer de 10s"""
        ultima_pasta = self.carregar_ultima_pasta()

        root = tk.Tk()
        root.title("Processador - Pasta Destino")
        root.geometry("650x450")

        # Centralizar janela
        x = (root.winfo_screenwidth() // 2) - 325
        y = (root.winfo_screenheight() // 2) - 225
        root.geometry(f"+{x}+{y}")

        frame = tk.Frame(root, padx=20, pady=20, bg='white')
        frame.pack(fill=tk.BOTH, expand=True)

        # Título
        tk.Label(
            frame,
            text="📂 Pasta DESTINO",
            font=('Arial', 14, 'bold'),
            bg='white'
        ).pack(pady=(0, 15))

        # Mensagem
        if ultima_pasta:
            msg = f"Timer de 10s para usar:\n\n{ultima_pasta}"
        else:
            msg = "Primeira execução - selecione pasta"

        tk.Label(
            frame,
            text=msg,
            justify=tk.LEFT,
            font=('Arial', 9),
            bg='white',
            wraplength=600
        ).pack(pady=(0, 15))

        # Timer
        contador = [10]
        if ultima_pasta:
            label_timer = tk.Label(
                frame,
                text=f"{contador[0]}s",
                font=('Arial', 24, 'bold'),
                fg='#FF4444',
                bg='white'
            )
            label_timer.pack(pady=(5, 20))

            def countdown():
                if contador[0] > 0 and not self.timeout_ocorreu:
                    contador[0] -= 1
                    label_timer.config(text=f"{contador[0]}s")
                    root.after(1000, countdown)
                elif contador[0] == 0:
                    self.timeout_ocorreu = True
                    self.resultado['path'] = ultima_pasta
                    self.resultado['acao'] = 'TIMEOUT'
                    root.quit()
                    root.destroy()

            root.after(1000, countdown)

        # Botões
        def escolher_nova():
            self.timeout_ocorreu = True
            root.withdraw()
            nova_pasta = filedialog.askdirectory(
                title="Pasta DESTINO",
                initialdir=ultima_pasta if ultima_pasta else None
            )

            if nova_pasta:
                valido, msg = self.validar_pasta_destino(nova_pasta)
                if not valido:
                    messagebox.showerror("Pasta Inválida", msg)
                    self.resultado['path'] = ultima_pasta
                    self.resultado['acao'] = 'CANCELADO'
                else:
                    self.resultado['path'] = Path(nova_pasta)
                    self.resultado['acao'] = 'NOVA'
            else:
                self.resultado['path'] = ultima_pasta
                self.resultado['acao'] = 'CANCELADO'

            root.quit()
            root.destroy()

        def usar_ultima():
            self.timeout_ocorreu = True
            self.resultado['path'] = ultima_pasta
            self.resultado['acao'] = 'MANTEVE'
            root.quit()
            root.destroy()

        frame_btns = tk.Frame(frame, bg='white')
        frame_btns.pack(side=tk.BOTTOM, pady=15)

        tk.Button(
            frame_btns,
            text="📁 Nova Pasta",
            command=escolher_nova,
            width=20,
            height=2,
            font=('Arial', 10, 'bold'),
            bg='#4CAF50',
            fg='white'
        ).pack(side=tk.LEFT, padx=10)

        if ultima_pasta:
            tk.Button(
                frame_btns,
                text="✅ Usar Última",
                command=usar_ultima,
                width=20,
                height=2,
                font=('Arial', 10),
                bg='#2196F3',
                fg='white'
            ).pack(side=tk.LEFT, padx=10)

        root.mainloop()
        return self.resultado

# ═══════════════════════════════════════════════════════════════════
# CLASSE: SeletorOrigemComTimer (GUI MIGRAÇÃO)
# ═══════════════════════════════════════════════════════════════════

class SeletorOrigemComTimer:
    """Pergunta se deseja copiar arquivos de execução anterior"""

    def __init__(self):
        self.resultado = {'copiar': False, 'path': None}
        self.timeout_ocorreu = False

    def perguntar_origem(self):
        """GUI com timer de 5s (default: NÃO)"""
        root = tk.Tk()
        root.title("Processador - Copiar Arquivos Anteriores?")
        root.geometry("650x350")

        x = (root.winfo_screenwidth() // 2) - 325
        y = (root.winfo_screenheight() // 2) - 175
        root.geometry(f"+{x}+{y}")

        frame = tk.Frame(root, padx=20, pady=20, bg='white')
        frame.pack(fill=tk.BOTH, expand=True)

        tk.Label(
            frame,
            text="📂 Copiar arquivos de execução anterior?",
            font=('Arial', 14, 'bold'),
            bg='white'
        ).pack(pady=(0, 15))

        msg = (
            "Se houver dicionários, logs ou outputs anteriores,\n"
            "você pode copiá-los para a nova estrutura."
        )
        tk.Label(
            frame,
            text=msg,
            justify=tk.LEFT,
            font=('Arial', 9),
            bg='white',
            wraplength=600
        ).pack(pady=(0, 15))

        contador = [5]
        label_timer = tk.Label(
            frame,
            text=f"{contador[0]}s (auto: NÃO)",
            font=('Arial', 18, 'bold'),
            fg='#FF6600',
            bg='white'
        )
        label_timer.pack(pady=(5, 20))

        def countdown():
            if contador[0] > 0 and not self.timeout_ocorreu:
                contador[0] -= 1
                label_timer.config(text=f"{contador[0]}s (auto: NÃO)")
                root.after(1000, countdown)
            elif contador[0] == 0:
                self.timeout_ocorreu = True
                self.resultado['copiar'] = False
                root.quit()
                root.destroy()

        root.after(1000, countdown)

        def sim_copiar():
            self.timeout_ocorreu = True
            root.withdraw()
            pasta_origem = filedialog.askdirectory(
                title="Selecione pasta ORIGEM (execução anterior)"
            )
            if pasta_origem:
                self.resultado['copiar'] = True
                self.resultado['path'] = Path(pasta_origem)
            else:
                self.resultado['copiar'] = False
            root.quit()
            root.destroy()

        def nao_copiar():
            self.timeout_ocorreu = True
            self.resultado['copiar'] = False
            root.quit()
            root.destroy()

        frame_btns = tk.Frame(frame, bg='white')
        frame_btns.pack(side=tk.BOTTOM, pady=15)

        tk.Button(
            frame_btns,
            text="✅ SIM - Selecionar Origem",
            command=sim_copiar,
            width=25,
            height=2,
            font=('Arial', 10, 'bold'),
            bg='#4CAF50',
            fg='white'
        ).pack(side=tk.LEFT, padx=10)

        tk.Button(
            frame_btns,
            text="❌ NÃO - Começar do Zero",
            command=nao_copiar,
            width=25,
            height=2,
            font=('Arial', 10),
            bg='#757575',
            fg='white'
        ).pack(side=tk.LEFT, padx=10)

        root.mainloop()
        return self.resultado

# ═══════════════════════════════════════════════════════════════════
# CLASSE: LimpadorRoot (DETECÇÃO DE POLUIÇÃO)
# ═══════════════════════════════════════════════════════════════════

class LimpadorRoot:
    """Detecta e limpa pastas antigas no root"""

    def __init__(self, pasta_root):
        self.pasta_root = Path(pasta_root)

    def detectar_poluicao(self):
        """Encontra pastas numeradas antigas"""
        pastas_numeradas = [
            p for p in self.pasta_root.iterdir()
            if p.is_dir() and p.name[:2].isdigit() and '_' in p.name
        ]
        return pastas_numeradas

    def perguntar_limpeza(self, pastas):
        """GUI para decidir o que fazer com pastas antigas"""
        root = tk.Tk()
        root.title("Processador - Limpar Root?")
        root.geometry("650x400")

        x = (root.winfo_screenwidth() // 2) - 325
        y = (root.winfo_screenheight() // 2) - 200
        root.geometry(f"+{x}+{y}")

        frame = tk.Frame(root, padx=20, pady=20, bg='white')
        frame.pack(fill=tk.BOTH, expand=True)

        tk.Label(
            frame,
            text="⚠️  Pastas antigas detectadas no root",
            font=('Arial', 14, 'bold'),
            bg='white',
            fg='#FF6600'
        ).pack(pady=(0, 10))

        msg = f"Encontradas {len(pastas)} pastas soltas:\n\n"
        msg += "\n".join([f"• {p.name}" for p in pastas[:5]])
        if len(pastas) > 5:
            msg += f"\n... e mais {len(pastas)-5}"

        tk.Label(
            frame,
            text=msg,
            justify=tk.LEFT,
            font=('Arial', 9),
            bg='white',
            wraplength=600
        ).pack(pady=(0, 15))

        resultado = {'acao': None}

        def mover():
            resultado['acao'] = 'MOVER'
            root.quit()
            root.destroy()

        def deletar():
            resultado['acao'] = 'DELETAR'
            root.quit()
            root.destroy()

        def ignorar():
            resultado['acao'] = 'IGNORAR'
            root.quit()
            root.destroy()

        frame_btns = tk.Frame(frame, bg='white')
        frame_btns.pack(side=tk.BOTTOM, pady=15)

        tk.Button(
            frame_btns,
            text="📦 Mover p/ Estrutura",
            command=mover,
            width=20,
            height=2,
            font=('Arial', 9, 'bold'),
            bg='#4CAF50',
            fg='white'
        ).pack(side=tk.LEFT, padx=5)

        tk.Button(
            frame_btns,
            text="🗑️  Deletar",
            command=deletar,
            width=15,
            height=2,
            font=('Arial', 9),
            bg='#F44336',
            fg='white'
        ).pack(side=tk.LEFT, padx=5)

        tk.Button(
            frame_btns,
            text="⏭️  Ignorar",
            command=ignorar,
            width=15,
            height=2,
            font=('Arial', 9),
            bg='#757575',
            fg='white'
        ).pack(side=tk.LEFT, padx=5)

        root.mainloop()
        return resultado['acao']

# ═══════════════════════════════════════════════════════════════════
# CLASSE: GerenciadorMigracao (CÓPIA COMPLETA COM LOG)
# ═══════════════════════════════════════════════════════════════════

class GerenciadorMigracao:
    """Gerencia cópia completa de execuções anteriores"""

    def __init__(self, pasta_origem, pasta_destino_container):
        self.pasta_origem = Path(pasta_origem)
        self.pasta_destino = Path(pasta_destino_container)
        self.timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        self.log_detalhado = []
        self.erros = []

    def detectar_estrutura(self):
        """Detecta pastas e dicionários na origem"""
        pastas = [
            p for p in self.pasta_origem.iterdir()
            if p.is_dir() and (
                p.name[:2].isdigit() or
                'dicionario' in p.name.lower()
            )
        ]

        dicionarios = []
        pasta_dict = self.pasta_origem / '05_Dicionarios'

        if pasta_dict.exists():
            dicionarios.extend(list(pasta_dict.glob('*.json')))

        dicionarios.extend(
            list(self.pasta_origem.glob('dicionario*.json'))
        )
        dicionarios = list(set(dicionarios))

        return pastas, dicionarios

    def validar_dicionario(self, arquivo_json):
        """Valida integridade do dicionário JSON"""
        try:
            with open(arquivo_json, 'r', encoding='utf-8') as f:
                data = json.load(f)

            # Verificar estrutura mínima
            if not isinstance(data, dict):
                return False, "JSON não é um dicionário"

            return True, "✅ Válido"
        except json.JSONDecodeError as e:
            return False, f"JSON inválido: {str(e)}"
        except Exception as e:
            return False, f"Erro: {str(e)}"

    def copiar_tudo(self):
        """Copia tudo com tratamento de erros"""
        pastas, dicionarios = self.detectar_estrutura()

        print(f"\n📂 MIGRAÇÃO COMPLETA")
        print("─" * 70)
        print(f"   De: {self.pasta_origem}")
        print(f"   Para: {self.pasta_destino}")
        print(f"   Pastas: {len(pastas)}")
        print(f"   Dicionários: {len(dicionarios)}")
        print()

        if not pastas and not dicionarios:
            print("ℹ️  Nada para copiar")
            return {'migrado': False}

        print("Copiar? (S/N ou Enter=S): ", end='')
        resposta = input().strip().upper()
        if resposta and resposta != 'S':
            print("❌ Migração cancelada")
            return {'migrado': False}

        print(f"\n🔄 Copiando...\n")

        arquivos_copiados = 0
        bytes_copiados = 0
        dicionarios_copiados = []

        # Copiar pastas
        for pasta in sorted(pastas):
            try:
                if pasta.name == '05_Dicionarios':
                    continue

                destino_pasta = self.pasta_destino / pasta.name
                destino_pasta.mkdir(parents=True, exist_ok=True)

                print(f"📁 {pasta.name}", end='')
                arquivos_pasta = 0
                bytes_pasta = 0

                for arquivo in pasta.rglob('*'):
                    if arquivo.is_file():
                        try:
                            destino_arq = (
                                destino_pasta /
                                arquivo.relative_to(pasta)
                            )
                            destino_arq.parent.mkdir(
                                parents=True,
                                exist_ok=True
                            )
                            shutil.copy2(arquivo, destino_arq)
                            arquivos_copiados += 1
                            arquivos_pasta += 1
                            bytes_pasta += arquivo.stat().st_size
                        except Exception as e:
                            self.erros.append({
                                'arquivo': str(arquivo),
                                'erro': str(e)
                            })

                bytes_copiados += bytes_pasta
                tamanho_kb = bytes_pasta/1024
                print(f" → {arquivos_pasta} arquivos ({tamanho_kb:.1f} KB)")

                self.log_detalhado.append({
                    'tipo': 'pasta',
                    'nome': pasta.name,
                    'arquivos': arquivos_pasta,
                    'bytes': bytes_pasta
                })

            except Exception as e:
                print(f" ❌ ERRO: {str(e)}")
                self.erros.append({
                    'pasta': pasta.name,
                    'erro': str(e)
                })

        # Copiar dicionários
        if dicionarios:
            pasta_dict_destino = self.pasta_destino / '05_Dicionarios'
            pasta_dict_destino.mkdir(parents=True, exist_ok=True)

            print(f"\n📚 DICIONÁRIOS ({len(dicionarios)}):")

            for dic in dicionarios:
                try:
                    # Validar antes de copiar
                    valido, msg = self.validar_dicionario(dic)

                    destino_dic = pasta_dict_destino / dic.name
                    shutil.copy2(dic, destino_dic)
                    tamanho = dic.stat().st_size

                    status = "✅" if valido else "⚠️"
                    tamanho_kb = tamanho/1024
                    print(f"   {status} {dic.name} ({tamanho_kb:.1f} KB) - {msg}")

                    dicionarios_copiados.append(str(destino_dic))
                    arquivos_copiados += 1
                    bytes_copiados += tamanho

                    self.log_detalhado.append({
                        'tipo': 'dicionario',
                        'nome': dic.name,
                        'bytes': tamanho,
                        'path': str(destino_dic),
                        'validado': valido
                    })

                except Exception as e:
                    print(f"   ❌ {dic.name}: {str(e)}")
                    self.erros.append({
                        'dicionario': dic.name,
                        'erro': str(e)
                    })

        total_mb = bytes_copiados/1024/1024
        print(f"\n✅ TOTAL: {arquivos_copiados} arquivos, {total_mb:.2f} MB")

        if self.erros:
            print(f"⚠️  {len(self.erros)} erros durante cópia (ver log)")

        self._salvar_log_local({
            'migrado': True,
            'arquivos': arquivos_copiados,
            'bytes': bytes_copiados,
            'dicionarios': len(dicionarios_copiados),
            'erros': len(self.erros),
            'detalhes': self.log_detalhado,
            'log_erros': self.erros
        })

        return {
            'migrado': True,
            'arquivos': arquivos_copiados,
            'dicionarios': dicionarios_copiados,
            'erros': self.erros
        }

    def _salvar_log_local(self, info):
        """Salva log detalhado da migração"""
        log_file = self.pasta_destino / 'log_migracoes.json'

        if log_file.exists():
            with open(log_file, 'r', encoding='utf-8') as f:
                historico = json.load(f)
        else:
            historico = {'migracoes': []}

        entrada = {
            'timestamp': self.timestamp,
            'data_hora': datetime.now().isoformat(),
            'pasta_origem': str(self.pasta_origem),
            'pasta_destino': str(self.pasta_destino),
            **info
        }

        historico['migracoes'].append(entrada)
        historico['ultima_migracao'] = self.timestamp

        with open(log_file, 'w', encoding='utf-8') as f:
            json.dump(historico, f, indent=2, ensure_ascii=False)

        print(f"💾 Log: {log_file.name}")

# ═══════════════════════════════════════════════════════════════════
# CLASSE: FileManagerInterativo (GERENCIADOR DE ARQUIVOS)
# ═══════════════════════════════════════════════════════════════════

class FileManagerInterativo:
    """Gerenciador de arquivos e estrutura de pastas"""

    def __init__(self, base_path=None):
        self.base_path = Path(base_path) if base_path else Path.cwd()

        # Estrutura de pastas padrão
        self.pastas = {
            'entrada': self.base_path / '01_Entrada',
            'processados': self.base_path / '02_Processados',
            'outputs': self.base_path / '03_Outputs',
            'logs': self.base_path / '04_Logs',
            'dicionarios': self.base_path / '05_Dicionarios',
            'codigos_integracao': self.base_path / '06_Codigos_Integracao'
        }

        # Criar todas as pastas
        for pasta in self.pastas.values():
            pasta.mkdir(parents=True, exist_ok=True)

        self.timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

        print(f"✅ FileManager inicializado")
        print(f"   📂 Container: {self.base_path.name}")
        print(f"   🕐 Timestamp: {self.timestamp}")

    def salvar(self, df, nome, tipo='xlsx', pasta='processados'):
        """Salva DataFrame na pasta especificada"""
        arquivo = (
            self.pastas[pasta] /
            f"{nome}_{self.timestamp}.{tipo}"
        )

        if tipo == 'xlsx':
            df.to_excel(arquivo, index=False, engine='openpyxl')
        elif tipo == 'csv':
            df.to_csv(arquivo, index=False, encoding='utf-8-sig')

        return arquivo

    def abrir_pasta(self, pasta):
        """Abre pasta no explorer do sistema"""
        caminho = self.pastas[pasta]
        sistema = platform.system()

        try:
            if sistema == 'Windows':
                os.startfile(caminho)
            elif sistema == 'Darwin':  # macOS
                subprocess.run(['open', caminho])
            else:  # Linux
                subprocess.run(['xdg-open', caminho])
        except Exception as e:
            print(f"⚠️  Não foi possível abrir pasta: {e}")

# ═══════════════════════════════════════════════════════════════════
# FUNÇÃO: gerar_readme
# ═══════════════════════════════════════════════════════════════════

def gerar_readme(pasta_base, versao_bloco1):
    """Gera README.md na pasta container"""
    readme = f"""# 📚 PROCESSADOR DE ARQUIVOS DESCONHECIDOS

**Gerado:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
**Pasta:** {pasta_base}
**Versão BLOCO 1:** {versao_bloco1}

---

## 📁 ESTRUTURA

```
{pasta_base.name}/
├── 01_Entrada/          ← Arquivos originais
├── 02_Processados/      ← Dados limpos
├── 03_Outputs/          ← Resultados finais
├── 04_Logs/             ← Logs de execução ⭐ COMUNICAÇÃO VIA LOG
├── 05_Dicionarios/      ← Mapeamentos DE-PARA
├── 06_Codigos_Integracao/ ← Scripts reutilizáveis
├── README.md            ← Este arquivo
└── log_migracoes.json   ← Histórico de migrações
```

---

## 📚 LOCALIZADOR DE DICIONÁRIO

**Para notebooks consumidores (BLOCO 2+):**

```python
from bloco1 import LocalizadorDicionario

# Obter dicionário atual
dicionario_path = LocalizadorDicionario.obter_dicionario_atual()

# Obter pasta base
pasta_base = LocalizadorDicionario.obter_pasta_base_atual()

# Obter timestamp da execução
timestamp = LocalizadorDicionario.obter_timestamp_atual()

# Carregar dicionário
import json
with open(dicionario_path, 'r', encoding='utf-8') as f:
    dicionario = json.load(f)
```

**Log global:** `~/.processador_dicionario_localizador.json`

---

## 🔗 COMUNICAÇÃO ENTRE BLOCOS (0% MEMÓRIA, 100% LOG)

Todos os blocos seguem o padrão:

1. **LER** do LOG GLOBAL:
   - pasta_base_atual
   - timestamp
   - dicionario_atual (se existir)

2. **RECRIAR** objetos localmente:
   - FileManager(pasta_base)
   - Carregar dicionário de 04_Logs/

3. **PROCESSAR** dados do bloco

4. **SALVAR** estado em 04_Logs/:
   - .bloco_N_state.json
   - Dados específicos do bloco

---

## 🔄 HISTÓRICO DE MIGRAÇÕES

Ver: `log_migracoes.json`

---

## 📋 ESTADO DO BLOCO 1

Ver: `04_Logs/.bloco_1_state.json`

---

## 🆘 SUPORTE

- Erros: `04_Logs/`
- Dicionário perdido: Execute BLOCO 1
- Migração: Consulte `log_migracoes.json`
- Versão do código: `{versao_bloco1}`
"""

    readme_path = pasta_base / 'README.md'
    with open(readme_path, 'w', encoding='utf-8') as f:
        f.write(readme)

    print(f"📖 README: {readme_path.name}")

# ═══════════════════════════════════════════════════════════════════
# EXECUÇÃO PRINCIPAL DO BLOCO 1
# ═══════════════════════════════════════════════════════════════════

print("\n" + "="*70)
print("🔵 ETAPA 1: SELECIONANDO PASTA DESTINO...")
print("="*70)

seletor = SeletorPastaComTimer()
resultado_destino = seletor.selecionar_com_timer()

if not resultado_destino['path']:
    print("❌ Nenhuma pasta selecionada")
    raise ValueError("Execução cancelada")

print(f"\n✅ Destino: {resultado_destino['path']}")
print(f"   Ação: {resultado_destino['acao']}")
seletor.salvar_escolha(resultado_destino['path'])

pasta_root_destino = resultado_destino['path']

# ═══════════════════════════════════════════════════════════════════
print("\n" + "="*70)
print("🔵 ETAPA 2: VERIFICANDO POLUIÇÃO NO ROOT...")
print("="*70)

limpador = LimpadorRoot(pasta_root_destino)
pastas_poluidas = limpador.detectar_poluicao()

acao = None
if pastas_poluidas:
    print(f"\n⚠️  {len(pastas_poluidas)} pastas antigas no root!")
    acao = limpador.perguntar_limpeza(pastas_poluidas)

    if acao == 'DELETAR':
        print("\n🗑️  Deletando...")
        for pasta in pastas_poluidas:
            try:
                shutil.rmtree(pasta)
                print(f"   ✅ {pasta.name}")
            except Exception as e:
                print(f"   ❌ {pasta.name}: {e}")

    elif acao == 'MOVER':
        print("\n📦 Mover será feito após criar container")

    else:
        print("\n⏭️  Ignorando pastas antigas")
else:
    print("✅ Root limpo")

# ═══════════════════════════════════════════════════════════════════
print("\n" + "="*70)
print("🔵 ETAPA 3: CRIANDO PASTA CONTAINER...")
print("="*70)

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
nome_container = f"PROCESSAR_ARQUIVOS_{timestamp}"
pasta_container = pasta_root_destino / nome_container

pasta_container.mkdir(parents=True, exist_ok=True)
print(f"✅ Container: {nome_container}")

# ═══════════════════════════════════════════════════════════════════
print("\n" + "="*70)
print("🔵 ETAPA 4: COPIAR DE EXECUÇÃO ANTERIOR?")
print("="*70)

seletor_origem = SeletorOrigemComTimer()
resultado_origem = seletor_origem.perguntar_origem()

dicionarios_migrados = []
info_mig = {}

if resultado_origem['copiar'] and resultado_origem['path']:
    print(f"\n📂 Origem: {resultado_origem['path']}")
    gerenciador_mig = GerenciadorMigracao(
        resultado_origem['path'],
        pasta_container
    )
    info_mig = gerenciador_mig.copiar_tudo()

    if info_mig.get('migrado'):
        print(f"\n✅ Migração concluída")
        if info_mig.get('dicionarios'):
            dicionarios_migrados = info_mig['dicionarios']
            print(f"   📚 {len(dicionarios_migrados)} dicionários copiados")
        if info_mig.get('erros'):
            print(f"   ⚠️  {len(info_mig['erros'])} erros (ver log)")
else:
    print("✅ Começando do zero (sem cópia)")

# Mover pastas antigas se solicitado
if pastas_poluidas and acao == 'MOVER':
    print("\n📦 Movendo pastas antigas para container...")
    for pasta in pastas_poluidas:
        try:
            destino = pasta_container / pasta.name
            if destino.exists():
                shutil.rmtree(destino)
            shutil.move(str(pasta), str(destino))
            print(f"   ✅ {pasta.name}")
        except Exception as e:
            print(f"   ❌ {pasta.name}: {e}")

# ═══════════════════════════════════════════════════════════════════
print("\n" + "="*70)
print("🔵 ETAPA 5: INICIALIZANDO FILEMANAGER...")
print("="*70)

fm = FileManagerInterativo(pasta_container)

# Detectar dicionário migrado (se existir)
pasta_dict = fm.pastas['dicionarios']
arquivos_dict = list(pasta_dict.glob('*.json'))

dicionario_existente = None
if arquivos_dict:
    # Usar o mais recente
    dicionario_existente = max(
        arquivos_dict,
        key=lambda p: p.stat().st_mtime
    )
    print(f"📚 Dicionário detectado: {dicionario_existente.name}")

# Registrar no localizador com timestamp E versão
LocalizadorDicionario.registrar_mudanca(
    pasta_base=pasta_container,
    timestamp=timestamp,
    dicionario_path=dicionario_existente,  # None se não existir
    versao_bloco1=VERSAO_BLOCO1
)

# Gerar README
gerar_readme(pasta_container, VERSAO_BLOCO1)

# ═══════════════════════════════════════════════════════════════════
# MELHORIA 1: SALVAR ESTADO LOCAL (NOVO v4.4)
# ═══════════════════════════════════════════════════════════════════

print("\n" + "="*70)
print("🔵 ETAPA 6: SALVANDO ESTADO LOCAL...")
print("="*70)

# Calcular tamanho total do container
tamanho_total = sum(
    f.stat().st_size for f in pasta_container.rglob('*') if f.is_file()
) / 1024 / 1024

estado_bloco1 = {
    'bloco': 1,
    'versao': VERSAO_BLOCO1,
    'versao_codigo': VERSAO_BLOCO1,
    'data_versao': DATA_VERSAO,
    'timestamp_execucao': timestamp,
    'timestamp_registro': datetime.now().isoformat(),
    'status': 'concluido',
    'pasta_container': {
        'nome': pasta_container.name,
        'caminho': str(pasta_container),
        'tamanho_mb': round(tamanho_total, 2)
    },
    'filemanager': {
        'base_path': str(fm.base_path),
        'timestamp': fm.timestamp,
        'pastas_criadas': list(fm.pastas.keys())
    },
    'migracao': {
        'realizada': resultado_origem.get('copiar', False),
        'arquivos_migrados': info_mig.get('arquivos', 0),
        'pasta_origem': str(resultado_origem.get('path', '')) if resultado_origem.get('copiar') else None
    },
    'localizador': {
        'log_file': str(LocalizadorDicionario.LOG_FILE),
        'pasta_base_registrada': str(pasta_container),
        'timestamp_registrado': timestamp,
        'dicionario_registrado': str(dicionario_existente) if dicionario_existente else None
    },
    'poluicao_root': {
        'detectada': len(pastas_poluidas) if pastas_poluidas else 0,
        'acao_tomada': acao if pastas_poluidas else 'NENHUMA'
    },
    'changelog': CHANGELOG_V44
}

arquivo_estado = fm.pastas['logs'] / '.bloco_1_state.json'
with open(arquivo_estado, 'w', encoding='utf-8') as f:
    json.dump(estado_bloco1, f, indent=2, ensure_ascii=False)

print(f"✅ Estado local salvo")
print(f"   📄 {arquivo_estado.name}")
print(f"   📊 Tamanho container: {tamanho_total:.2f} MB")

# ═══════════════════════════════════════════════════════════════════
print("\n" + "="*70)
print("✅ BLOCO 1 v4.4 CONCLUÍDO COM SUCESSO")
print("="*70)
print(f"\n📂 Container: {pasta_container}")
print(f"🕐 Timestamp: {timestamp}")
print(f"📍 Localizador: {LocalizadorDicionario.LOG_FILE}")
print(f"🔖 Versão: {VERSAO_BLOCO1}")
print(f"\n📋 Estrutura criada:")
for nome, pasta in fm.pastas.items():
    print(f"   • {pasta.name}")
print(f"\n💾 Arquivos de estado:")
print(f"   • LOG GLOBAL: {LocalizadorDicionario.LOG_FILE.name}")
print(f"   • Estado local: {arquivo_estado.name}")
print(f"   • README: README.md")
print("\n" + "="*70)
print("\n💡 Próximo: BLOCO 2 vai ler configuração do LOG GLOBAL")
print("="*70)

 🔍 PROCESSADOR DE ARQUIVOS DESCONHECIDOS v4.4
 Versão: 4.4 | Data: 2025-10-17
 Timer | Migração | Dicionários | Validações | Logs | Estado

🔍 Validando dependências...
   ✅ pandas
   ✅ numpy
   ✅ openpyxl
   ✅ xlrd
   ✅ tkinter
✅ Todas as dependências instaladas!

✅ Imports carregados

🔵 ETAPA 1: SELECIONANDO PASTA DESTINO...

✅ Destino: E:\OneDrive - VIBRA\NMCV - Documentos\Indicador\_DataLake\2- Dados Processados (PROCESSED)
   Ação: MANTEVE

🔵 ETAPA 2: VERIFICANDO POLUIÇÃO NO ROOT...
✅ Root limpo

🔵 ETAPA 3: CRIANDO PASTA CONTAINER...
✅ Container: PROCESSAR_ARQUIVOS_20251019_060722

🔵 ETAPA 4: COPIAR DE EXECUÇÃO ANTERIOR?
✅ Começando do zero (sem cópia)

🔵 ETAPA 5: INICIALIZANDO FILEMANAGER...
✅ FileManager inicializado
   📂 Container: PROCESSAR_ARQUIVOS_20251019_060722
   🕐 Timestamp: 20251019_060722

📍 Localizador atualizado:
   Container: PROCESSAR_ARQUIVOS_20251019_060722
   Timestamp: 20251019_060722
   Versão BLOCO 1: 4.4
   Log: C:\Users\fpsou\.processador_dicionario_localiza

In [26]:
# ===================================================================
# BLOCO 2: CLASSES AUXILIARES
# Versao: 4.3 - REVISADO (COMUNICACAO VIA LOG COMPLETA)
# ===================================================================

import pandas as pd
import numpy as np
import tkinter as tk
from tkinter import filedialog, messagebox
from pathlib import Path
import json
import re
from datetime import datetime
import warnings
from collections import Counter
import os
import platform
import subprocess

warnings.filterwarnings('ignore')

print("\n" + "="*70)
print("BLOCO 2: CLASSES AUXILIARES v4.3 REVISADO")
print("="*70)

# ===================================================================
# CLASSE: LocalizadorDicionario (INTEGRADA DO BLOCO 1)
# ===================================================================

class LocalizadorDicionario:
    """
    Sistema de localizacao persistente de dicionarios entre
    sessoes.

    Mantem log global em: ~/.processador_dicionario_localizador.json
    """

    LOG_FILE = Path.home() / '.processador_dicionario_localizador.json'

    @classmethod
    def carregar_log(cls):
        """Carrega log global com fallback para encoding"""
        if cls.LOG_FILE.exists():
            for encoding in ['utf-8', 'utf-8-sig', 'latin-1']:
                try:
                    with open(cls.LOG_FILE, 'r', encoding=encoding) as f:
                        return json.load(f)
                except (UnicodeDecodeError, json.JSONDecodeError):
                    continue
        return {
            'versao': '2.0',
            'dicionario_atual': None,
            'pasta_base_atual': None,
            'historico': []
        }

    @classmethod
    def obter_dicionario_atual(cls):
        """Retorna Path do dicionario atual"""
        log = cls.carregar_log()
        if not log['dicionario_atual']:
            raise FileNotFoundError(
                "Dicionario nao encontrado! Execute BLOCO 1."
            )

        dicionario_path = Path(log['dicionario_atual'])
        if not dicionario_path.exists():
            raise FileNotFoundError(
                f"Dicionario nao existe: {dicionario_path}"
            )

        return dicionario_path

    @classmethod
    def obter_pasta_base_atual(cls):
        """Retorna Path da pasta base atual"""
        log = cls.carregar_log()
        if not log['pasta_base_atual']:
            raise FileNotFoundError(
                "Pasta base nao encontrada! Execute BLOCO 1."
            )

        pasta_base = Path(log['pasta_base_atual'])
        if not pasta_base.exists():
            raise FileNotFoundError(
                f"Pasta base nao existe: {pasta_base}"
            )

        return pasta_base

# ===================================================================
# CLASSE: FileManagerInterativo (INTEGRADA DO BLOCO 1)
# ===================================================================

class FileManagerInterativo:
    """Gerenciador de arquivos e estrutura de pastas"""

    def __init__(self, base_path):
        self.base_path = Path(base_path)

        # Estrutura de pastas padrao
        self.pastas = {
            'entrada': self.base_path / '01_Entrada',
            'processados': self.base_path / '02_Processados',
            'outputs': self.base_path / '03_Outputs',
            'logs': self.base_path / '04_Logs',
            'dicionarios': self.base_path / '05_Dicionarios',
            'codigos_integracao': self.base_path / '06_Codigos_Integracao'
        }

        # Criar todas as pastas
        for pasta in self.pastas.values():
            pasta.mkdir(parents=True, exist_ok=True)

        self.timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

    def salvar(self, df, nome, tipo='xlsx', pasta='processados'):
        """Salva DataFrame na pasta especificada"""
        arquivo = self.pastas[pasta] / f"{nome}_{self.timestamp}.{tipo}"

        if tipo == 'xlsx':
            df.to_excel(arquivo, index=False, engine='openpyxl')
        elif tipo == 'csv':
            df.to_csv(arquivo, index=False, encoding='utf-8-sig')

        return arquivo

    def abrir_pasta(self, pasta):
        """Abre pasta no explorer do sistema"""
        caminho = self.pastas[pasta]
        sistema = platform.system()

        try:
            if sistema == 'Windows':
                os.startfile(caminho)
            elif sistema == 'Darwin':  # macOS
                subprocess.run(['open', caminho])
            else:  # Linux
                subprocess.run(['xdg-open', caminho])
        except Exception as e:
            print(f"Nao foi possivel abrir pasta: {e}")

# ===================================================================
# CLASSE: SeletorArquivo (GUI COM TIMER E VALIDACOES)
# ===================================================================

class SeletorArquivo:
    """Seletor de arquivo com timer de 10s e validacoes robustas"""

    CONFIG_FILE = Path.home() / '.processador_last_file.json'

    def __init__(self):
        self.resultado = {'path': None, 'acao': None}
        self.timeout_ocorreu = False

    def carregar_ultimo_arquivo(self):
        """Carrega ultimo arquivo usado"""
        if self.CONFIG_FILE.exists():
            try:
                with open(self.CONFIG_FILE, 'r', encoding='utf-8') as f:
                    config = json.load(f)
                ultimo_arquivo = Path(config.get('last_file', ''))
                if ultimo_arquivo.exists():
                    return ultimo_arquivo
            except:
                pass
        return None

    def salvar_escolha(self, arquivo):
        """Salva escolha para proxima execucao"""
        config = {
            'last_file': str(arquivo),
            'timestamp': datetime.now().isoformat()
        }
        with open(self.CONFIG_FILE, 'w', encoding='utf-8') as f:
            json.dump(config, f, indent=2)

    def validar_arquivo(self, arquivo_path):
        """Valida se arquivo e adequado para processamento"""
        arquivo = Path(arquivo_path)

        # Verificar existencia
        if not arquivo.exists():
            return False, "Arquivo nao existe"

        # Verificar se e arquivo (nao diretorio)
        if not arquivo.is_file():
            return False, "Nao e um arquivo"

        # Verificar permissao de leitura
        if not os.access(arquivo, os.R_OK):
            return False, "Sem permissao de leitura"

        # Verificar tamanho (maximo 500MB)
        tamanho_mb = arquivo.stat().st_size / (1024 * 1024)
        if tamanho_mb > 500:
            return False, f"Arquivo muito grande ({tamanho_mb:.1f}MB)"

        # Verificar extensao
        extensoes_validas = {'.xlsx', '.xls', '.csv', '.txt'}
        if arquivo.suffix.lower() not in extensoes_validas:
            return False, f"Extensao invalida ({arquivo.suffix})"

        return True, "Arquivo valido"

    def selecionar_com_timer(self):
        """Exibe GUI com timer de 10s"""
        ultimo_arquivo = self.carregar_ultimo_arquivo()

        root = tk.Tk()
        root.title("Processador - Selecionar Arquivo")
        root.geometry("650x450")

        # Centralizar janela
        x = (root.winfo_screenwidth() // 2) - 325
        y = (root.winfo_screenheight() // 2) - 225
        root.geometry(f"+{x}+{y}")

        frame = tk.Frame(root, padx=20, pady=20, bg='white')
        frame.pack(fill=tk.BOTH, expand=True)

        # Titulo
        tk.Label(
            frame,
            text="Selecionar Arquivo",
            font=('Arial', 14, 'bold'),
            bg='white'
        ).pack(pady=(0, 15))

        # Mensagem
        if ultimo_arquivo:
            msg = f"Timer de 10s para usar:\n\n{ultimo_arquivo.name}"
        else:
            msg = "Primeira execucao - selecione arquivo"

        tk.Label(
            frame,
            text=msg,
            justify=tk.LEFT,
            font=('Arial', 9),
            bg='white',
            wraplength=600
        ).pack(pady=(0, 15))

        # Timer
        contador = [15]
        if ultimo_arquivo:
            label_timer = tk.Label(
                frame,
                text=f"{contador[0]}s",
                font=('Arial', 24, 'bold'),
                fg='#FF4444',
                bg='white'
            )
            label_timer.pack(pady=(5, 20))

            def countdown():
                if contador[0] > 0 and not self.timeout_ocorreu:
                    contador[0] -= 1
                    label_timer.config(text=f"{contador[0]}s")
                    root.after(1000, countdown)
                elif contador[0] == 0:
                    self.timeout_ocorreu = True
                    self.resultado['path'] = ultimo_arquivo
                    self.resultado['acao'] = 'TIMEOUT'
                    root.quit()
                    root.destroy()

            root.after(1000, countdown)

        # Botoes
        def escolher_novo():
            self.timeout_ocorreu = True
            root.withdraw()

            novo_arquivo = filedialog.askopenfilename(
                title="Selecionar Arquivo",
                initialdir=ultimo_arquivo.parent if ultimo_arquivo else None,
                filetypes=[
                    ("Arquivos suportados", "*.xlsx;*.xls;*.csv;*.txt"),
                    ("Excel", "*.xlsx;*.xls"),
                    ("CSV", "*.csv"),
                    ("Todos", "*.*")
                ]
            )

            if novo_arquivo:
                valido, msg = self.validar_arquivo(novo_arquivo)
                if not valido:
                    messagebox.showerror("Arquivo Invalido", msg)
                    self.resultado['path'] = ultimo_arquivo
                    self.resultado['acao'] = 'CANCELADO'
                else:
                    self.resultado['path'] = Path(novo_arquivo)
                    self.resultado['acao'] = 'NOVO'
            else:
                self.resultado['path'] = ultimo_arquivo
                self.resultado['acao'] = 'CANCELADO'

            root.quit()
            root.destroy()

        def usar_ultimo():
            self.timeout_ocorreu = True
            self.resultado['path'] = ultimo_arquivo
            self.resultado['acao'] = 'MANTEVE'
            root.quit()
            root.destroy()

        frame_btns = tk.Frame(frame, bg='white')
        frame_btns.pack(side=tk.BOTTOM, pady=15)

        tk.Button(
            frame_btns,
            text="Novo Arquivo",
            command=escolher_novo,
            width=20,
            height=2,
            font=('Arial', 10, 'bold'),
            bg='#4CAF50',
            fg='white'
        ).pack(side=tk.LEFT, padx=10)

        if ultimo_arquivo:
            tk.Button(
                frame_btns,
                text="Usar Ultimo",
                command=usar_ultimo,
                width=20,
                height=2,
                font=('Arial', 10),
                bg='#2196F3',
                fg='white'
            ).pack(side=tk.LEFT, padx=10)

        root.mainloop()
        return self.resultado

# ===================================================================
# CLASSE: DetectorCabecalho (ANALISE INTELIGENTE COM LOG)
# ===================================================================

class DetectorCabecalho:
    """
    Detecta automaticamente a linha de cabecalho em arquivos.

    Usa sistema de scoring baseado em:
    - Preenchimento (70%+ colunas com dados)
    - Tipo String (80%+ colunas texto)
    - Valores unicos (indicador de rotulos)
    - Palavras-chave tipicas de cabecalho
    - Posicao na planilha (primeiras linhas tem prioridade)
    """

    # CONFIGURACAO EXTERNALIZAVEL
    PALAVRAS_CHAVE_PADRAO = [
        'codigo', 'nome', 'descri', 'data', 'valor', 'quantidade',
        'centro', 'produto', 'material', 'sigla', 'tipo', 'grupo'
    ]

    def __init__(self, df, palavras_chave=None):
        self.df = df
        self.scores = []
        self.log_decisoes = []
        self.palavras_chave = (
            palavras_chave if palavras_chave
            else self.PALAVRAS_CHAVE_PADRAO
        )

    def detectar(self, n_linhas=50):
        """
        Analisa primeiras n linhas e retorna indice do cabecalho.

        Args:
            n_linhas: Numero de linhas a analisar

        Returns:
            dict: {
                'indice': int,  # Linha detectada como cabecalho
                'score': float,  # Confianca da deteccao (0-1)
                'metodo': str,   # Como foi detectado
                'scores_todas_linhas': list,  # Para debug
                'log_decisoes': list  # Historico de analise
            }
        """
        n_linhas = min(n_linhas, len(self.df))

        for i in range(n_linhas):
            linha = self.df.iloc[i]
            score = 0
            detalhes = {'linha': i, 'criterios': {}}

            # Criterio 1: Preenchimento (30 pontos)
            preenchimento = linha.notna().sum() / len(linha)
            if preenchimento >= 0.7:
                pontos = 30 * (preenchimento - 0.7) / 0.3
                score += pontos
                detalhes['criterios']['preenchimento'] = (
                    f"{preenchimento:.1%} (+{pontos:.1f})"
                )

            # Criterio 2: Tipo String (30 pontos)
            tipos_string = sum(isinstance(v, str) for v in linha)
            proporcao_string = tipos_string / len(linha)
            if proporcao_string >= 0.8:
                pontos = 30 * (proporcao_string - 0.8) / 0.2
                score += pontos
                detalhes['criterios']['strings'] = (
                    f"{proporcao_string:.1%} (+{pontos:.1f})"
                )

            # Criterio 3: Valores unicos (20 pontos)
            valores_unicos = len(
                set(str(v) for v in linha if pd.notna(v))
            )
            if valores_unicos >= len(linha) * 0.8:
                pontos = 20
                score += pontos
                detalhes['criterios']['unicos'] = (
                    f"{valores_unicos}/{len(linha)} (+{pontos})"
                )

            # Criterio 4: Palavras-chave (10 pontos)
            texto_linha = ' '.join(
                str(v).lower() for v in linha if pd.notna(v)
            )
            palavras_encontradas = sum(
                1 for p in self.palavras_chave if p in texto_linha
            )
            if palavras_encontradas > 0:
                pontos = min(10, palavras_encontradas * 3)
                score += pontos
                detalhes['criterios']['palavras'] = (
                    f"{palavras_encontradas} palavras (+{pontos})"
                )

            # Criterio 5: Posicao (10 pontos)
            # Primeiras linhas tem vantagem
            if i < 50:
                pontos = 10 * (1 - (i / 50))
                score += pontos
                detalhes['criterios']['posicao'] = (
                    f"linha {i} (+{pontos:.1f})"
                )

            detalhes['score_total'] = score
            self.scores.append(score)
            self.log_decisoes.append(detalhes)

        # Encontrar melhor score
        melhor_indice = self.scores.index(max(self.scores))
        melhor_score = self.scores[melhor_indice]

        # Normalizar score para 0-1
        score_normalizado = min(1.0, melhor_score / 100)

        resultado = {
            'indice': melhor_indice,
            'score': score_normalizado,
            'metodo': 'SCORING_AUTOMATICO',
            'scores_todas_linhas': self.scores,
            'log_decisoes': self.log_decisoes
        }

        return resultado

# ===================================================================
# INICIALIZACAO DO FILEMANAGER (CONECTANDO COM BLOCO 1)
# ===================================================================

print("\n" + "="*70)
print("INICIALIZANDO FILEMANAGER - CONECTANDO COM BLOCO 1")
print("="*70)

try:
    pasta_base = LocalizadorDicionario.obter_pasta_base_atual()

    print(f"Container do BLOCO 1 encontrado!")
    print(f"   {pasta_base}")

    fm = FileManagerInterativo(pasta_base)

except FileNotFoundError as e:
    print(f"\n{e}")
    print("\nATENCAO: Execute o BLOCO 1 primeiro!")
    raise

# ===================================================================
# SALVAR ESTADO DO BLOCO 2 NO LOG
# ===================================================================

estado_bloco2 = {
    'bloco': 2,
    'versao': '4.3',
    'status': 'concluido',
    'timestamp': datetime.now().isoformat(),
    'classes_carregadas': [
        'LocalizadorDicionario',
        'FileManagerInterativo',
        'SeletorArquivo',
        'DetectorCabecalho'
    ],
    'filemanager': {
        'base_path': str(fm.base_path),
        'timestamp': fm.timestamp
    }
}

arquivo_estado = fm.pastas['logs'] / '.bloco_2_state.json'
with open(arquivo_estado, 'w', encoding='utf-8') as f:
    json.dump(estado_bloco2, f, indent=2, ensure_ascii=False)

print("\n" + "="*70)
print("BLOCO 2 CONCLUIDO")
print("="*70)
print("\nClasses carregadas:")
print("   LocalizadorDicionario ........... OK")
print("   FileManagerInterativo ........... OK")
print("   SeletorArquivo .................. OK")
print("   DetectorCabecalho ............... OK")
print("\nFileManager ativo:")
print(f"   Base: {fm.base_path}")
print(f"   Timestamp: {fm.timestamp}")
print("\nEstrutura de pastas:")
for nome, pasta in fm.pastas.items():
    print(f"   {nome.ljust(20)}: {pasta.name}")
print("\nEstado salvo:")
print(f"   {arquivo_estado.name}")
print("\n" + "="*70)
print("Digite 'BLOCO 2 OK' para prosseguir ao BLOCO 3")
print("="*70)


BLOCO 2: CLASSES AUXILIARES v4.3 REVISADO

INICIALIZANDO FILEMANAGER - CONECTANDO COM BLOCO 1
Container do BLOCO 1 encontrado!
   E:\OneDrive - VIBRA\NMCV - Documentos\Indicador\_DataLake\2- Dados Processados (PROCESSED)\PROCESSAR_ARQUIVOS_20251019_060722

BLOCO 2 CONCLUIDO

Classes carregadas:
   LocalizadorDicionario ........... OK
   FileManagerInterativo ........... OK
   SeletorArquivo .................. OK
   DetectorCabecalho ............... OK

FileManager ativo:
   Base: E:\OneDrive - VIBRA\NMCV - Documentos\Indicador\_DataLake\2- Dados Processados (PROCESSED)\PROCESSAR_ARQUIVOS_20251019_060722
   Timestamp: 20251019_060725

Estrutura de pastas:
   entrada             : 01_Entrada
   processados         : 02_Processados
   outputs             : 03_Outputs
   logs                : 04_Logs
   dicionarios         : 05_Dicionarios
   codigos_integracao  : 06_Codigos_Integracao

Estado salvo:
   .bloco_2_state.json

Digite 'BLOCO 2 OK' para prosseguir ao BLOCO 3


In [27]:
# ===================================================================
# BLOCO 3: DICIONÁRIO INTELIGENTE + CLASSE GUI COM TIMER
# Versão: v4.5 - Dicionário + GUIComTimer (seleção no BLOCO 4)
# ===================================================================

import pandas as pd
import numpy as np
import json
import re
import tkinter as tk
from pathlib import Path
from datetime import datetime

print("\n" + "="*70)
print("BLOCO 3: DICIONÁRIO INTELIGENTE + GUI COM TIMER")
print("="*70)

# ===================================================================
# 1. LER CONFIGURAÇÕES DO BLOCO ANTERIOR (VIA LOG)
# ===================================================================

log_global = Path.home() / '.processador_dicionario_localizador.json'

if not log_global.exists():
    raise FileNotFoundError(
        "❌ LOG GLOBAL não encontrado!\n"
        "   Execute BLOCO 1 primeiro."
    )

with open(log_global, 'r', encoding='utf-8') as f:
    config = json.load(f)

pasta_base = Path(config['pasta_base_atual'])
timestamp_execucao = config['timestamp']

print(f"\n✅ CONFIGURAÇÃO CARREGADA DO LOG GLOBAL")
print(f"   📁 Pasta base: {pasta_base.name}")
print(f"   🕐 Timestamp: {timestamp_execucao}")

# ===================================================================
# 2. VALIDAR QUE BLOCO 2 FOI EXECUTADO
# ===================================================================

log_bloco2 = pasta_base / '04_Logs' / '.bloco_2_state.json'

if not log_bloco2.exists():
    raise FileNotFoundError(
        "❌ BLOCO 2 não foi executado!\n"
        "   Execute BLOCO 2 primeiro."
    )

with open(log_bloco2, 'r', encoding='utf-8') as f:
    estado_bloco2 = json.load(f)

print(f"\n✅ BLOCO 2 VALIDADO")
print(f"   Executado em: {estado_bloco2['timestamp']}")
print(f"   Classes: {', '.join(estado_bloco2['classes_carregadas'])}")

# ===================================================================
# 3. RECRIAR FILEMANAGER (NÃO ASSUMIR MEMÓRIA)
# ===================================================================

class FileManagerInterativo:
    """Gerenciador de arquivos"""
    def __init__(self, base_path):
        self.base_path = Path(base_path)
        self.pastas = {
            'entrada': self.base_path / '01_Entrada',
            'processados': self.base_path / '02_Processados',
            'outputs': self.base_path / '03_Outputs',
            'logs': self.base_path / '04_Logs',
            'dicionarios': self.base_path / '05_Dicionarios',
            'codigos': self.base_path / '06_Codigos_Integracao'
        }

fm = FileManagerInterativo(pasta_base)
print(f"\n✅ FileManager recriado: {fm.base_path.name}")

# ===================================================================
# 4. CLASSE GUI COM TIMER (ex-BLOCO 4)
# ===================================================================

class GUIComTimer:
    """Implementa timer de 10s com countdown visual"""

    @staticmethod
    def criar_janela_com_timer(titulo, largura, altura, tem_timer=True):
        """Cria janela base com timer"""
        root = tk.Tk()
        root.title(titulo)
        root.geometry(f"{largura}x{altura}")
        root.resizable(False, False)

        # Centralizar
        x = (root.winfo_screenwidth() // 2) - (largura // 2)
        y = (root.winfo_screenheight() // 2) - (altura // 2)
        root.geometry(f"+{x}+{y}")
        root.attributes('-topmost', True)
        root.after(100, lambda: root.attributes('-topmost', False))

        frame = tk.Frame(root, padx=20, pady=20, bg='white')
        frame.pack(fill=tk.BOTH, expand=True)

        resultado = {'valor': None, 'cancelado': False, 'timeout': False}
        contador = [10] if tem_timer else [0]

        return root, frame, resultado, contador

    @staticmethod
    def adicionar_timer(frame, root, resultado, contador):
        """Adiciona timer visual"""
        label_timer = tk.Label(
            frame,
            text=f"⏱️  {contador[0]}s",
            font=('Arial', 16, 'bold'),
            fg='#FF4444',
            bg='white'
        )
        label_timer.pack(pady=(5, 15))

        def countdown():
            if contador[0] > 0 and not resultado['cancelado']:
                contador[0] -= 1
                label_timer.config(text=f"⏱️  {contador[0]}s")
                root.after(1000, countdown)
            elif contador[0] == 0 and not resultado['cancelado']:
                resultado['timeout'] = True
                root.quit()
                root.destroy()

        return countdown

    @staticmethod
    def criar_botoes(frame, cmd_principal, cmd_secundario=None,
                     label_principal="Confirmar",
                     label_secundario="Usar Último"):
        """Cria botões padronizados"""
        tk.Frame(frame, height=2, bg='#CCCCCC').pack(
            fill=tk.X, pady=10
        )

        frame_btns = tk.Frame(frame, bg='white')
        frame_btns.pack(side=tk.BOTTOM, pady=10)

        tk.Button(
            frame_btns,
            text=label_principal,
            command=cmd_principal,
            width=18,
            height=2,
            bg='#4CAF50',
            fg='white',
            font=('Arial', 10, 'bold'),
            cursor='hand2'
        ).pack(side=tk.LEFT, padx=5)

        if cmd_secundario:
            tk.Button(
                frame_btns,
                text=label_secundario,
                command=cmd_secundario,
                width=18,
                height=2,
                bg='#2196F3',
                fg='white',
                font=('Arial', 10),
                cursor='hand2'
            ).pack(side=tk.LEFT, padx=5)

print("\n✅ Classe GUIComTimer carregada")

# ===================================================================
# 5. DICIONÁRIO INTELIGENTE
# ===================================================================

class DicionarioInteligente:
    """Dicionário com detecção avançada"""

    def __init__(self, fm):
        self.fm = fm
        self.arquivo_dict = fm.pastas['logs'] / 'DICT_Dicionario_Persistente.json'
        self.dados = self._carregar_ou_criar()

    def _carregar_ou_criar(self):
        if self.arquivo_dict.exists():
            try:
                with open(self.arquivo_dict, 'r', encoding='utf-8') as f:
                    dados = json.load(f)

                if 'campos_conhecidos' not in dados:
                    dados = self._migrar_formato_antigo(dados)
                    self._salvar(dados)

                n_campos = len(dados['campos_conhecidos'])
                n_arquivos = len(dados.get('historico_arquivos', []))

                print(f"\n✅ DICIONÁRIO PERSISTENTE CARREGADO")
                print(f"   📚 {n_campos} campos conhecidos")
                print(f"   📁 {n_arquivos} arquivos processados")

                return dados

            except Exception as e:
                print(f"\n⚠️  Erro ao carregar: {e}")
                print("   Criando novo dicionário...")
                dados = self._criar_novo()
                self._salvar(dados)
                return dados
        else:
            print(f"\n📝 CRIANDO NOVO DICIONÁRIO...")
            dados = self._criar_novo()
            self._salvar(dados)
            print(f"✅ Dicionário criado: {len(dados['campos_conhecidos'])} campos")
            return dados

    def _migrar_formato_antigo(self, dados_antigos):
        novo = self._criar_novo()
        if 'arquivos_processados' in dados_antigos:
            novo['historico_arquivos'] = dados_antigos['arquivos_processados']
        return novo

    def _criar_novo(self):
        """Dicionário com 22 campos padrão"""
        return {
            'versao': '4.5',
            'criado_em': datetime.now().isoformat(),
            'ultima_atualizacao': datetime.now().isoformat(),
            'config_sistema': {
                'timeout_sessao_minutos': 60,
                'padroes_csv_detectados': []
            },
            'campos_conhecidos': {
                'Centro': {
                    'tipo_dado': 'Codigo_Centro',
                    'regex': r'^[5-9]\d{3}$',
                    'sinonimos': ['Centro', 'Código de Centro', 'Cod Centro',
                                  'Unidade Operacional:Centro'],
                    'exemplos': ['5025', '5065', '5174'],
                    'descricao': 'Código numérico de 4 dígitos',
                    'aprendido_de': 'PADRAO_INICIAL'
                },
                'Sigla': {
                    'tipo_dado': 'Sigla_Base',
                    'regex': r'^[A-Z]{4,10}$',
                    'sinonimos': ['Sigla', 'Sigla Base', 'Sigla Centro', 'Base'],
                    'exemplos': ['BABET', 'BAPLAN', 'AIBET'],
                    'descricao': 'Sigla alfabética (4-10 letras maiúsculas)',
                    'aprendido_de': 'PADRAO_INICIAL'
                },
                'Codigo_Produto': {
                    'tipo_dado': 'Codigo_Material',
                    'regex': r'^\d{1,2}\.\d{3}\.\d{3}$|^\d{7,8}$',
                    'sinonimos': ['Código Produto', 'Código Material',
                                  'Cod Produto', 'Cod Material'],
                    'exemplos': ['10.123.456', '1.234.567', '1234567'],
                    'descricao': 'Código numérico do material/produto',
                    'aprendido_de': 'PADRAO_INICIAL'
                },
                'Codigo_Grupo_Produto': {
                    'tipo_dado': 'Codigo_Grupo',
                    'regex': r'^\d{1,2}\.\d{3}\.\d{3}$|^\d{7,8}$|^[A-Z_]+$',
                    'sinonimos': ['Cód Grupo de produto', 'Código Grupo',
                                  'Grupo Produto', 'Produto:CodGrupoProduto'],
                    'exemplos': ['10.123.456', 'DIESEL_S10_SIMPLES',
                                 'GASOLINA_COMUM'],
                    'descricao': 'Código grupo (numérico OU texto_underscore)',
                    'aprendido_de': 'PADRAO_INICIAL'
                },
                'Desc_Grupo_Produto': {
                    'tipo_dado': 'Texto_Descricao',
                    'regex': r'^[A-Za-z0-9\s\-]+$',
                    'sinonimos': ['Desc. Grupo de Produto', 'Descrição Produto',
                                  'Nome Produto', 'Produto'],
                    'exemplos': ['DIESEL S10', 'GASOLINA COMUM',
                                 'ETANOL HIDRATADO'],
                    'descricao': 'Descrição textual do grupo de produto',
                    'aprendido_de': 'PADRAO_INICIAL'
                },
                'Nome_Pessoa': {
                    'tipo_dado': 'Texto_Nome_Pessoa',
                    'regex': r'^[A-ZÁÀÂÃÉÈÊÍÏÓÔÕÖÚÇÑ][a-záàâãéèêíïóôõöúçñ]+(\s[A-ZÁÀÂÃÉÈÊÍÏÓÔÕÖÚÇÑ][a-záàâãéèêíïóôõöúçñ]+)+$',
                    'sinonimos': ['Criado por', 'Nome', 'Responsável',
                                  'Solicitante'],
                    'exemplos': ['Kenedy Vinícius Rodrigues',
                                 'Joao Carlos Stival'],
                    'descricao': 'Nome completo de pessoa',
                    'aprendido_de': 'PADRAO_INICIAL'
                },
                'Email': {
                    'tipo_dado': 'Texto_Email',
                    'regex': r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$',
                    'sinonimos': ['Email', 'E-mail', 'Modificado por'],
                    'exemplos': ['usuario@vibraenergia.com.br'],
                    'descricao': 'Endereço de email',
                    'aprendido_de': 'PADRAO_INICIAL'
                },
                'Status_Workflow': {
                    'tipo_dado': 'Texto_Status',
                    'regex': r'^(Em aprovação|Aprovado|Rejeitado|Pendente|Concluído|Ítem Criado|Item Criado)$',
                    'sinonimos': ['Status', 'Status Aprovação', 'Situação'],
                    'exemplos': ['Em aprovação', 'Ítem Criado', 'Aprovado'],
                    'descricao': 'Status de workflow/aprovação',
                    'aprendido_de': 'PADRAO_INICIAL'
                },
                'Booleano_Texto': {
                    'tipo_dado': 'Texto_Booleano',
                    'regex': r'^(Sim|Não|sim|não|SIM|NÃO|Yes|No|TRUE|FALSE)$',
                    'sinonimos': ['Concluído?', 'Ativo?', 'Habilitado?'],
                    'exemplos': ['Sim', 'Não'],
                    'descricao': 'Valor booleano como texto',
                    'aprendido_de': 'PADRAO_INICIAL'
                },
                'Texto_Longo': {
                    'tipo_dado': 'Texto_Justificativa',
                    'regex': r'^.{50,}$',
                    'sinonimos': ['Justificativa', 'Observação', 'Comentário'],
                    'exemplos': ['Solicitamos a revisão do limite...'],
                    'descricao': 'Texto longo (mais de 50 caracteres)',
                    'aprendido_de': 'PADRAO_INICIAL'
                },
                'Path_URL': {
                    'tipo_dado': 'Texto_Caminho',
                    'regex': r'^(teams/|http|https|ftp|\\\\|/).*',
                    'sinonimos': ['Caminho', 'Path', 'URL', 'Link'],
                    'exemplos': ['teams/portaleso/Lists/...',
                                 'https://example.com'],
                    'descricao': 'Caminho de arquivo ou URL',
                    'aprendido_de': 'PADRAO_INICIAL'
                },
                'Sigla_Curta': {
                    'tipo_dado': 'Texto_Sigla_Curta',
                    'regex': r'^[A-Z]{2,4}$',
                    'sinonimos': ['CME', 'Gerência', 'UF', 'Tipo'],
                    'exemplos': ['OPC', 'OPN', 'Norte', 'Sul', 'CME'],
                    'descricao': 'Sigla curta (2-4 letras maiúsculas)',
                    'aprendido_de': 'PADRAO_INICIAL'
                },
                'Tipo_Item': {
                    'tipo_dado': 'Texto_Tipo_Item',
                    'regex': r'^(Item|Documento|Pasta|Arquivo)$',
                    'sinonimos': ['Tipo de Item', 'Tipo'],
                    'exemplos': ['Item'],
                    'descricao': 'Tipo de item em lista SharePoint',
                    'aprendido_de': 'PADRAO_INICIAL'
                },
                'Data_ISO': {
                    'tipo_dado': 'Data_YYYY-MM-DD',
                    'regex': r'^\d{4}-\d{2}-\d{2}$',
                    'sinonimos': ['Data', 'Período',
                                  'Período Início Validade Novo Limite'],
                    'exemplos': ['2025-08-01', '2024-12-31', '2025-01-07'],
                    'descricao': 'Data formato ISO (YYYY-MM-DD)',
                    'aprendido_de': 'PADRAO_INICIAL'
                },
                'Datetime_ISO': {
                    'tipo_dado': 'Datetime_YYYY-MM-DD_HH:MM:SS',
                    'regex': r'^\d{4}-\d{2}-\d{2}\s\d{2}:\d{2}:\d{2}$',
                    'sinonimos': ['Criado', 'Modificado', 'Data Hora',
                                  'Timestamp'],
                    'exemplos': ['2025-08-04 19:22:17',
                                 '2025-08-04 20:45:37'],
                    'descricao': 'Data e hora formato ISO',
                    'aprendido_de': 'PADRAO_INICIAL'
                },
                'Data_BR': {
                    'tipo_dado': 'Data_DD/MM/YYYY',
                    'regex': r'^\d{2}/\d{2}/\d{4}$',
                    'sinonimos': ['Data'],
                    'exemplos': ['15/01/2024', '31/12/2025'],
                    'descricao': 'Data formato brasileiro',
                    'aprendido_de': 'PADRAO_INICIAL'
                },
                'Percentual_Decimal': {
                    'tipo_dado': 'Numero_Percentual_Decimal',
                    'regex': r'^-?\d+(\.\d+)?$',
                    'sinonimos': ['Limite Inferior Atual',
                                  'Limite Superior Atual', 'AVG VI %',
                                  '% VI', 'AVG VI % 2024 SAP'],
                    'exemplos': ['-0.08', '0.08', '-0.14', '0.05', '-0.03'],
                    'descricao': 'Percentual em decimal (pode ser negativo)',
                    'aprendido_de': 'PADRAO_INICIAL'
                },
                'Percentual_Com_Simbolo': {
                    'tipo_dado': 'Numero_Percentual_Simbolo',
                    'regex': r'^-?\d+(\.\d+)?%$',
                    'sinonimos': ['Percentual', '% VI'],
                    'exemplos': ['10.5%', '-5.2%', '100%'],
                    'descricao': 'Percentual com símbolo %',
                    'aprendido_de': 'PADRAO_INICIAL'
                },
                'Numero_Inteiro': {
                    'tipo_dado': 'Numero_Inteiro',
                    'regex': r'^-?\d+$',
                    'sinonimos': ['Quantidade', 'Qtd', 'Total'],
                    'exemplos': ['123', '-456', '1000'],
                    'descricao': 'Número inteiro',
                    'aprendido_de': 'PADRAO_INICIAL'
                },
                'Monetario': {
                    'tipo_dado': 'Numero_Monetario',
                    'regex': r'^R\$\s?-?\d{1,3}(\.\d{3})*(,\d{2})?$|^-?\d+([.,]\d{2})?$',
                    'sinonimos': ['Valor', 'Custo', 'Preço', 'R$'],
                    'exemplos': ['R$ 1.234,56', '1234.56'],
                    'descricao': 'Valor monetário',
                    'aprendido_de': 'PADRAO_INICIAL'
                },
                'Unidade_Operacional_Nome': {
                    'tipo_dado': 'Texto_Unidade_Operacional',
                    'regex': r'^[A-Z]{4,10}\s+Base\s+de\s+.+$',
                    'sinonimos': ['Unidade Operacional', 'Nome Base'],
                    'exemplos': ['BABET Base de Betim',
                                 'BAPLAN Base de Paulínia'],
                    'descricao': 'Nome completo da unidade operacional',
                    'aprendido_de': 'PADRAO_INICIAL'
                },
                'Rotulo_Retencao': {
                    'tipo_dado': 'Texto_Rotulo_Vazio',
                    'regex': r'^(NaN|nan|None|null|)$',
                    'sinonimos': ['Rótulo de retenção Aplicado'],
                    'exemplos': ['NaN'],
                    'descricao': 'Campo de rótulo (geralmente vazio)',
                    'aprendido_de': 'PADRAO_INICIAL'
                }
            },
            'historico_arquivos': []
        }

    def detectar_campo(self, coluna_nome, valores_amostra):
        """Detecção AVANÇADA com múltiplas estratégias"""
        valores_str = [str(v).strip() for v in valores_amostra
                      if pd.notna(v) and str(v).strip() not in
                      ['', 'nan', 'None']]

        if not valores_str:
            return {
                'campo_detectado': 'VAZIO',
                'confianca': 0.0,
                'score_conteudo': 0.0,
                'score_nome': 0.0,
                'matches': 0,
                'total': 0,
                'ambiguidade': False,
                'candidatos': [],
                'metodo': 'VAZIO'
            }

        # Heurísticas específicas
        campo_heuristico = self._detectar_por_heuristica(
            coluna_nome, valores_str
        )

        if campo_heuristico and campo_heuristico.get('confianca', 0) >= 0.85:
            campo_heuristico.setdefault('score_nome', 0.0)
            campo_heuristico.setdefault('score_conteudo',
                                        campo_heuristico.get('confianca', 0.0))
            campo_heuristico.setdefault('matches', len(valores_str))
            campo_heuristico.setdefault('total', len(valores_str))
            campo_heuristico.setdefault('ambiguidade', False)
            campo_heuristico.setdefault('candidatos', [])
            return campo_heuristico

        # Match por regex
        resultados_regex = []
        for nome_campo, info in self.dados['campos_conhecidos'].items():
            matches = sum(1 for v in valores_str
                         if re.match(info['regex'], v))
            score_conteudo = matches / len(valores_str)

            score_nome = 0.0
            for sinonimo in info['sinonimos']:
                if sinonimo.lower() in coluna_nome.lower():
                    score_nome = 0.3
                    break

            score_final = score_conteudo + score_nome

            resultados_regex.append({
                'campo': nome_campo,
                'score': min(score_final, 1.0),
                'score_conteudo': score_conteudo,
                'score_nome': score_nome,
                'matches': matches,
                'total': len(valores_str)
            })

        resultados_regex = sorted(resultados_regex,
                                 key=lambda x: x['score'],
                                 reverse=True)
        melhor_regex = resultados_regex[0]
        segundo_regex = resultados_regex[1] if len(resultados_regex) > 1 else None

        ambiguidade = False
        candidatos = []
        if segundo_regex and abs(melhor_regex['score'] -
                                segundo_regex['score']) < 0.10:
            ambiguidade = True
            candidatos = [segundo_regex['campo']]

        resultado_final = {
            'campo_detectado': melhor_regex['campo'],
            'confianca': melhor_regex['score'],
            'score_conteudo': melhor_regex['score_conteudo'],
            'score_nome': melhor_regex['score_nome'],
            'matches': melhor_regex['matches'],
            'total': melhor_regex['total'],
            'ambiguidade': ambiguidade,
            'candidatos': candidatos,
            'metodo': 'REGEX'
        }

        if resultado_final['confianca'] < 0.50:
            resultado_final['campo_detectado'] = 'DESCONHECIDO'

        return resultado_final

    def _detectar_por_heuristica(self, nome_coluna, valores_str):
        """Detecção por heurísticas específicas"""
        if not valores_str:
            return None

        tamanho_medio = sum(len(v) for v in valores_str) / len(valores_str)
        valores_unicos = set(valores_str)

        if all(re.match(r'^\d{4}-\d{2}-\d{2}$', v) for v in valores_str[:5]):
            return {'campo_detectado': 'Data_ISO', 'confianca': 0.95,
                   'metodo': 'HEURISTICA_DATA_ISO'}

        if all(re.match(r'^\d{4}-\d{2}-\d{2}\s\d{2}:\d{2}:\d{2}', v)
               for v in valores_str[:5]):
            return {'campo_detectado': 'Datetime_ISO', 'confianca': 0.95,
                   'metodo': 'HEURISTICA_DATETIME'}

        if all('@' in v for v in valores_str):
            return {'campo_detectado': 'Email', 'confianca': 0.95,
                   'metodo': 'HEURISTICA_EMAIL'}

        if 'limite' in nome_coluna.lower() or '%' in nome_coluna or 'vi' in nome_coluna.lower():
            try:
                valores_float = [float(v) for v in valores_str
                                if v not in ['nan', '', 'None', 'NaN']]
                if valores_float and all(-1 <= v <= 1 for v in valores_float):
                    return {'campo_detectado': 'Percentual_Decimal',
                           'confianca': 0.90,
                           'metodo': 'HEURISTICA_PERCENTUAL'}
            except:
                pass

        if tamanho_medio > 50:
            return {'campo_detectado': 'Texto_Longo', 'confianca': 0.85,
                   'metodo': 'HEURISTICA_TEXTO_LONGO'}

        if any(v.startswith(('teams/', 'http', 'https', '//', '\\\\'))
               for v in valores_str):
            return {'campo_detectado': 'Path_URL', 'confianca': 0.90,
                   'metodo': 'HEURISTICA_PATH'}

        if valores_unicos <= {'Sim', 'Não', 'sim', 'não'}:
            return {'campo_detectado': 'Booleano_Texto', 'confianca': 0.95,
                   'metodo': 'HEURISTICA_BOOLEANO'}

        if 'por' in nome_coluna.lower() or 'nome' in nome_coluna.lower():
            if all(len(v.split()) >= 2 and '@' not in v
                   for v in valores_str[:5]):
                return {'campo_detectado': 'Nome_Pessoa', 'confianca': 0.85,
                       'metodo': 'HEURISTICA_NOME'}

        palavras_status = {'em aprovação', 'aprovado', 'rejeitado',
                          'ítem criado', 'item criado', 'pendente'}
        if any(v.lower() in palavras_status for v in valores_str):
            return {'campo_detectado': 'Status_Workflow', 'confianca': 0.90,
                   'metodo': 'HEURISTICA_STATUS'}

        if 'base de' in ' '.join(valores_str[:3]).lower():
            return {'campo_detectado': 'Unidade_Operacional_Nome',
                   'confianca': 0.90,
                   'metodo': 'HEURISTICA_UNIDADE_OP'}

        return None

    def atualizar_historico(self, info):
        self.dados['historico_arquivos'].append(info)
        self.dados['ultima_atualizacao'] = datetime.now().isoformat()
        self._salvar(self.dados)

    def _salvar(self, dados):
        with open(self.arquivo_dict, 'w', encoding='utf-8') as f:
            json.dump(dados, f, indent=2, ensure_ascii=False)

# ===================================================================
# 6. INICIALIZAR DICIONÁRIO
# ===================================================================

print("\n" + "="*70)
print("INICIALIZANDO DICIONÁRIO")
print("="*70)

dicionario = DicionarioInteligente(fm)

# ===================================================================
# 7. SALVAR ESTADO PARA PRÓXIMO BLOCO
# ===================================================================

estado_bloco3 = {
    'bloco': 3,
    'versao': '4.5',
    'timestamp': datetime.now().isoformat(),
    'status': 'concluido',
    'dicionario': {
        'arquivo': str(dicionario.arquivo_dict),
        'campos_conhecidos': len(dicionario.dados['campos_conhecidos']),
        'arquivos_processados': len(dicionario.dados['historico_arquivos']),
        'config_sistema': dicionario.dados.get('config_sistema', {})
    },
    'componentes_carregados': ['DicionarioInteligente', 'GUIComTimer']
}

arquivo_estado = fm.pastas['logs'] / '.bloco_3_state.json'
with open(arquivo_estado, 'w', encoding='utf-8') as f:
    json.dump(estado_bloco3, f, indent=2, ensure_ascii=False)

print("\n" + "="*70)
print("✅ BLOCO 3 CONCLUÍDO")
print("="*70)
print("\n📋 Componentes:")
print("   • DicionarioInteligente ........... ✅")
n_campos = len(dicionario.dados['campos_conhecidos'])
n_arquivos = len(dicionario.dados['historico_arquivos'])
print(f"     - Campos: {n_campos}")
print(f"     - Arquivos: {n_arquivos}")
timeout = dicionario.dados.get('config_sistema', {}).get('timeout_sessao_minutos', 60)
print(f"     - Timeout: {timeout}min")
print("   • GUIComTimer ..................... ✅")
print("\n💾 Estado salvo em:")
print(f"   • .bloco_3_state.json")
print(f"   • DICT_Dicionario_Persistente.json")
print("\n💡 Próximo: BLOCO 4 seleciona arquivo de dados")
print("="*70)


BLOCO 3: DICIONÁRIO INTELIGENTE + GUI COM TIMER

✅ CONFIGURAÇÃO CARREGADA DO LOG GLOBAL
   📁 Pasta base: PROCESSAR_ARQUIVOS_20251019_060722
   🕐 Timestamp: 20251019_060722

✅ BLOCO 2 VALIDADO
   Executado em: 2025-10-19T06:07:25.342924
   Classes: LocalizadorDicionario, FileManagerInterativo, SeletorArquivo, DetectorCabecalho

✅ FileManager recriado: PROCESSAR_ARQUIVOS_20251019_060722

✅ Classe GUIComTimer carregada

INICIALIZANDO DICIONÁRIO

📝 CRIANDO NOVO DICIONÁRIO...
✅ Dicionário criado: 22 campos

✅ BLOCO 3 CONCLUÍDO

📋 Componentes:
   • DicionarioInteligente ........... ✅
     - Campos: 22
     - Arquivos: 0
     - Timeout: 60min
   • GUIComTimer ..................... ✅

💾 Estado salvo em:
   • .bloco_3_state.json
   • DICT_Dicionario_Persistente.json

💡 Próximo: BLOCO 4 seleciona arquivo de dados


In [28]:
# ═══════════════════════════════════════════════════════════════════
# BLOCO 4 - SELEÇÃO DE ARQUIVO - SUPORTE MULTI-FORMATO (REVISADO v3.1)
# ═══════════════════════════════════════════════════════════════════
# COMUNICAÇÃO VIA LOG:
# - LÊ: pasta_base (LOG global), timestamp, dicionário persistente
# - RECRIA: FileManager localmente
# - SALVA: arquivo selecionado + config CSV (se houver)
#
# MUDANÇAS v3.1:
# - Navegação inteligente: abre pasta 01_Entrada por padrão
# - Histórico de pastas (últimas 5)
# - Botão para última pasta usada (com timer)
# ═══════════════════════════════════════════════════════════════════

print("="*70)
print("BLOCO 4: SELEÇÃO DE ARQUIVO")
print("="*70)

# ═══════════════════════════════════════════════════════════════════
# 1. LER CONFIGURAÇÕES DO BLOCO ANTERIOR (VIA LOG)
# ═══════════════════════════════════════════════════════════════════

from pathlib import Path
import json
from datetime import datetime

log_global = Path.home() / '.processador_dicionario_localizador.json'

if not log_global.exists():
    raise FileNotFoundError(
        "❌ LOG GLOBAL não encontrado!\n"
        "   Execute BLOCO 1 primeiro para criar a estrutura."
    )

with open(log_global, 'r', encoding='utf-8') as f:
    config = json.load(f)

pasta_base = Path(config['pasta_base_atual'])
timestamp_execucao = config['timestamp']

print(f"\n📂 Pasta base carregada: {pasta_base.name}")
print(f"⏰ Timestamp: {timestamp_execucao}")

# ═══════════════════════════════════════════════════════════════════
# 2. RECRIAR OBJETOS NECESSÁRIOS (NÃO ASSUMIR MEMÓRIA)
# ═══════════════════════════════════════════════════════════════════

# Recriar FileManager
fm = FileManagerInterativo(pasta_base)

# Carregar dicionário persistente
dict_file = fm.pastas['logs'] / 'DICT_Dicionario_Persistente.json'

if dict_file.exists():
    with open(dict_file, 'r', encoding='utf-8') as f:
        DICIONARIO_PERSISTENTE = json.load(f)
    print(f"📚 Dicionário carregado: {len(DICIONARIO_PERSISTENTE.get('campos_conhecidos', {}))} campos")
else:
    raise FileNotFoundError(
        "❌ Dicionário não encontrado!\n"
        "   Execute BLOCO 3 primeiro."
    )

# ═══════════════════════════════════════════════════════════════════
# CONSTANTES E CONFIGURAÇÕES
# ═══════════════════════════════════════════════════════════════════

FILETYPES_SUPORTADOS = [
    ("Todos os suportados", "*.xlsx *.xls *.xlsm *.csv *.txt"),
    ("Excel", "*.xlsx *.xls *.xlsm"),
    ("CSV", "*.csv"),
    ("TXT (Tabelas)", "*.txt"),
    ("Todos", "*.*")
]

TIMEOUT_SESSAO_MINUTOS = DICIONARIO_PERSISTENTE.get(
    'config_sistema', {}
).get('timeout_sessao_minutos', 60)

# ═══════════════════════════════════════════════════════════════════
# FUNÇÕES AUXILIARES
# ═══════════════════════════════════════════════════════════════════

def detectar_config_csv(arquivo_path):
    """
    Detecta encoding e separador ideal para CSV.

    Returns:
        dict: {'encoding': str, 'sep': str, 'colunas': int}
        None: se falhar
    """
    import pandas as pd

    encodings = ['utf-8', 'latin-1', 'cp1252', 'iso-8859-1']
    separadores = [',', ';', '\t', '|']

    melhor_config = None
    max_colunas = 0

    for encoding in encodings:
        for sep in separadores:
            try:
                df_test = pd.read_csv(
                    arquivo_path,
                    nrows=5,
                    encoding=encoding,
                    sep=sep,
                    on_bad_lines='skip'
                )
                n_cols = len(df_test.columns)

                if n_cols > max_colunas and n_cols > 1:
                    max_colunas = n_cols
                    melhor_config = {
                        'encoding': encoding,
                        'sep': sep,
                        'colunas': n_cols
                    }
            except:
                continue

    return melhor_config


def validar_arquivo_selecionado(arquivo_path):
    """
    Validação básica do arquivo selecionado.

    Raises:
        FileNotFoundError: Se arquivo não existe
        ValueError: Se arquivo inválido

    Returns:
        dict: Informações de validação
    """
    import pandas as pd

    if not arquivo_path.exists():
        raise FileNotFoundError(f"Arquivo não encontrado: {arquivo_path}")

    tamanho = arquivo_path.stat().st_size
    if tamanho == 0:
        raise ValueError(f"Arquivo vazio: {arquivo_path.name}")

    extensao = arquivo_path.suffix.lower()
    extensoes_validas = ['.xlsx', '.xls', '.xlsm', '.csv', '.txt']

    if extensao not in extensoes_validas:
        print(f"   ⚠️ Extensão incomum: {extensao}")

    config_extra = {}

    try:
        if extensao in ['.xlsx', '.xls', '.xlsm']:
            pd.read_excel(arquivo_path, nrows=1)

        elif extensao == '.csv':
            config_csv = detectar_config_csv(arquivo_path)

            if config_csv:
                config_extra['csv'] = config_csv
                print(f"   📊 CSV: {config_csv['colunas']} colunas")
                print(f"   🔤 Encoding: {config_csv['encoding']}")
                print(f"   ➗ Separador: {repr(config_csv['sep'])}")
            else:
                print(f"   ⚠️ Config CSV não detectada automaticamente")

    except Exception as e:
        raise ValueError(f"Arquivo corrompido ou ilegível: {str(e)[:100]}")

    return {
        'valido': True,
        'tamanho_kb': tamanho / 1024,
        'extensao': extensao,
        **config_extra
    }


def carregar_preview_inteligente(arquivo_path, frame_preview):
    """
    Carrega preview do arquivo com tratamento de erro amigável.

    Args:
        arquivo_path: Path do arquivo
        frame_preview: Frame tkinter para mostrar preview
    """
    import tkinter as tk
    import pandas as pd

    extensao = arquivo_path.suffix.lower()

    try:
        if extensao in ['.xlsx', '.xls', '.xlsm']:
            df_quick = pd.read_excel(arquivo_path, nrows=3)

        elif extensao == '.csv':
            config_csv = detectar_config_csv(arquivo_path)
            if config_csv:
                df_quick = pd.read_csv(
                    arquivo_path,
                    nrows=3,
                    encoding=config_csv['encoding'],
                    sep=config_csv['sep']
                )
            else:
                df_quick = pd.read_csv(arquivo_path, nrows=3)

        elif extensao == '.txt':
            for sep in ['\t', ';', '|', ',']:
                try:
                    df_quick = pd.read_csv(arquivo_path, nrows=3, sep=sep)
                    if len(df_quick.columns) > 1:
                        break
                except:
                    continue

        preview_text = f"{len(df_quick)} linhas × {len(df_quick.columns)} colunas"
        tk.Label(
            frame_preview,
            text=preview_text,
            font=('Arial', 8),
            bg='#F5F5F5',
            fg='#666666',
            anchor='w'
        ).pack(fill=tk.X, padx=5, pady=(0, 3))

    except Exception as e:
        tk.Label(
            frame_preview,
            text="⚠️ Preview indisponível",
            font=('Arial', 8),
            bg='#F5F5F5',
            fg='#FF6B6B',
            anchor='w'
        ).pack(fill=tk.X, padx=5, pady=(0, 3))

        print(f"   ⚠️ Preview falhou: {str(e)[:50]}")

# ═══════════════════════════════════════════════════════════════════
# CARREGAR HISTÓRICO DE NAVEGAÇÃO (NOVO v3.1)
# ═══════════════════════════════════════════════════════════════════

historico_file = fm.pastas['logs'] / '.historico_pastas_navegacao.json'

if historico_file.exists():
    try:
        with open(historico_file, 'r', encoding='utf-8') as f:
            historico_pastas = json.load(f)
    except:
        historico_pastas = {'ultima_pasta': None, 'historico': []}
else:
    historico_pastas = {'ultima_pasta': None, 'historico': []}

# Determinar pasta inicial padrão
pasta_entrada = fm.pastas['entrada']
ultima_pasta_usada = None

if historico_pastas.get('ultima_pasta'):
    ultima_pasta_usada = Path(historico_pastas['ultima_pasta'])
    if not ultima_pasta_usada.exists():
        ultima_pasta_usada = None

# ═══════════════════════════════════════════════════════════════════
# 3. PROCESSAR DADOS DO BLOCO (LÓGICA PRINCIPAL)
# ═══════════════════════════════════════════════════════════════════

import tkinter as tk
from tkinter import filedialog

# Carregar última seleção
config_file = fm.pastas['logs'] / '.ultimo_arquivo.json'
ultimo_arquivo = None
sessao_atual = False

if config_file.exists():
    try:
        with open(config_file, 'r', encoding='utf-8') as f:
            config = json.load(f)

        try:
            ts_config = datetime.fromisoformat(config.get('timestamp', ''))
            ts_agora = datetime.now()
            diff_minutos = (ts_agora - ts_config).total_seconds() / 60

            if diff_minutos < TIMEOUT_SESSAO_MINUTOS:
                caminho_salvo = config.get('caminho')
                if caminho_salvo and Path(caminho_salvo).exists():
                    ultimo_arquivo = Path(caminho_salvo)
                    sessao_atual = True
        except:
            pass
    except:
        pass

print(f"\n💡 Última seleção: {ultimo_arquivo.name if ultimo_arquivo else 'Nenhuma'}")
print(f"   Mesma sessão: {'Sim' if sessao_atual else 'Não'}")
if ultima_pasta_usada:
    print(f"   Última pasta: {ultima_pasta_usada}")

# CASO 1: Tem arquivo da sessão atual → GUI com timer
if ultimo_arquivo and sessao_atual:
    def selecionar_arquivo_com_timer(ultimo_path):
        """GUI com timer para confirmar ou trocar arquivo"""
        root, frame, resultado, contador = GUIComTimer.criar_janela_com_timer(
            "DETECTOR - Seleção de Arquivo",
            650, 520,
            tem_timer=True
        )

        tk.Label(
            frame,
            text="📂 Seleção de Arquivo",
            font=('Arial', 14, 'bold'),
            bg='white'
        ).pack(pady=(0, 15))

        tk.Label(
            frame,
            text="💡 Último arquivo selecionado nesta sessão:",
            font=('Arial', 10),
            bg='white'
        ).pack(pady=(0, 5))

        extensao = ultimo_path.suffix.lower()
        if extensao in ['.xlsx', '.xls', '.xlsm']:
            tipo_arquivo = "Excel"
            icone = "📊"
        elif extensao == '.csv':
            tipo_arquivo = "CSV"
            icone = "📄"
        elif extensao == '.txt':
            tipo_arquivo = "TXT"
            icone = "📝"
        else:
            tipo_arquivo = "Desconhecido"
            icone = "❓"

        frame_info = tk.Frame(frame, bg='#E3F2FD', relief=tk.SUNKEN, borderwidth=2)
        frame_info.pack(fill=tk.X, pady=(0, 10), padx=10)

        tk.Label(
            frame_info,
            text=f"{icone} {ultimo_path.name}",
            font=('Arial', 10, 'bold'),
            bg='#E3F2FD',
            fg='#1565C0',
            anchor='w'
        ).pack(fill=tk.X, padx=10, pady=(5, 2))

        tamanho_kb = ultimo_path.stat().st_size / 1024
        tk.Label(
            frame_info,
            text=f"📦 Tipo: {tipo_arquivo} | 📏 Tamanho: {tamanho_kb:.1f} KB",
            font=('Arial', 9),
            bg='#E3F2FD',
            fg='#1565C0',
            anchor='w'
        ).pack(fill=tk.X, padx=10, pady=(0, 2))

        tk.Label(
            frame_info,
            text=f"📂 Local: {ultimo_path.parent}",
            font=('Arial', 9),
            bg='#E3F2FD',
            fg='#1565C0',
            anchor='w',
            wraplength=600
        ).pack(fill=tk.X, padx=10, pady=(0, 5))

        countdown = GUIComTimer.adicionar_timer(frame, root, resultado, contador)

        tk.Label(
            frame,
            text="Deseja usar este arquivo ou escolher outro?",
            font=('Arial', 10),
            bg='white'
        ).pack(pady=(10, 10))

        frame_preview = tk.Frame(frame, bg='#F5F5F5', relief=tk.SUNKEN, borderwidth=1)
        frame_preview.pack(fill=tk.X, padx=10, pady=(0, 10))

        tk.Label(
            frame_preview,
            text="📊 Preview (3 primeiras linhas):",
            font=('Arial', 9, 'bold'),
            bg='#F5F5F5',
            anchor='w'
        ).pack(fill=tk.X, padx=5, pady=(3, 2))

        carregar_preview_inteligente(ultimo_path, frame_preview)

        def usar_ultimo():
            resultado['cancelado'] = True
            resultado['valor'] = ultimo_path
            root.quit()
            root.destroy()

        def escolher_novo():
            resultado['cancelado'] = True
            root.withdraw()

            # NAVEGAÇÃO INTELIGENTE: usar última pasta ou pasta_entrada
            pasta_inicial = ultima_pasta_usada if ultima_pasta_usada else pasta_entrada

            arquivo = filedialog.askopenfilename(
                title="Selecione o arquivo de dados",
                initialdir=str(pasta_inicial),
                filetypes=FILETYPES_SUPORTADOS
            )

            resultado['valor'] = Path(arquivo) if arquivo else ultimo_path
            root.quit()
            root.destroy()

        tk.Frame(frame, height=2, bg='#CCCCCC').pack(fill=tk.X, pady=10)

        frame_btns = tk.Frame(frame, bg='white')
        frame_btns.pack(side=tk.BOTTOM, pady=10)

        tk.Button(
            frame_btns,
            text="Escolher Novo Arquivo",
            command=escolher_novo,
            width=22,
            height=2,
            font=('Arial', 10, 'bold'),
            bg='#4CAF50',
            fg='white',
            cursor='hand2'
        ).pack(side=tk.LEFT, padx=5)

        nome_curto = ultimo_path.name[:15] + '...' if len(ultimo_path.name) > 15 else ultimo_path.name
        tk.Button(
            frame_btns,
            text=f"Usar '{nome_curto}' (10s)",
            command=usar_ultimo,
            width=30,
            height=2,
            font=('Arial', 10),
            bg='#2196F3',
            fg='white',
            cursor='hand2'
        ).pack(side=tk.LEFT, padx=5)

        root.after(1000, countdown)
        root.mainloop()

        if resultado.get('timeout'):
            print(f"   ⏱️ Timeout (10s) - usando último arquivo")
            return ultimo_path

        return resultado['valor']

    print("\nAbrindo janela...")
    arquivo_selecionado = selecionar_arquivo_com_timer(ultimo_arquivo)

# CASO 2: Não tem arquivo da sessão → GUI com navegação inteligente
else:
    def selecionar_arquivo_direto():
        """GUI direta para primeira seleção com navegação inteligente"""

        # NAVEGAÇÃO INTELIGENTE: última pasta usada OU pasta 01_Entrada
        pasta_inicial = ultima_pasta_usada if ultima_pasta_usada else pasta_entrada

        print(f"   📁 Abrindo em: {pasta_inicial.name}")

        root = tk.Tk()
        root.withdraw()
        root.attributes('-topmost', True)

        arquivo = filedialog.askopenfilename(
            title="Selecione o arquivo de dados",
            initialdir=str(pasta_inicial),
            filetypes=FILETYPES_SUPORTADOS
        )

        root.destroy()

        if not arquivo:
            raise ValueError("❌ Nenhum arquivo selecionado")

        return Path(arquivo)

    print("\nAbrindo janela de seleção...")
    print("(A janela pode estar atrás do navegador)")
    arquivo_selecionado = selecionar_arquivo_direto()

# ═══════════════════════════════════════════════════════════════════
# ATUALIZAR HISTÓRICO DE NAVEGAÇÃO (NOVO v3.1)
# ═══════════════════════════════════════════════════════════════════

pasta_do_arquivo = arquivo_selecionado.parent

# Salvar última pasta usada
historico_pastas['ultima_pasta'] = str(pasta_do_arquivo)

# Atualizar histórico (manter últimas 5 pastas)
historico = historico_pastas.get('historico', [])

# Remover duplicatas
if str(pasta_do_arquivo) in historico:
    historico.remove(str(pasta_do_arquivo))

# Adicionar no topo
historico.insert(0, str(pasta_do_arquivo))

# Manter apenas 5 mais recentes
historico_pastas['historico'] = historico[:5]
historico_pastas['ultima_atualizacao'] = datetime.now().isoformat()

# Salvar histórico
with open(historico_file, 'w', encoding='utf-8') as f:
    json.dump(historico_pastas, f, indent=2, ensure_ascii=False)

# ═══════════════════════════════════════════════════════════════════
# VALIDAÇÃO E DETECÇÃO DE TIPO
# ═══════════════════════════════════════════════════════════════════

print("\n🔍 Validando arquivo...")

try:
    info_validacao = validar_arquivo_selecionado(arquivo_selecionado)
    print("   ✅ Arquivo validado com sucesso")
except Exception as e:
    print(f"   ❌ ERRO: {e}")
    raise

extensao = arquivo_selecionado.suffix.lower()
if extensao in ['.xlsx', '.xls', '.xlsm']:
    tipo_arquivo = "Excel"
elif extensao == '.csv':
    tipo_arquivo = "CSV"
elif extensao == '.txt':
    tipo_arquivo = "TXT"
else:
    tipo_arquivo = "Desconhecido"

# ═══════════════════════════════════════════════════════════════════
# 4. SALVAR ESTADO PARA PRÓXIMO BLOCO (VIA LOG)
# ═══════════════════════════════════════════════════════════════════

config_salvar = {
    'nome': arquivo_selecionado.name,
    'caminho': str(arquivo_selecionado),
    'tamanho_kb': info_validacao['tamanho_kb'],
    'tipo': tipo_arquivo,
    'extensao': extensao,
    'timestamp': datetime.now().isoformat()
}

if 'csv' in info_validacao:
    config_salvar['config_csv'] = info_validacao['csv']

with open(config_file, 'w', encoding='utf-8') as f:
    json.dump(config_salvar, f, indent=2, ensure_ascii=False)

# Salvar estado do BLOCO 4
estado_bloco = {
    'bloco': 4,
    'status': 'concluido',
    'timestamp': datetime.now().isoformat(),
    'arquivo_selecionado': arquivo_selecionado.name,
    'tipo': tipo_arquivo,
    'tamanho_kb': info_validacao['tamanho_kb']
}

with open(fm.pastas['logs'] / '.bloco_5_state.json', 'w') as f:
    json.dump(estado_bloco, f, indent=2)

# ═══════════════════════════════════════════════════════════════════
# 5. CONFIRMAÇÃO FINAL
# ═══════════════════════════════════════════════════════════════════

print(f"\n{'='*70}")
print("✅ ARQUIVO SELECIONADO E VALIDADO")
print(f"{'='*70}")
print(f"📄 Nome: {arquivo_selecionado.name}")
print(f"📦 Tipo: {tipo_arquivo}")
print(f"📏 Tamanho: {info_validacao['tamanho_kb']:.1f} KB")
print(f"📂 Pasta: {arquivo_selecionado.parent.name}")

if 'csv' in info_validacao:
    csv_info = info_validacao['csv']
    print(f"📊 Colunas detectadas: {csv_info['colunas']}")
    print(f"🔤 Encoding: {csv_info['encoding']}")
    print(f"➗ Separador: {repr(csv_info['sep'])}")

print(f"{'='*70}")
print("✅ BLOCO 4 CONCLUÍDO")
print(f"{'='*70}")
print(f"\n💾 Estado salvo em: {fm.pastas['logs']}")
print(f"📋 Próximo: BLOCO 6 carregará o arquivo usando esta configuração")

BLOCO 4: SELEÇÃO DE ARQUIVO

📂 Pasta base carregada: PROCESSAR_ARQUIVOS_20251019_060722
⏰ Timestamp: 20251019_060722
📚 Dicionário carregado: 22 campos

💡 Última seleção: Nenhuma
   Mesma sessão: Não

Abrindo janela de seleção...
(A janela pode estar atrás do navegador)
   📁 Abrindo em: 01_Entrada

🔍 Validando arquivo...
   ✅ Arquivo validado com sucesso

✅ ARQUIVO SELECIONADO E VALIDADO
📄 Nome: Cópia de xSAPtemp4687_JAN_25.xls
📦 Tipo: Excel
📏 Tamanho: 16257.5 KB
📂 Pasta: Dado BW
✅ BLOCO 4 CONCLUÍDO

💾 Estado salvo em: E:\OneDrive - VIBRA\NMCV - Documentos\Indicador\_DataLake\2- Dados Processados (PROCESSED)\PROCESSAR_ARQUIVOS_20251019_060722\04_Logs
📋 Próximo: BLOCO 6 carregará o arquivo usando esta configuração


In [29]:
# ═══════════════════════════════════════════════════════════════
# BLOCO 5: CARREGAMENTO INTELIGENTE - EXCEL E CSV
# ═══════════════════════════════════════════════════════════════

import json
from datetime import datetime

print("\n" + "="*70)
print("📥 CARREGAMENTO DO ARQUIVO")
print("="*70)

# Detectar tipo de arquivo pela extensão
extensao = arquivo_selecionado.suffix.lower()
print(f"\n🔍 Extensão detectada: {extensao}")

# ═══════════════════════════════════════════════════════════════
# CASO 1: ARQUIVOS EXCEL (.xls, .xlsx, .xlsm)
# ═══════════════════════════════════════════════════════════════

if extensao in ['.xls', '.xlsx', '.xlsm']:
    print(f"📊 Tipo: EXCEL")

    try:
        # Tentar xlrd primeiro (para .xls antigos)
        workbook = xlrd.open_workbook(str(arquivo_selecionado))
        sheets = workbook.sheet_names()
        metodo_carga = 'xlrd'
        print(f"   ✅ Método: xlrd (XLS)")

    except:
        # Se falhar, usar pandas (para .xlsx/.xlsm)
        try:
            workbook = pd.ExcelFile(str(arquivo_selecionado))
            sheets = workbook.sheet_names
            metodo_carga = 'pandas'
            print(f"   ✅ Método: pandas (XLSX/XLSM)")
        except Exception as e:
            print(f"   ❌ ERRO ao abrir Excel: {e}")
            raise

    print(f"\n📋 Sheets encontradas: {len(sheets)}")
    for i, sheet in enumerate(sheets, 1):
        print(f"   {i}. {sheet}")

    tipo_arquivo = 'EXCEL'
    separador_detectado = None
    skiprows_csv = 0

# ═══════════════════════════════════════════════════════════════
# CASO 2: ARQUIVOS CSV (.csv)
# ═══════════════════════════════════════════════════════════════

elif extensao == '.csv':
    print(f"📄 Tipo: CSV")

    try:
        # Ler primeira linha para detectar separador
        with open(arquivo_selecionado, 'r', encoding='cp1252') as f:
            primeira_linha = f.readline().strip()

        print(f"\n🔍 Primeira linha: {primeira_linha[:100]}")

        # Detectar separador
        separador_detectado = None

        # Caso 1: Linha explícita com "sep="
        if primeira_linha.lower().startswith('sep='):
            separador_detectado = primeira_linha.split('=')[1]
            skiprows_csv = 1
            print(f"   ✅ Separador explícito: '{separador_detectado}'")

        # Caso 2: Tentar detectar automaticamente
        else:
            for sep in ['^', ';', ',', '\t', '|']:
                df_test = pd.read_csv(
                    arquivo_selecionado,
                    nrows=2,
                    sep=sep,
                    encoding='cp1252',
                    on_bad_lines='skip'
                )
                if len(df_test.columns) > 1:
                    separador_detectado = sep
                    skiprows_csv = 0
                    print(f"   ✅ Separador auto: '{separador_detectado}'")
                    break

        if not separador_detectado:
            raise ValueError("❌ Separador CSV não detectado")

        # Carregar preview
        df_preview = pd.read_csv(
            arquivo_selecionado,
            sep=separador_detectado,
            encoding='cp1252',
            skiprows=skiprows_csv,
            nrows=5
        )

        print(f"\n📊 Estrutura do CSV:")
        print(f"   Colunas: {len(df_preview.columns)}")
        print(f"   Encoding: cp1252")
        print(f"   Separador: '{separador_detectado}'")

        # Simular sheets (CSV = 1 sheet virtual)
        sheets = ['Dados CSV']
        metodo_carga = 'csv'
        workbook = None
        tipo_arquivo = 'CSV'

        print(f"\n📋 Sheet virtual: 'Dados CSV'")

    except Exception as e:
        print(f"   ❌ ERRO ao processar CSV: {e}")
        raise

# ═══════════════════════════════════════════════════════════════
# CASO 3: ARQUIVOS TXT (.txt)
# ═══════════════════════════════════════════════════════════════

elif extensao == '.txt':
    print(f"📝 Tipo: TXT")
    print(f"   ℹ️  Processamento similar a CSV")

    try:
        # Ler primeira linha para detectar separador
        with open(arquivo_selecionado, 'r', encoding='cp1252') as f:
            primeira_linha = f.readline().strip()

        print(f"\n🔍 Primeira linha: {primeira_linha[:100]}")

        # Detectar separador
        separador_detectado = None

        # Caso 1: Linha explícita com "sep="
        if primeira_linha.lower().startswith('sep='):
            separador_detectado = primeira_linha.split('=')[1]
            skiprows_csv = 1
            print(f"   ✅ Separador explícito: '{separador_detectado}'")

        # Caso 2: Tentar detectar automaticamente
        else:
            for sep in ['^', ';', ',', '\t', '|']:
                df_test = pd.read_csv(
                    arquivo_selecionado,
                    nrows=2,
                    sep=sep,
                    encoding='cp1252',
                    on_bad_lines='skip'
                )
                if len(df_test.columns) > 1:
                    separador_detectado = sep
                    skiprows_csv = 0
                    print(f"   ✅ Separador auto: '{separador_detectado}'")
                    break

        if not separador_detectado:
            raise ValueError("❌ Separador TXT não detectado")

        # Carregar preview
        df_preview = pd.read_csv(
            arquivo_selecionado,
            sep=separador_detectado,
            encoding='cp1252',
            skiprows=skiprows_csv,
            nrows=5
        )

        print(f"\n📊 Estrutura do TXT:")
        print(f"   Colunas: {len(df_preview.columns)}")
        print(f"   Encoding: cp1252")
        print(f"   Separador: '{separador_detectado}'")

        # Simular sheets (TXT = 1 sheet virtual)
        sheets = ['Dados TXT']
        metodo_carga = 'csv'
        workbook = None
        tipo_arquivo = 'TXT'

        print(f"\n📋 Sheet virtual: 'Dados TXT'")

    except Exception as e:
        print(f"   ❌ ERRO ao processar TXT: {e}")
        raise

else:
    raise ValueError(f"❌ Formato não suportado: {extensao}")

# ═══════════════════════════════════════════════════════════════
# SALVAMENTO DE ESTADO NO LOG
# ═══════════════════════════════════════════════════════════════

estado_bloco5 = {
    'bloco': 5,
    'status': 'concluido',
    'timestamp': datetime.now().isoformat(),
    'tipo_arquivo': tipo_arquivo,
    'metodo_carga': metodo_carga,
    'extensao': extensao,
    'sheets': sheets,
    'workbook_path': str(arquivo_selecionado),
    'separador_detectado': separador_detectado,
    'skiprows_csv': skiprows_csv
}

arquivo_estado = fm.pastas['logs'] / '.bloco_5_state.json'
with open(arquivo_estado, 'w', encoding='utf-8') as f:
    json.dump(estado_bloco5, f, indent=2, ensure_ascii=False)

# ═══════════════════════════════════════════════════════════════
# RESUMO DO CARREGAMENTO
# ═══════════════════════════════════════════════════════════════

print("\n" + "─"*70)
print("✅ CARREGAMENTO CONCLUÍDO")
print("─"*70)
print(f"   Tipo: {tipo_arquivo}")
print(f"   Método: {metodo_carga}")
print(f"   Sheets/Tabelas: {len(sheets)}")

print("\n" + "="*70)
print("✅ BLOCO 5 CONCLUÍDO")
print("="*70)
print(f"💾 Estado salvo: .bloco_5_state.json")
print(f"📋 Próximo: BLOCO 6 selecionará a sheet e fará preview")


📥 CARREGAMENTO DO ARQUIVO

🔍 Extensão detectada: .xls
📊 Tipo: EXCEL
   ✅ Método: xlrd (XLS)

📋 Sheets encontradas: 11
   1. SAPBEXqueriesDefunct
   2. SAPBEXfiltersDefunct
   3. Valor da Variação Total
   4. Valor da Variação Total Grupo
   5. Limite Técnico
   6. Justificar
   7. Limite Técnico Grupo
   8. BExRepositorySheet
   9. Justificar Grupo
   10. Custo do Produto
   11. Imposto

──────────────────────────────────────────────────────────────────────
✅ CARREGAMENTO CONCLUÍDO
──────────────────────────────────────────────────────────────────────
   Tipo: EXCEL
   Método: xlrd
   Sheets/Tabelas: 11

✅ BLOCO 5 CONCLUÍDO
💾 Estado salvo: .bloco_5_state.json
📋 Próximo: BLOCO 6 selecionará a sheet e fará preview


In [30]:
# ═══════════════════════════════════════════════════════════════════
# BLOCO 6 - SELEÇÃO DE SHEET (COM SUPORTE CSV)
# ═══════════════════════════════════════════════════════════════════

print("\n" + "="*70)
print("📋 SELEÇÃO DE SHEET/TABELA")
print("="*70)

# Carregar última seleção
config_file = fm.pastas['logs'] / '.ultima_sheet.json'
ultima_sheet = None
arquivo_mudou = True

if config_file.exists():
    try:
        with open(config_file, 'r', encoding='utf-8') as f:
            config = json.load(f)
            # Verificar se é o mesmo arquivo E timestamp recente (última hora)
            if config.get('arquivo') == arquivo_selecionado.name:
                ultima_sheet = config.get('sheet')
                # Verificar se timestamp é recente
                try:
                    ts_salvo = datetime.fromisoformat(config.get('timestamp', ''))
                    ts_agora = datetime.now()
                    diff_minutos = (ts_agora - ts_salvo).total_seconds() / 60

                    if diff_minutos < 60:  # Última hora
                        arquivo_mudou = False
                except:
                    pass
    except:
        pass

print(f"\n💡 Última sheet: {ultima_sheet if ultima_sheet else 'Nenhuma'}")
print(f"   Arquivo mudou: {'Sim' if arquivo_mudou else 'Não'}")

# ═══════════════════════════════════════════════════════════════════
# CASO 1: CSV - SELEÇÃO AUTOMÁTICA (apenas 1 sheet virtual)
# ═══════════════════════════════════════════════════════════════════

if tipo_arquivo == 'CSV':
    sheet_nome = sheets[0]  # 'Dados CSV'
    print(f"\n✅ Arquivo CSV - usando sheet virtual automática: '{sheet_nome}'")

# ═══════════════════════════════════════════════════════════════════
# CASO 2: EXCEL - Apenas 1 sheet E arquivo mudou → Usar diretamente
# ═══════════════════════════════════════════════════════════════════

elif len(sheets) == 1 and arquivo_mudou:
    sheet_nome = sheets[0]
    print(f"\n✅ Apenas 1 sheet - selecionando automaticamente: '{sheet_nome}'")

# ═══════════════════════════════════════════════════════════════════
# CASO 3: EXCEL - Mais de 1 sheet OU tem histórico → GUI COM TIMER
# ═══════════════════════════════════════════════════════════════════

else:
    def selecionar_sheet_com_timer(sheets, ultima=None, mostrar_timer=True):
        """GUI com timer para seleção de sheet"""
        root, frame, resultado, contador = GUIComTimer.criar_janela_com_timer(
            "DETECTOR - Seleção de Sheet",
            600, 450,
            tem_timer=(mostrar_timer and ultima is not None)
        )

        # Título
        tk.Label(
            frame,
            text="📋 Seleção de Sheet",
            font=('Arial', 14, 'bold'),
            bg='white'
        ).pack(pady=(0, 10))

        tk.Label(
            frame,
            text="Selecione a Sheet para processar:",
            font=('Arial', 12, 'bold'),
            bg='white'
        ).pack(pady=(0, 10))

        # Timer (se tem última E timer ativo)
        if ultima and mostrar_timer:
            tk.Label(
                frame,
                text=f"💡 Última sheet usada: '{ultima}'",
                font=('Arial', 10),
                bg='#E3F2FD',
                fg='#1565C0',
                padx=10,
                pady=10
            ).pack(fill=tk.X, pady=(0, 5))

            countdown = GUIComTimer.adicionar_timer(frame, root, resultado, contador)

        # Listbox
        frame_list = tk.Frame(frame, bg='white')
        frame_list.pack(fill=tk.BOTH, expand=True, pady=(0, 10))

        scrollbar = tk.Scrollbar(frame_list)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

        listbox = tk.Listbox(
            frame_list,
            yscrollcommand=scrollbar.set,
            font=('Arial', 10),
            height=8
        )
        listbox.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        scrollbar.config(command=listbox.yview)

        for sheet in sheets:
            listbox.insert(tk.END, sheet)

        # Selecionar última ou primeira
        if ultima and ultima in sheets:
            idx = sheets.index(ultima)
            listbox.select_set(idx)
            listbox.see(idx)
        else:
            listbox.select_set(0)

        # Funções
        def nova_selecao():
            resultado['cancelado'] = True
            selecao = listbox.curselection()
            if selecao:
                resultado['valor'] = sheets[selecao[0]]
            root.quit()
            root.destroy()

        def usar_ultima():
            resultado['cancelado'] = True
            resultado['valor'] = ultima
            root.quit()
            root.destroy()

        def duplo_clique(event):
            nova_selecao()

        listbox.bind('<Double-Button-1>', duplo_clique)

        # Botões
        GUIComTimer.criar_botoes(
            frame,
            nova_selecao,
            usar_ultima if (ultima and mostrar_timer) else None,
            "Selecionar",
            f"Usar '{ultima}' (10s)" if ultima else None
        )

        # Iniciar timer
        if ultima and mostrar_timer:
            root.after(1000, countdown)

        root.mainloop()

        # Processar resultado
        if resultado.get('timeout') and ultima:
            print(f"   ⏱️  Timeout (10s) - usando última sheet")
            return ultima

        return resultado['valor']

    # Executar GUI
    print(f"\nAbrindo janela de seleção...")
    sheet_nome = selecionar_sheet_com_timer(
        sheets,
        ultima_sheet,
        mostrar_timer=(not arquivo_mudou)  # Timer apenas se mesmo arquivo
    )

# ═══════════════════════════════════════════════════════════════════
# SALVAR ESCOLHA
# ═══════════════════════════════════════════════════════════════════

with open(config_file, 'w', encoding='utf-8') as f:
    json.dump({
        'arquivo': arquivo_selecionado.name,
        'sheet': sheet_nome,
        'timestamp': datetime.now().isoformat()
    }, f, indent=2)

print(f"\n✅ Sheet selecionada: '{sheet_nome}'")

# ═══════════════════════════════════════════════════════════════════
# RESUMO
# ═══════════════════════════════════════════════════════════════════

print("\n" + "─"*70)
print("✅ SELEÇÃO CONCLUÍDA")
print("─"*70)
print(f"   Arquivo: {arquivo_selecionado.name}")
print(f"   Sheet: {sheet_nome}")
print(f"   Tipo: {tipo_arquivo}")

# ═══════════════════════════════════════════════════════════════════
# SALVAMENTO DE ESTADO (ADICIONADO - NÃO REMOVE NADA ACIMA)
# ═══════════════════════════════════════════════════════════════════

estado_bloco6 = {
    'timestamp': datetime.now().isoformat(),
    'bloco': 6,
    'nome': 'SELEÇÃO DE SHEET',
    'status': 'concluido',
    'arquivo': arquivo_selecionado.name,
    'sheet_selecionada': sheet_nome,
    'tipo_arquivo': tipo_arquivo,
    'total_sheets': len(sheets),
    'lista_sheets': sheets,
    'metodo_selecao': 'automatico' if (tipo_arquivo == 'CSV' or len(sheets) == 1) else 'gui',
    'arquivo_mudou': arquivo_mudou,
    'tinha_historico': ultima_sheet is not None
}

arquivo_estado = fm.pastas['logs'] / '.bloco_6_state.json'
with open(arquivo_estado, 'w', encoding='utf-8') as f:
    json.dump(estado_bloco6, f, indent=2, ensure_ascii=False)

print(f"\n💾 Estado salvo: {arquivo_estado.name}")


📋 SELEÇÃO DE SHEET/TABELA

💡 Última sheet: Nenhuma
   Arquivo mudou: Sim

Abrindo janela de seleção...

✅ Sheet selecionada: 'Valor da Variação Total'

──────────────────────────────────────────────────────────────────────
✅ SELEÇÃO CONCLUÍDA
──────────────────────────────────────────────────────────────────────
   Arquivo: Cópia de xSAPtemp4687_JAN_25.xls
   Sheet: Valor da Variação Total
   Tipo: EXCEL

💾 Estado salvo: .bloco_6_state.json


In [31]:
# ═══════════════════════════════════════════════════════════════════
# BLOCO 7 - PREVIEW VISUAL (50 linhas × 20 colunas) - SUPORTE CSV
# ═══════════════════════════════════════════════════════════════════
# FIX: datetime → string ISO antes de salvar JSON (TypeError corrigido)
# ═══════════════════════════════════════════════════════════════════

print("\n" + "="*70)
print("👀 PREVIEW DO ARQUIVO")
print("="*70)

# ═══════════════════════════════════════════════════════════════════
# CASO 1: EXCEL com xlrd (arquivos .xls antigos)
# ═══════════════════════════════════════════════════════════════════

if metodo_carga == 'xlrd':
    print("📊 Método: xlrd")

    sheet = workbook.sheet_by_name(sheet_nome)
    data_preview = []

    for row_idx in range(min(50, sheet.nrows)):
        data_preview.append(sheet.row_values(row_idx))

    df_preview = pd.DataFrame(data_preview)

# ═══════════════════════════════════════════════════════════════════
# CASO 2: EXCEL com pandas (arquivos .xlsx/.xlsm)
# ═══════════════════════════════════════════════════════════════════

elif metodo_carga == 'pandas':
    print("📊 Método: pandas Excel")

    df_preview = pd.read_excel(
        workbook,
        sheet_name=sheet_nome,
        nrows=50,
        header=None
    )

# ═══════════════════════════════════════════════════════════════════
# CASO 3: CSV 🆕
# ═══════════════════════════════════════════════════════════════════

elif metodo_carga == 'csv':
    print("📄 Método: CSV")

    df_preview = pd.read_csv(
        arquivo_selecionado,
        sep=separador_detectado,
        encoding='cp1252',
        skiprows=skiprows_csv,
        nrows=50,
        header=None  # Sem cabeçalho por enquanto
    )

else:
    raise ValueError(f"❌ Método de carga desconhecido: {metodo_carga}")

# ═══════════════════════════════════════════════════════════════════
# LIMITAR A 20 COLUNAS PARA VISUALIZAÇÃO
# ═══════════════════════════════════════════════════════════════════

df_preview_limitado = df_preview.iloc[:, :20].copy()

# ═══════════════════════════════════════════════════════════════════
# EXIBIR INFORMAÇÕES
# ═══════════════════════════════════════════════════════════════════

print(f"\n📊 Dimensões do preview:")
print(f"   Total: {df_preview.shape[0]} linhas × {df_preview.shape[1]} colunas")
print(f"   Exibindo: {df_preview_limitado.shape[0]} linhas × {df_preview_limitado.shape[1]} colunas")

print(f"\n👁️  Preview (primeiras 50 linhas, até 20 colunas):")
print("─" * 70)

# Usar display ou print dependendo do ambiente
try:
    display(df_preview_limitado)
except NameError:
    print(df_preview_limitado.to_string())

print("─" * 70)

# ═══════════════════════════════════════════════════════════════════
# SALVAMENTO DE ESTADO E PREVIEW (ADICIONADO - NÃO REMOVE NADA ACIMA)
# ═══════════════════════════════════════════════════════════════════

# ✅ FIX: Função para converter datetime → string ISO
def converter_para_json(valor):
    """Converte datetime/Timestamp para string ISO, senão mantém valor"""
    if pd.isna(valor):
        return None
    elif isinstance(valor, (datetime, pd.Timestamp)):
        return valor.isoformat()
    else:
        return valor

# Salvar preview em JSON (seguindo padrão do BLOCO 8)
# ✅ FIX: Aplicar conversão em cada célula
preview_data_raw = df_preview_limitado.values.tolist()
preview_data_safe = [[converter_para_json(cel) for cel in row] for row in preview_data_raw]

preview_data = {
    'dimensoes': {
        'linhas_total': int(df_preview.shape[0]),
        'colunas_total': int(df_preview.shape[1]),
        'linhas_exibidas': int(df_preview_limitado.shape[0]),
        'colunas_exibidas': int(df_preview_limitado.shape[1])
    },
    'preview_limitado': preview_data_safe  # ✅ Agora é JSON-safe
}

preview_file = fm.pastas['logs'] / '.bloco_7_preview.json'
with open(preview_file, 'w', encoding='utf-8') as f:
    json.dump(preview_data, f, indent=2, ensure_ascii=False)

# Salvar estado do bloco
estado_bloco7 = {
    'timestamp': datetime.now().isoformat(),
    'bloco': 7,
    'nome': 'PREVIEW VISUAL',
    'status': 'concluido',
    'arquivo': arquivo_selecionado.name,
    'sheet': sheet_nome,
    'metodo_carga': metodo_carga,
    'dimensoes_preview': {
        'linhas_carregadas': int(df_preview.shape[0]),
        'colunas_carregadas': int(df_preview.shape[1]),
        'linhas_exibidas': int(df_preview_limitado.shape[0]),
        'colunas_exibidas': int(df_preview_limitado.shape[1])
    },
    'arquivo_preview': preview_file.name
}

arquivo_estado = fm.pastas['logs'] / '.bloco_7_state.json'
with open(arquivo_estado, 'w', encoding='utf-8') as f:
    json.dump(estado_bloco7, f, indent=2, ensure_ascii=False)

print(f"\n💾 Estado salvo: {arquivo_estado.name}")
print(f"💾 Preview salvo: {preview_file.name}")


👀 PREVIEW DO ARQUIVO
📊 Método: xlrd

📊 Dimensões do preview:
   Total: 50 linhas × 60 colunas
   Exibindo: 50 linhas × 20 colunas

👁️  Preview (primeiras 50 linhas, até 20 colunas):
──────────────────────────────────────────────────────────────────────


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,,,,,,,,,,,,,,,,VARIAÇÃO %,LIMITE TÉCNICO,,FALTA,SOBRA
1,,,,,,,,,,,,,,,,,,,,
2,,,,,,,,,,,,,,,,LIMITE DE COMPETÊNCIA,DIRETORIA,PRES,N2,N3
3,,,,,,,,,,,,,,,,R$ por produto / mês,">60.000.000,00",30000000.0,1000000.0,300000.0
4,,,,,,,,,,,,,,,,,,,,
5,,,,,,,,,,,,,,,,,,,,
6,,,,,,,,,,,,Variação de Estoque por Centro - GPA,,,,,,,,
7,,,,,,,,,,,,,,,,,,,,
8,,,,,,,,,,,,Centro de lucro,,,,,,,,
9,,,,,,,,,,,,Centro,,,,,,,,


──────────────────────────────────────────────────────────────────────

💾 Estado salvo: .bloco_7_state.json
💾 Preview salvo: .bloco_7_preview.json


In [32]:
# ═══════════════════════════════════════════════════════════════
# BLOCO 8 - DETECÇÃO E SELEÇÃO AVANÇADA DE CABEÇALHO - COMPLETO
# VERSÃO REVISADA v2.1 - CORREÇÕES DE BUGS
# Com: Dicionário + Análise Repetição + Multi-linha + Análise Colunas COMPLETA
# Mudanças v2.1:
#   - Corrigido bug string vazia no critério 12
#   - Corrigido penalidade diversidade no critério 5
#   - Tolerância a gaps de até 3 colunas inválidas
# ═══════════════════════════════════════════════════════════════

from difflib import SequenceMatcher
import re

print("\n" + "="*70)
print("🎯 DETECÇÃO E SELEÇÃO DE CABEÇALHO")
print("="*70)

# ═══════════════════════════════════════════════════════════════
# CARREGAR DICIONÁRIO PERSISTENTE (se não estiver carregado)
# ═══════════════════════════════════════════════════════════════

print("\n🔍 Verificando cabeçalho multi-linha...")

if 'DICIONARIO_PERSISTENTE' not in globals():
    print("\n📚 Carregando dicionário persistente...")

    locais_dicionario = [
        Path.cwd() / 'DICT_Dicionario_Persistente.json',
        fm.pastas['logs'] / 'DICT_Dicionario_Persistente.json',
        Path.cwd().parent / 'DICT_Dicionario_Persistente.json',
    ]

    DICIONARIO_PERSISTENTE = None

    for local in locais_dicionario:
        if local.exists():
            try:
                with open(local, 'r', encoding='utf-8') as f:
                    DICIONARIO_PERSISTENTE = json.load(f)
                print(f"   ✅ Carregado de: {local.name}")
                break
            except:
                continue

    if not DICIONARIO_PERSISTENTE:
        print(f"   ℹ️  Dicionário não encontrado - criando vazio")
        DICIONARIO_PERSISTENTE = {
            'arquivos': {},
            'ultima_atualizacao': None,
            'versao': '1.0'
        }
else:
    print("\n📚 Dicionário persistente já carregado")

# ═══════════════════════════════════════════════════════════════
# FUNÇÃO AVANÇADA DE AVALIAÇÃO DE LINHA
# ═══════════════════════════════════════════════════════════════

def avaliar_linha_cabecalho_avancada(
    linha, idx, total_linhas, df_preview, dicionario_persistente
):
    """Avalia linha como candidata a cabeçalho com múltiplas heurísticas."""
    celulas = [
        str(c).strip() for c in linha
        if str(c).strip() and str(c).strip().lower() not in
        ['nan', 'none', '']
    ]

    if not celulas:
        return {
            'score': 0.0,
            'detalhes': 'Linha vazia',
            'matches_dicionario': []
        }

    score = 0.0
    detalhes = []

    # CRITÉRIO 1: PROPORÇÃO DE CÉLULAS PREENCHIDAS (peso 2.0)
    prop_preenchidas = len(celulas) / len(linha)
    score_preenchidas = prop_preenchidas * 2.0
    score += score_preenchidas
    detalhes.append(
        f"Preench: {prop_preenchidas:.0%} (+{score_preenchidas:.1f})"
    )

    # CRITÉRIO 2: PROPORÇÃO DE TEXTO (peso 2.5)
    tem_texto = sum(1 for c in celulas if re.search(r'[a-zA-Z]', c))
    prop_texto = tem_texto / len(celulas) if celulas else 0
    score_texto = prop_texto * 2.5
    score += score_texto
    detalhes.append(f"Texto: {prop_texto:.0%} (+{score_texto:.1f})")

    # CRITÉRIO 3: MATCH COM DICIONÁRIO PERSISTENTE (peso 4.0)
    bonus_dicionario = 0.0
    matches_dicionario = []

    if dicionario_persistente and 'arquivos' in dicionario_persistente:
        campos_conhecidos = set()
        for arquivo_info in dicionario_persistente.get(
            'arquivos', {}
        ).values():
            if 'campos_mapeados' in arquivo_info:
                for campo_info in arquivo_info[
                    'campos_mapeados'
                ].values():
                    if 'nome_original' in campo_info:
                        campos_conhecidos.add(
                            campo_info['nome_original'].lower()
                        )
                    if 'nome_padrao' in campo_info:
                        campos_conhecidos.add(
                            campo_info['nome_padrao'].lower()
                        )

        for celula in celulas:
            celula_lower = celula.lower()
            if celula_lower in campos_conhecidos:
                bonus_dicionario += 0.8
                matches_dicionario.append(celula[:20])
            elif any(
                conhecido in celula_lower
                for conhecido in campos_conhecidos
            ):
                bonus_dicionario += 0.4
                matches_dicionario.append(f"{celula[:15]}*")

    bonus_dicionario = min(bonus_dicionario, 4.0)
    score += bonus_dicionario

    if bonus_dicionario > 0:
        detalhes.append(
            f"Dict: {len(matches_dicionario)}m (+{bonus_dicionario:.1f})"
        )

    # CRITÉRIO 4: ANÁLISE DE REPETIÇÃO (peso 3.0)
    try:
        linhas_futuras = min(20, total_linhas - idx - 1)

        if linhas_futuras >= 5:
            colunas_com_repeticao = 0
            total_colunas_analisadas = 0

            for col_idx, valor_atual in enumerate(linha):
                valor_atual_str = str(valor_atual).strip()

                if not valor_atual_str or valor_atual_str.lower() in [
                    'nan', 'none', ''
                ]:
                    continue

                total_colunas_analisadas += 1

                repeticoes = 0
                for i in range(1, min(linhas_futuras + 1, 21)):
                    if idx + i < len(df_preview):
                        valor_futuro = str(
                            df_preview.iloc[idx + i, col_idx]
                        ).strip()
                        if valor_futuro == valor_atual_str:
                            repeticoes += 1

                if repeticoes >= 2:
                    colunas_com_repeticao += 1

            if total_colunas_analisadas > 0:
                prop_repeticao = (
                    colunas_com_repeticao / total_colunas_analisadas
                )
                score_repeticao = (1 - prop_repeticao) * 3.0
                score += score_repeticao
                detalhes.append(
                    f"Unic: {(1-prop_repeticao):.0%} "
                    f"(+{score_repeticao:.1f})"
                )
    except:
        pass

    # CRITÉRIO 5: TAMANHO MÉDIO DE STRINGS (peso 1.0)
    tamanho_medio = np.mean([len(c) for c in celulas]) if celulas else 0
    if 5 <= tamanho_medio <= 50:
        score += 1.0
        detalhes.append(f"Tam: {tamanho_medio:.0f} (+1.0)")

    # CRITÉRIO 6: UNICIDADE DENTRO DA LINHA (peso 1.5)
    if len(celulas) == len(set(celulas)):
        score += 1.5
        detalhes.append("Únicos (+1.5)")

    # CRITÉRIO 7: POSIÇÃO NO ARQUIVO (peso 0.5)
    if idx < 50:
        bonus_posicao = (50 - idx) / 100
        score += bonus_posicao
        detalhes.append(f"Pos: {idx+1} (+{bonus_posicao:.2f})")

    # CRITÉRIO 8: ANÁLISE DE DADOS ABAIXO (peso 2.5)
    try:
        if idx + 5 < total_linhas:
            celulas_match_dict = 0
            celulas_numericas_puras = 0
            celulas_com_numeros = 0
            total_celulas_validas = 0

            campos_dict_lower = set()
            if dicionario_persistente and 'arquivos' in dicionario_persistente:
                for arquivo_info in dicionario_persistente.get(
                    'arquivos', {}
                ).values():
                    if 'campos_mapeados' in arquivo_info:
                        for campo_info in arquivo_info[
                            'campos_mapeados'
                        ].values():
                            if 'nome_original' in campo_info:
                                campos_dict_lower.add(
                                    campo_info['nome_original'].lower()
                                )
                            if 'nome_padrao' in campo_info:
                                campos_dict_lower.add(
                                    campo_info['nome_padrao'].lower()
                                )
                            if 'sinonimos' in campo_info:
                                for sin in campo_info['sinonimos']:
                                    campos_dict_lower.add(sin.lower())

            for offset in range(1, 6):
                if idx + offset < len(df_preview):
                    linha_seguinte = df_preview.iloc[idx + offset]

                    for celula in linha_seguinte:
                        celula_str = str(celula).strip()

                        if not celula_str or celula_str.lower() in [
                            'nan', 'none', ''
                        ]:
                            continue

                        total_celulas_validas += 1
                        celula_lower = celula_str.lower()

                        matched = False
                        if campos_dict_lower:
                            if celula_lower in campos_dict_lower:
                                celulas_match_dict += 1
                                matched = True
                            else:
                                for campo_conhecido in campos_dict_lower:
                                    if (campo_conhecido in celula_lower or
                                        celula_lower in campo_conhecido):
                                        if len(campo_conhecido) >= 3:
                                            celulas_match_dict += 1
                                            matched = True
                                            break

                        if not matched:
                            apenas_numeros = re.sub(
                                r'[^0-9.]', '', celula_str
                            )

                            if len(apenas_numeros) > 0:
                                prop_digitos = (
                                    len(apenas_numeros) / len(celula_str)
                                )
                                if prop_digitos > 0.5:
                                    celulas_numericas_puras += 1
                                elif re.search(r'\d', celula_str):
                                    celulas_com_numeros += 1

            if total_celulas_validas > 0:
                prop_dict = celulas_match_dict / total_celulas_validas
                prop_num_puras = (
                    celulas_numericas_puras / total_celulas_validas
                )
                prop_com_num = celulas_com_numeros / total_celulas_validas

                bonus_dados = 0.0
                metodo_usado = None

                if prop_dict > 0.4:
                    bonus_dados = 2.5
                    metodo_usado = f"Dict:{prop_dict:.0%}"
                elif prop_num_puras > 0.6:
                    bonus_dados = 2.0
                    metodo_usado = f"Num:{prop_num_puras:.0%}"
                elif (prop_num_puras + prop_com_num) > 0.7:
                    bonus_dados = 1.0
                    metodo_usado = f"Misto:{(prop_num_puras+prop_com_num):.0%}"

                if bonus_dados > 0:
                    score += bonus_dados
                    detalhes.append(
                        f"DadosAbaixo:{metodo_usado} (+{bonus_dados:.1f})"
                    )
    except:
        pass

    # CRITÉRIO 9: ANTI-DADOS (penalidade)
    try:
        celulas_linha_atual = [
            str(c).strip() for c in linha
            if str(c).strip() and str(c).strip().lower() not in
            ['nan', 'none', '']
        ]

        if celulas_linha_atual:
            num_puras_linha = 0
            for celula in celulas_linha_atual:
                apenas_numeros = re.sub(r'[^0-9.]', '', celula)
                if len(apenas_numeros) > 0:
                    prop_digitos = len(apenas_numeros) / len(celula)
                    if prop_digitos > 0.5:
                        num_puras_linha += 1

            prop_num_linha = num_puras_linha / len(celulas_linha_atual)

            repeticoes_detectadas = 0
            if idx + 5 < total_linhas:
                for col_idx, valor_atual in enumerate(linha):
                    valor_atual_str = str(valor_atual).strip()

                    if (not valor_atual_str or
                        valor_atual_str.lower() in ['nan', 'none', '']):
                        continue

                    for offset in range(1, min(6, total_linhas - idx)):
                        if idx + offset < len(df_preview):
                            valor_seguinte = str(
                                df_preview.iloc[idx + offset, col_idx]
                            ).strip()
                            if valor_seguinte == valor_atual_str:
                                repeticoes_detectadas += 1
                                break

            prop_repeticoes = (
                repeticoes_detectadas / len(celulas_linha_atual)
                if celulas_linha_atual else 0
            )

            penalidade = 0.0

            if prop_num_linha > 0.6 and prop_repeticoes > 0.3:
                penalidade = -3.0
                score += penalidade
                detalhes.append(
                    f"AntiDados:Num{prop_num_linha:.0%}+Rep"
                    f"{prop_repeticoes:.0%} ({penalidade:.1f})"
                )
            elif prop_num_linha > 0.7:
                penalidade = -1.5
                score += penalidade
                detalhes.append(
                    f"AntiDados:Num{prop_num_linha:.0%} ({penalidade:.1f})"
                )
    except:
        pass

    # CRITÉRIO 10: PADRÃO DE RÓTULOS (+4.0 pontos)
    try:
        palavras_rotulo = [
            'centro', 'produto', 'material', 'data', 'valor', 'quantidade',
            'codigo', 'nome', 'descricao', 'tipo', 'categoria', 'grupo',
            'sigla', 'unidade', 'medida', 'periodo', 'mes', 'ano',
            'referencia', 'documento', 'numero', 'id', 'chave', 'hierarq',
            'lucro', 'receita', 'custo', 'despesa', 'saldo', 'total',
            'indice', 'variacao', 'percentual', 'taxa', 'margem'
        ]

        if dicionario_persistente and 'arquivos' in dicionario_persistente:
            for arquivo_info in dicionario_persistente.get(
                'arquivos', {}
            ).values():
                if 'campos_mapeados' in arquivo_info:
                    for campo_info in arquivo_info[
                        'campos_mapeados'
                    ].values():
                        if 'nome_original' in campo_info:
                            palavras_rotulo.append(
                                campo_info['nome_original'].lower()
                            )
                        if 'nome_padrao' in campo_info:
                            palavras_rotulo.append(
                                campo_info['nome_padrao'].lower()
                            )

        palavras_rotulo = set(palavras_rotulo)

        matches_rotulo = 0
        celulas_validas = 0

        for celula in linha:
            celula_str = str(celula).strip()
            if not celula_str or celula_str.lower() in ['nan', 'none', '']:
                continue

            celulas_validas += 1
            celula_lower = celula_str.lower()

            if celula_lower in palavras_rotulo:
                matches_rotulo += 1
            else:
                for palavra in palavras_rotulo:
                    if len(palavra) >= 4 and palavra in celula_lower:
                        matches_rotulo += 1
                        break

        if celulas_validas > 0:
            prop_rotulos = matches_rotulo / celulas_validas

            if prop_rotulos > 0.4:
                bonus_rotulos = 4.0
                score += bonus_rotulos
                detalhes.append(
                    f"Rotulos:{prop_rotulos:.0%} (+{bonus_rotulos:.1f})"
                )
            elif prop_rotulos > 0.25:
                bonus_rotulos = 2.0
                score += bonus_rotulos
                detalhes.append(
                    f"Rotulos:{prop_rotulos:.0%} (+{bonus_rotulos:.1f})"
                )
    except:
        pass

    # CRITÉRIO 11: ANTI-REPETIÇÃO FORTE (-4.0 pontos)
    try:
        if idx + 10 < total_linhas:
            colunas_com_repeticao_forte = 0
            total_colunas_analisadas = 0

            for col_idx, valor_atual in enumerate(linha):
                valor_atual_str = str(valor_atual).strip()

                if not valor_atual_str or valor_atual_str.lower() in [
                    'nan', 'none', ''
                ]:
                    continue

                total_colunas_analisadas += 1

                repeticoes = 0
                for offset in range(1, min(11, total_linhas - idx)):
                    if idx + offset < len(df_preview):
                        valor_seg = str(
                            df_preview.iloc[idx + offset, col_idx]
                        ).strip()
                        if valor_seg == valor_atual_str:
                            repeticoes += 1

                if repeticoes >= 5:
                    colunas_com_repeticao_forte += 1

            if total_colunas_analisadas > 0:
                prop_rep_forte = (
                    colunas_com_repeticao_forte / total_colunas_analisadas
                )

                if prop_rep_forte > 0.3:
                    penalidade_rep = -4.0
                    score += penalidade_rep
                    detalhes.append(
                        f"AntiRep:{prop_rep_forte:.0%} "
                        f"({penalidade_rep:.1f})"
                    )
    except:
        pass

    # CRITÉRIO 12: DENSIDADE DE RÓTULOS DO DICIONÁRIO (+3.0 pontos)
    try:
        if dicionario_persistente and 'arquivos' in dicionario_persistente:
            campos_conhecidos = {}

            for arquivo_info in dicionario_persistente.get(
                'arquivos', {}
            ).values():
                if 'campos_mapeados' in arquivo_info:
                    for campo_info in arquivo_info[
                        'campos_mapeados'
                    ].values():
                        nome_orig = campo_info.get('nome_original', '')
                        nome_pad = campo_info.get('nome_padrao', '')

                        if nome_orig:
                            campos_conhecidos[nome_orig.lower()] = True
                        if nome_pad:
                            campos_conhecidos[nome_pad.lower()] = True

            if campos_conhecidos:
                matches_exatos = 0
                celulas_validas = 0

                for celula in linha:
                    celula_str = str(celula).strip()
                    if not celula_str or celula_str.lower() in [
                        'nan', 'none', ''
                    ]:
                        continue

                    celulas_validas += 1
                    celula_lower = celula_str.lower()

                    if celula_lower in campos_conhecidos:
                        matches_exatos += 1

                if celulas_validas > 0:
                    prop_dict_exato = matches_exatos / celulas_validas

                    if prop_dict_exato > 0.5:
                        bonus_dict_dens = 3.0
                        score += bonus_dict_dens
                        detalhes.append(
                            f"DictDens:{prop_dict_exato:.0%} "
                            f"(+{bonus_dict_dens:.1f})"
                        )
                    elif prop_dict_exato > 0.3:
                        bonus_dict_dens = 1.5
                        score += bonus_dict_dens
                        detalhes.append(
                            f"DictDens:{prop_dict_exato:.0%} "
                            f"(+{bonus_dict_dens:.1f})"
                        )
    except:
        pass

    return {
        'score': score,
        'detalhes': ' | '.join(detalhes),
        'matches_dicionario': matches_dicionario
    }

# ═══════════════════════════════════════════════════════════════
# AVALIAR TODAS AS LINHAS
# ═══════════════════════════════════════════════════════════════

print("\n📊 Analisando linhas para detectar cabeçalho...")

if metodo_carga == 'csv':
    data_para_analise = df_preview.values.tolist()
elif metodo_carga == 'xlrd':
    data_para_analise = []
    sheet = workbook.sheet_by_name(sheet_nome)
    for row_idx in range(min(50, sheet.nrows)):
        data_para_analise.append(sheet.row_values(row_idx))
else:
    data_para_analise = df_preview.values.tolist()

scores = []

for idx, linha in enumerate(data_para_analise):
    resultado = avaliar_linha_cabecalho_avancada(
        linha,
        idx,
        len(data_para_analise),
        df_preview,
        DICIONARIO_PERSISTENTE
    )

    scores.append({
        'linha_excel': idx + 1,
        'indice': idx,
        'score': resultado['score'],
        'detalhes': resultado['detalhes'],
        'matches': resultado['matches_dicionario']
    })

scores = sorted(scores, key=lambda x: x['score'], reverse=True)

# ═══════════════════════════════════════════════════════════════
# EXIBIR TOP 5 CANDIDATOS
# ═══════════════════════════════════════════════════════════════

print("\n🏆 Top 5 candidatos a cabeçalho:")
print("=" * 70)
print("📍 NUMERAÇÃO: Usamos índice Python (preview inicia em 0)")
print("   • Índice 0 = Linha 1 no Excel")
print("=" * 70)

for i, item in enumerate(scores[:5], 1):
    idx_py = item['indice']
    linha_excel = item['linha_excel']

    print(f"\n   {i}º. Índice {idx_py} (Excel: Linha {linha_excel})")
    print(f"       Score: {item['score']:.2f}/24.5")
    print(f"       {item['detalhes']}")
    if item['matches']:
        matches_str = ', '.join(item['matches'][:5])
        print(f"       Matches: {matches_str}")

melhor = scores[0]
print(f"\n{'='*70}")
print(
    f"🎯 SUGESTÃO AUTOMÁTICA: Índice {melhor['indice']} "
    f"(Excel: Linha {melhor['linha_excel']})"
)
print(f"   Confiança: {melhor['score']:.2f}/24.5")
print("=" * 70)

# ═══════════════════════════════════════════════════════════════
# ANÁLISE DE COLUNAS VÁLIDAS - SISTEMA COMPLETO v2.1
# CORREÇÃO: Bugs nos critérios 5 e 12
# ═══════════════════════════════════════════════════════════════

def analisar_coluna_valida_COMPLETA(
    col_idx,
    nome_coluna,
    dados_coluna,
    dicionario,
    todas_colunas_info=None
):
    """
    Análise COMPLETA de coluna com 12 critérios avançados.
    Funciona para TABELAS TRANSACIONAIS e RELATÓRIOS BI.
    """
    score = 0.0
    razoes = []
    metodo_usado = None

    valores = [
        str(v).strip() for v in dados_coluna
        if str(v).strip() and str(v).strip().lower() not in ['nan', 'none', '']
    ]

    if not valores:
        return {
            'valida': False,
            'score': 0.0,
            'razoes': ['Coluna vazia'],
            'tipo_detectado': 'VAZIA',
            'confianca': 0.0,
            'metodo': 'VAZIO',
            'prop_preenchimento': 0.0,
            'match_dicionario': None
        }

    # CRITÉRIO 1: SIMILARIDADE COM ALIASES DO DICIONÁRIO (peso 8.0)
    nome_lower = str(nome_coluna).lower().strip()
    melhor_match_alias = None
    melhor_score_alias = 0.0
    campo_matched = None

    if dicionario and 'arquivos' in dicionario:
        aliases_por_campo = {}

        for arq_info in dicionario.get('arquivos', {}).values():
            if 'campos_mapeados' in arq_info:
                for nome_campo, campo_info in arq_info['campos_mapeados'].items():
                    if nome_campo not in aliases_por_campo:
                        aliases_por_campo[nome_campo] = set()

                    if 'nome_original' in campo_info:
                        aliases_por_campo[nome_campo].add(
                            campo_info['nome_original'].lower()
                        )

                    if 'nome_padrao' in campo_info:
                        aliases_por_campo[nome_campo].add(
                            campo_info['nome_padrao'].lower()
                        )

                    if 'sinonimos' in campo_info:
                        for sin in campo_info['sinonimos']:
                            aliases_por_campo[nome_campo].add(sin.lower())

        for campo, aliases in aliases_por_campo.items():
            for alias in aliases:
                if nome_lower == alias:
                    melhor_score_alias = 1.0
                    melhor_match_alias = alias
                    campo_matched = campo
                    break

                similaridade = SequenceMatcher(None, nome_lower, alias).ratio()

                if similaridade > melhor_score_alias:
                    melhor_score_alias = similaridade
                    melhor_match_alias = alias
                    campo_matched = campo

            if melhor_score_alias == 1.0:
                break

    if melhor_score_alias >= 0.95:
        bonus_alias = 8.0
        score += bonus_alias
        razoes.append(f"Alias:Exato({melhor_score_alias:.0%}) +{bonus_alias:.1f}")
        metodo_usado = 'ALIAS_EXATO'

    elif melhor_score_alias >= 0.80:
        bonus_alias = 6.0
        score += bonus_alias
        razoes.append(f"Alias:Similar({melhor_score_alias:.0%}) +{bonus_alias:.1f}")
        metodo_usado = 'ALIAS_SIMILAR'

    elif melhor_score_alias >= 0.60:
        bonus_alias = 3.0
        score += bonus_alias
        razoes.append(f"Alias:Parcial({melhor_score_alias:.0%}) +{bonus_alias:.1f}")
        metodo_usado = 'ALIAS_PARCIAL'

    # CRITÉRIO 2: REGEX NOS CONTEÚDOS vs DICIONÁRIO (peso 7.0)
    melhor_match_regex = None
    melhor_score_regex = 0.0

    if dicionario and 'arquivos' in dicionario:
        padroes_por_campo = {}

        for arq_info in dicionario.get('arquivos', {}).values():
            if 'campos_mapeados' in arq_info:
                for nome_campo, campo_info in arq_info['campos_mapeados'].items():
                    if 'regex' in campo_info:
                        if nome_campo not in padroes_por_campo:
                            padroes_por_campo[nome_campo] = []
                        padroes_por_campo[nome_campo].append(campo_info['regex'])

        for campo, padroes in padroes_por_campo.items():
            for padrao in padroes:
                try:
                    matches = sum(
                        1 for v in valores[:50]
                        if re.match(padrao, v, re.IGNORECASE)
                    )

                    prop_matches = matches / min(len(valores), 50)

                    if prop_matches > melhor_score_regex:
                        melhor_score_regex = prop_matches
                        melhor_match_regex = campo
                except:
                    continue

    if melhor_score_regex >= 0.80:
        bonus_regex = 7.0
        score += bonus_regex
        razoes.append(f"Regex:{melhor_score_regex:.0%} +{bonus_regex:.1f}")
        if not metodo_usado:
            metodo_usado = 'REGEX_CONTEUDO'

    elif melhor_score_regex >= 0.60:
        bonus_regex = 4.0
        score += bonus_regex
        razoes.append(f"Regex:{melhor_score_regex:.0%} +{bonus_regex:.1f}")
        if not metodo_usado:
            metodo_usado = 'REGEX_PARCIAL'

    # CRITÉRIO 3: SIMILARIDADE COM CONTEÚDOS CONHECIDOS (peso 6.0)
    melhor_match_conteudo = None
    melhor_score_conteudo = 0.0

    if dicionario and 'arquivos' in dicionario:
        exemplos_por_campo = {}

        for arq_info in dicionario.get('arquivos', {}).values():
            if 'campos_mapeados' in arq_info:
                for nome_campo, campo_info in arq_info['campos_mapeados'].items():
                    if 'exemplos' in campo_info:
                        if nome_campo not in exemplos_por_campo:
                            exemplos_por_campo[nome_campo] = set()

                        for exemplo in campo_info['exemplos']:
                            exemplos_por_campo[nome_campo].add(
                                str(exemplo).lower().strip()
                            )

        for campo, exemplos in exemplos_por_campo.items():
            matches = 0
            for valor in valores[:50]:
                valor_lower = valor.lower()

                if valor_lower in exemplos:
                    matches += 1
                else:
                    for exemplo in exemplos:
                        sim = SequenceMatcher(None, valor_lower, exemplo).ratio()
                        if sim >= 0.85:
                            matches += 1
                            break

            prop_matches = matches / min(len(valores), 50)

            if prop_matches > melhor_score_conteudo:
                melhor_score_conteudo = prop_matches
                melhor_match_conteudo = campo

    if melhor_score_conteudo >= 0.70:
        bonus_conteudo = 6.0
        score += bonus_conteudo
        razoes.append(f"Conteudo:{melhor_score_conteudo:.0%} +{bonus_conteudo:.1f}")
        if not metodo_usado:
            metodo_usado = 'CONTEUDO_SIMILAR'

    elif melhor_score_conteudo >= 0.50:
        bonus_conteudo = 3.0
        score += bonus_conteudo
        razoes.append(f"Conteudo:{melhor_score_conteudo:.0%} +{bonus_conteudo:.1f}")

    # CRITÉRIO 4: DETECÇÃO DE FÓRMULAS (penalidade -8.0)
    tem_formulas = False

    padroes_formula = [
        r'^=',
        r'^\+',
        r'^SUM\(',
        r'^IF\(',
        r'^VLOOKUP\(',
    ]

    for valor in valores[:20]:
        for padrao in padroes_formula:
            if re.match(padrao, valor, re.IGNORECASE):
                tem_formulas = True
                break
        if tem_formulas:
            break

    if tem_formulas:
        penalidade_formula = -8.0
        score += penalidade_formula
        razoes.append(f"Formula! {penalidade_formula:.1f}")
        if not metodo_usado:
            metodo_usado = 'FORMULA_DETECTADA'

    # CRITÉRIO 5: DIVERSIDADE DE VALORES (peso 4.0)
    # CORREÇÃO v2.1: Não penalizar dimensões BI com baixa diversidade na amostra
    valores_unicos = len(set(valores))
    total_valores = len(valores)
    prop_unicos = valores_unicos / total_valores if total_valores else 0

    if prop_unicos > 0.7:
        score += 4.0
        razoes.append(f"Divers:{prop_unicos:.0%} (+4.0)")
    elif prop_unicos > 0.4:
        score += 2.0
        razoes.append(f"Divers:{prop_unicos:.0%} (+2.0)")
    elif prop_unicos < 0.05 and valores_unicos <= 3:
        # Só penalizar se REALMENTE for flag (≤3 valores únicos E <5%)
        score -= 3.0
        razoes.append(f"Divers:{prop_unicos:.0%} (-3.0)")

    # CRITÉRIO 6: PADRÃO DE FLAGS (penalidade -4.0)
    flags_comuns = {'true', 'false', 'x', '✓', '0', '1', 'sim', 'não', 'yes', 'no'}
    valores_lower = [v.lower() for v in valores]

    matches_flag = sum(1 for v in valores_lower if v in flags_comuns)
    prop_flags = matches_flag / total_valores if total_valores else 0

    if prop_flags > 0.8:
        score -= 4.0
        razoes.append(f"Flags:{prop_flags:.0%} (-4.0)")

    # CRITÉRIO 7: PALAVRAS-CHAVE DE DADOS (peso 3.0)
    palavras_dados = [
        'centro', 'produto', 'material', 'codigo', 'nome', 'data',
        'valor', 'quantidade', 'preco', 'custo', 'receita',
        'hierarq', 'grupo', 'categoria', 'tipo', 'unidade',
        'periodo', 'mes', 'ano', 'sigla', 'descricao', 'lucro'
    ]

    tem_palavra_chave = any(palavra in nome_lower for palavra in palavras_dados)

    if tem_palavra_chave:
        score += 3.0
        razoes.append(f"Keyword (+3.0)")

    # CRITÉRIO 8: TAMANHO MÉDIO DOS VALORES (peso 2.0)
    tamanho_medio = sum(len(v) for v in valores) / len(valores)

    if 3 <= tamanho_medio <= 100:
        score += 2.0
        razoes.append(f"Tam:{tamanho_medio:.0f} (+2.0)")
    elif tamanho_medio <= 2:
        score -= 2.0
        razoes.append(f"Tam:{tamanho_medio:.0f} (-2.0)")

    # CRITÉRIO 9: MIX NUMÉRICO/ALFABÉTICO (peso 1.0)
    tem_numeros = sum(1 for v in valores if any(c.isdigit() for c in v))
    tem_letras = sum(1 for v in valores if any(c.isalpha() for c in v))

    if tem_numeros > 0 and tem_letras > 0:
        score += 1.0
        razoes.append(f"Mix (+1.0)")

    # CRITÉRIO 10: PREENCHIMENTO PARCIAL (penalidade -5.0)
    prop_preenchimento = len(valores) / len(dados_coluna) if dados_coluna else 0

    if prop_preenchimento < 0.30:
        penalidade_parcial = -5.0
        score += penalidade_parcial
        razoes.append(f"Parcial:{prop_preenchimento:.0%} {penalidade_parcial:.1f}")
        if not metodo_usado:
            metodo_usado = 'PREENCHIMENTO_PARCIAL'

    # CRITÉRIO 11: MUDANÇA ESTRUTURAL (penalidade -6.0)
    if todas_colunas_info and col_idx > 0:
        colunas_anteriores = todas_colunas_info[:col_idx]

        if len(colunas_anteriores) >= 5:
            preench_ultimas_5 = sum(
                c.get('prop_preenchimento', 1.0)
                for c in colunas_anteriores[-5:]
            ) / 5

            if preench_ultimas_5 > 0.80 and prop_preenchimento < 0.50:
                penalidade_estrutural = -6.0
                score += penalidade_estrutural
                razoes.append(
                    f"Estrutural:Queda ({penalidade_estrutural:.1f})"
                )
                if not metodo_usado:
                    metodo_usado = 'MUDANCA_ESTRUTURAL'

    # CRITÉRIO 12: PADRÃO DE NOME "VAZIO" (penalidade -7.0)
    # CORREÇÃO v2.1: Remover string vazia da lista
    nomes_vazios = ['unnamed', 'column', 'col', 'field', 'nan', 'none']

    nome_eh_vazio = (
        len(nome_lower) == 0 or
        nome_lower in nomes_vazios or
        (len(nome_lower) < 15 and any(vazio in nome_lower for vazio in nomes_vazios if vazio))
    )

    if nome_eh_vazio:
        penalidade_nome = -7.0
        score += penalidade_nome
        razoes.append(f"NomeVazio {penalidade_nome:.1f}")

    # DECISÃO FINAL
    confianca = max(0.0, min(1.0, (score + 10) / 40))

    if score >= 10.0:
        tipo = "DADOS"
        valida = True
    elif score >= 0.0:
        tipo = "INCERTO"
        valida = True
    else:
        tipo = "FLAG/FORMULA/AUXILIAR"
        valida = False

    if not metodo_usado:
        metodo_usado = 'HEURISTICAS_BASICAS'

    return {
        'valida': valida,
        'score': score,
        'razoes': razoes,
        'tipo_detectado': tipo,
        'confianca': confianca,
        'metodo': metodo_usado,
        'prop_preenchimento': prop_preenchimento,
        'match_dicionario': campo_matched or melhor_match_regex or melhor_match_conteudo
    }

# ═══════════════════════════════════════════════════════════════
# EXECUTAR ANÁLISE DE COLUNAS
# ═══════════════════════════════════════════════════════════════

print("\n" + "="*70)
print("🔍 ANALISANDO COLUNAS (Sistema Avançado v2.1)")
print("="*70)
print("Critérios: Similaridade + Regex + Conteúdo + Fórmulas + Estrutura")
print("Funciona para: Tabelas Transacionais e Relatórios BI")
print("="*70)

linha_cabecalho_detectado = data_para_analise[melhor['indice']]

inicio_dados = melhor['indice'] + 1
fim_dados = min(inicio_dados + 50, len(data_para_analise))
dados_para_colunas = data_para_analise[inicio_dados:fim_dados]

print(f"\n📊 Analisando {len(linha_cabecalho_detectado)} colunas...")
print(f"   Amostra de dados: {fim_dados - inicio_dados} linhas")

# PRIMEIRA PASSAGEM: Informações básicas
todas_colunas_info = []

for col_idx, nome_col in enumerate(linha_cabecalho_detectado):
    valores_col = [linha[col_idx] for linha in dados_para_colunas]

    valores_validos = [
        str(v).strip() for v in valores_col
        if str(v).strip() and str(v).strip().lower() not in ['nan', 'none', '']
    ]

    prop_preenchimento = len(valores_validos) / len(valores_col) if valores_col else 0

    todas_colunas_info.append({
        'indice': col_idx,
        'nome': str(nome_col),
        'valores': valores_col,
        'prop_preenchimento': prop_preenchimento
    })

# SEGUNDA PASSAGEM: Análise completa
colunas_analise = []

for col_info in todas_colunas_info:
    col_idx = col_info['indice']
    nome_col = col_info['nome']
    valores_col = col_info['valores']

    analise = analisar_coluna_valida_COMPLETA(
        col_idx,
        nome_col,
        valores_col,
        DICIONARIO_PERSISTENTE,
        todas_colunas_info=todas_colunas_info
    )

    colunas_analise.append({
        'indice': col_idx,
        'excel_col': col_idx + 1,
        'nome': str(nome_col)[:30],
        **analise
    })

colunas_validas = [c for c in colunas_analise if c['valida']]
colunas_invalidas = [c for c in colunas_analise if not c['valida']]

print(f"\n✅ Colunas VÁLIDAS (dados reais): {len(colunas_validas)}")
print(f"❌ Colunas INVÁLIDAS (flags/fórmulas/auxiliares): {len(colunas_invalidas)}")

# EXIBIR INVÁLIDAS
if colunas_invalidas:
    print(f"\n❌ COLUNAS DETECTADAS COMO INVÁLIDAS:")
    print(f"{'='*70}")

    for col in colunas_invalidas[:15]:
        print(
            f"   Col {col['excel_col']:2d} (idx {col['indice']:2d}): "
            f"{col['nome'][:25]:<25} | "
            f"Score: {col['score']:+6.1f} | "
            f"{col['tipo_detectado']}"
        )

        if col['razoes']:
            razoes_str = ' | '.join(col['razoes'][:4])
            print(f"      Razões: {razoes_str}")

        if col.get('match_dicionario'):
            print(f"      Match: {col['match_dicionario']}")

# EXIBIR VÁLIDAS
if colunas_validas:
    print(f"\n✅ TOP 10 COLUNAS VÁLIDAS (maiores scores):")
    print(f"{'='*70}")

    colunas_validas_sorted = sorted(
        colunas_validas,
        key=lambda x: x['score'],
        reverse=True
    )

    for col in colunas_validas_sorted[:10]:
        print(
            f"   Col {col['excel_col']:2d} (idx {col['indice']:2d}): "
            f"{col['nome'][:25]:<25} | "
            f"Score: {col['score']:+6.1f} | "
            f"Conf: {col['confianca']:.0%}"
        )

        if col['razoes']:
            razoes_str = ' | '.join(col['razoes'][:3])
            print(f"      {razoes_str}")

        if col.get('match_dicionario'):
            print(f"      ✓ Match: {col['match_dicionario']}")

# ═══════════════════════════════════════════════════════════════
# DETECTAR MUDANÇAS ESTRUTURAIS E AGRUPAR EM BLOCOS CONTÍNUOS
# v2.1: Tolerância a gaps de até 3 colunas inválidas
# ═══════════════════════════════════════════════════════════════

print(f"\n🔍 DETECTANDO MUDANÇAS ESTRUTURAIS:")
print(f"{'='*70}")

# Coletar índices de colunas válidas
colunas_validas_indices = [c['excel_col'] for c in colunas_analise if c['valida']]

if not colunas_validas_indices:
    blocos_continuos = []
else:
    # Agrupar com tolerância a gaps de até 3 colunas
    blocos_continuos = []
    bloco_atual = [colunas_validas_indices[0]]

    for i in range(1, len(colunas_validas_indices)):
        col_atual = colunas_validas_indices[i]
        col_anterior = colunas_validas_indices[i-1]

        gap = col_atual - col_anterior - 1

        # Se gap <= 3, considerar mesmo bloco (BI pode ter colunas vazias/auxiliares no meio)
        # Se gap > 3, começar novo bloco
        if gap <= 3:
            bloco_atual.append(col_atual)
        else:
            blocos_continuos.append(bloco_atual)
            bloco_atual = [col_atual]

    if bloco_atual:
        blocos_continuos.append(bloco_atual)

if len(blocos_continuos) > 1:
    print(f"\n⚠️  MÚLTIPLAS TABELAS DETECTADAS!")

    for i, bloco in enumerate(blocos_continuos, 1):
        primeira = min(bloco)
        ultima = max(bloco)
        tamanho = len(bloco)

        range_completo = ultima - primeira + 1
        gaps_internos = range_completo - tamanho

        print(f"\n   Tabela {i}:")
        print(f"      Range Excel: {primeira} a {ultima}")
        print(f"      Colunas válidas: {tamanho}")
        if gaps_internos > 0:
            print(f"      Gaps tolerados: {gaps_internos} col(s)")

        if i == 1:
            print(f"      ✓ TABELA PRINCIPAL (use esta!)")
        else:
            print(f"      ⚠️  Tabela auxiliar/complementar")

    primeira_valida = min(blocos_continuos[0])
    ultima_valida = max(blocos_continuos[0])

    print(f"\n💡 RECOMENDAÇÃO:")
    print(f"   Use apenas TABELA PRINCIPAL: colunas {primeira_valida} a {ultima_valida}")

else:
    if colunas_validas:
        primeira_valida = min(c['excel_col'] for c in colunas_validas)
        ultima_valida = max(c['excel_col'] for c in colunas_validas)

        print(f"\n✓ Estrutura contínua detectada")
        print(f"   Colunas válidas: {primeira_valida} a {ultima_valida}")

# DETERMINAR RANGE FINAL
if blocos_continuos:
    col_inicio_sugerido = min(blocos_continuos[0])
    col_fim_sugerido = max(blocos_continuos[0])

    total_range = col_fim_sugerido - col_inicio_sugerido + 1
    total_validas = len(blocos_continuos[0])
    total_gaps = total_range - total_validas
else:
    if colunas_validas:
        col_inicio_sugerido = min(c['excel_col'] for c in colunas_validas)
        col_fim_sugerido = max(c['excel_col'] for c in colunas_validas)
        total_range = col_fim_sugerido - col_inicio_sugerido + 1
        total_validas = len(colunas_validas)
        total_gaps = total_range - total_validas
    else:
        col_inicio_sugerido = 1
        col_fim_sugerido = len(linha_cabecalho_detectado)
        total_range = col_fim_sugerido
        total_validas = 0
        total_gaps = 0

print(f"\n🎯 RANGE FINAL SUGERIDO:")
print(f"{'='*70}")
print(f"   Excel: {col_inicio_sugerido} a {col_fim_sugerido}")
print(f"   Python: {col_inicio_sugerido-1} a {col_fim_sugerido}")
print(f"   Total range: {total_range} colunas")
print(f"   Colunas válidas: {total_validas}")
if total_gaps > 0:
    print(f"   Gaps internos: {total_gaps} (tolerados)")

if col_inicio_sugerido > 1:
    colunas_ignoradas = col_inicio_sugerido - 1
    print(f"\n   ⚠️  Ignorando colunas 1-{colunas_ignoradas}")

if len(blocos_continuos) > 1:
    total_auxiliares = sum(len(bloco) for bloco in blocos_continuos[1:])
    print(f"   ⚠️  Ignorando {total_auxiliares} colunas de tabelas auxiliares")

print("=" * 70)

# SALVAR RELATÓRIO
relatorio_colunas = {
    'timestamp': datetime.now().isoformat(),
    'arquivo': arquivo_selecionado.name,
    'sheet': sheet_nome,
    'total_colunas': len(colunas_analise),
    'colunas_validas': len(colunas_validas),
    'colunas_invalidas': len(colunas_invalidas),
    'blocos_detectados': len(blocos_continuos),
    'range_sugerido': {
        'inicio': col_inicio_sugerido,
        'fim': col_fim_sugerido,
        'total_range': total_range if blocos_continuos else col_fim_sugerido - col_inicio_sugerido + 1,
        'total_validas': total_validas if blocos_continuos else len(colunas_validas),
        'total_gaps': total_gaps if blocos_continuos else 0
    },
    'blocos': [
        {
            'bloco': i,
            'inicio': min(bloco),
            'fim': max(bloco),
            'colunas_validas': len(bloco),
            'range_completo': max(bloco) - min(bloco) + 1,
            'gaps': (max(bloco) - min(bloco) + 1) - len(bloco),
            'principal': i == 1
        }
        for i, bloco in enumerate(blocos_continuos, 1)
    ],
    'detalhes_colunas': [
        {
            'col': c['excel_col'],
            'nome': c['nome'],
            'valida': c['valida'],
            'score': c['score'],
            'tipo': c['tipo_detectado'],
            'metodo': c['metodo'],
            'match': c.get('match_dicionario')
        }
        for c in colunas_analise
    ]
}

with open(
    fm.pastas['logs'] / '.analise_colunas.json',
    'w',
    encoding='utf-8'
) as f:
    json.dump(relatorio_colunas, f, indent=2, ensure_ascii=False)

print(f"\n💾 Relatório salvo: .analise_colunas.json")


# ═══════════════════════════════════════════════════════════════
# NOVA FUNÇÃO: ANTI-DADOS PARA DETECÇÃO MULTI-LINHA
# Inserir ANTES da seção "DETECTAR MULTI-LINHA"
# ═══════════════════════════════════════════════════════════════

def linha_parece_dados(linha, dicionario_persistente):
    """
    Verifica se uma linha PARECE ser dados ao invés de cabeçalho.

    Returns:
        float: Score de "certeza que é dados" (0.0 a 1.0)
               > 0.6 = provavelmente DADOS
               < 0.4 = provavelmente CABEÇALHO
    """
    celulas = [
        str(c).strip() for c in linha
        if str(c).strip() and str(c).strip().lower() not in ['nan', 'none', '']
    ]

    if not celulas:
        return 0.0

    score_dados = 0.0

    # CRITÉRIO 1: Presença de IDs/códigos numéricos puros (peso alto)
    codigos_numericos = 0
    for celula in celulas:
        # Remove pontos e hífens para detectar códigos como "1.000.000" ou "10-234"
        apenas_digitos = re.sub(r'[.\-_/]', '', celula)

        # Se tem 5+ dígitos consecutivos, é muito provável que seja código/ID
        if len(apenas_digitos) >= 5 and apenas_digitos.isdigit():
            codigos_numericos += 1

    prop_codigos = codigos_numericos / len(celulas)
    if prop_codigos > 0.3:
        score_dados += 0.5  # Forte indicador de dados

    # CRITÉRIO 2: Match com VALORES conhecidos do dicionário (não nomes de campos)
    if dicionario_persistente and 'arquivos' in dicionario_persistente:
        valores_conhecidos = set()

        for arq_info in dicionario_persistente.get('arquivos', {}).values():
            if 'campos_mapeados' in arq_info:
                for campo_info in arq_info['campos_mapeados'].values():
                    # Pega EXEMPLOS de valores (dados), não nomes de campos
                    if 'exemplos' in campo_info:
                        for exemplo in campo_info['exemplos']:
                            valores_conhecidos.add(str(exemplo).lower().strip())

        matches_valores = 0
        for celula in celulas:
            celula_lower = celula.lower()
            if celula_lower in valores_conhecidos:
                matches_valores += 1
            # Similaridade parcial para valores longos
            elif len(celula) > 10:
                for valor_conhecido in valores_conhecidos:
                    if len(valor_conhecido) > 10:
                        sim = SequenceMatcher(None, celula_lower, valor_conhecido).ratio()
                        if sim > 0.85:
                            matches_valores += 1
                            break

        prop_match_valores = matches_valores / len(celulas)
        if prop_match_valores > 0.5:
            score_dados += 0.4  # Forte indicador de dados

    # CRITÉRIO 3: Padrão de nomenclatura de cabeçalho (AUSENTE = é dados)
    palavras_cabecalho = [
        'codigo', 'nome', 'descricao', 'data', 'valor', 'quantidade',
        'tipo', 'categoria', 'grupo', 'centro', 'material', 'produto',
        'hierarq', 'sigla', 'unidade', 'periodo', 'mes', 'ano'
    ]

    tem_palavra_cabecalho = any(
        any(palavra in celula.lower() for palavra in palavras_cabecalho)
        for celula in celulas
    )

    if not tem_palavra_cabecalho:
        score_dados += 0.2  # Moderado indicador de dados

    # CRITÉRIO 4: Tamanho muito curto (flags) ou muito longo (descrições)
    tamanho_medio = sum(len(c) for c in celulas) / len(celulas)
    if tamanho_medio < 3:
        # Muito curto = flags = dados
        score_dados += 0.1
    elif tamanho_medio > 40:
        # Muito longo = descrições detalhadas = dados
        score_dados += 0.15

    # CRITÉRIO 5: Padrões específicos de dados
    padroes_dados = [
        r'^\d{6,}$',                    # Códigos numéricos longos
        r'^[A-Z]{2,}_[A-Z_]+$',         # Padrões tipo DIESEL_MARÍTIMO_SIMP
        r'^\d{4}-\d{2}-\d{2}$',         # Datas ISO
        r'^\d+[.,]\d{2}$',              # Valores monetários
    ]

    matches_padroes = 0
    for celula in celulas:
        for padrao in padroes_dados:
            if re.match(padrao, celula):
                matches_padroes += 1
                break

    prop_padroes = matches_padroes / len(celulas)
    if prop_padroes > 0.3:
        score_dados += 0.3

    return min(1.0, score_dados)

# ═══════════════════════════════════════════════════════════════
# DETECTAR MULTI-LINHA
# ═══════════════════════════════════════════════════════════════

if len(scores) > 1:
    segundo = scores[1]

    # Verificar se segunda linha PARECE dados
    segunda_linha_dados = data_para_analise[segundo['indice']]
    prob_dados = linha_parece_dados(segunda_linha_dados, DICIONARIO_PERSISTENTE)

    print(f"\n🔍 Análise da linha seguinte (L{segundo['linha_excel']}):")
    print(f"   Score: {segundo['score']:.2f}")
    print(f"   Probabilidade de ser DADOS: {prob_dados:.0%}")

    # CRITÉRIOS MAIS RIGOROSOS para multi-linha:
    # 1. Score deve ser > 70% do melhor (não 50%)
    # 2. Probabilidade de ser dados deve ser < 40%
    # 3. Diferença de score deve ser razoável (não muito diferente)

    eh_multilinea = (
        segundo['indice'] == melhor['indice'] + 1 and
        segundo['score'] > (melhor['score'] * 0.70) and  # ← AUMENTADO de 0.5 para 0.7
        prob_dados < 0.4 and                              # ← NOVO critério anti-dados
        abs(melhor['score'] - segundo['score']) < 8.0    # ← NOVO: scores similares
    )

    if eh_multilinea:
        print(f"\n   ✅ CABEÇALHO MULTI-LINHA confirmado!")
        print(f"   Linha {melhor['linha_excel']}: Score {melhor['score']:.2f}")
        print(f"   Linha {segundo['linha_excel']}: Score {segundo['score']:.2f}")
        print(f"\n   💡 RECOMENDAÇÃO:")
        print(f"      1. CONCATENAR: Linha1 + ' - ' + Linha2")
        print(f"      2. USAR PRIMEIRA: Linha {melhor['linha_excel']}")
        print(f"      3. PERSONALIZAR via GUI")

        multi_linha_detectado = True
        linha_fim_sugerida = segundo['linha_excel']
    else:
        if prob_dados > 0.6:
            print(f"\n   ❌ Linha seguinte PARECE SER DADOS (não cabeçalho)")
            print(f"      Probabilidade: {prob_dados:.0%}")
        else:
            print(f"\n   ℹ️  Scores insuficientes para multi-linha")
            print(f"      Threshold: {melhor['score'] * 0.70:.2f}")
            print(f"      Score L{segundo['linha_excel']}: {segundo['score']:.2f}")

        multi_linha_detectado = False
        linha_fim_sugerida = melhor['linha_excel']
else:
    multi_linha_detectado = False
    linha_fim_sugerida = melhor['linha_excel']

# ═══════════════════════════════════════════════════════════════
# GUI AVANÇADA COM TIMER
# ═══════════════════════════════════════════════════════════════

def selecionar_range_cabecalho_com_timer(
    sugerido_linha,
    sugerido_linha_fim,
    total_linhas,
    total_colunas,
    multi_linha=False,
    col_inicio_sug=1,
    col_fim_sug=None
):
    """GUI avançada para seleção de range de cabeçalho."""

    if col_fim_sug is None:
        col_fim_sug = total_colunas

    config_file = fm.pastas['logs'] / '.ultimo_range_cabecalho.json'
    ultimo_config = None

    if config_file.exists():
        try:
            with open(config_file, 'r', encoding='utf-8') as f:
                cfg = json.load(f)
                if cfg.get('arquivo') == arquivo_selecionado.name:
                    ultimo_config = cfg
        except:
            pass

    root, frame, resultado, contador = GUIComTimer.criar_janela_com_timer(
        "DETECTOR - Seleção Avançada de Cabeçalho",
        650, 850,
        tem_timer=bool(ultimo_config)
    )

    tk.Label(
        frame,
        text="📋 Configuração de Cabeçalho e Dados",
        font=('Arial', 14, 'bold'),
        bg='white'
    ).pack(pady=(0, 10))

    explicacao = tk.Label(
        frame,
        text=(
            "📍 IMPORTANTE: Numeração\n\n"
            "• Índice Python (preview): inicia em 0\n"
            "• Linha Excel (arquivo): inicia em 1\n"
            "• Use LINHA EXCEL nos campos abaixo"
        ),
        font=('Arial', 9),
        bg='#E3F2FD',
        fg='#0D47A1',
        justify=tk.LEFT,
        padx=15,
        pady=10,
        relief=tk.RIDGE,
        borderwidth=2
    )
    explicacao.pack(fill=tk.X, pady=(0, 10))

    idx_sugerido = sugerido_linha - 1
    texto_sugestao = (
        f"🤖 SUGESTÃO AUTOMÁTICA\n"
        f"Cabeçalho: Índice {idx_sugerido} (Excel L{sugerido_linha})\n"
        f"Colunas: {col_inicio_sug} a {col_fim_sug} "
        f"({col_fim_sug - col_inicio_sug + 1} colunas)"
    )

    if multi_linha and sugerido_linha_fim != sugerido_linha:
        idx_fim = sugerido_linha_fim - 1
        texto_sugestao = (
            f"🤖 SUGESTÃO AUTOMÁTICA (MULTI-LINHA)\n"
            f"Cabeçalho: Índices {idx_sugerido}-{idx_fim} "
            f"(Excel L{sugerido_linha}-L{sugerido_linha_fim})\n"
            f"Colunas: {col_inicio_sug} a {col_fim_sug} "
            f"({col_fim_sug - col_inicio_sug + 1} colunas)"
        )

    tk.Label(
        frame,
        text=texto_sugestao,
        font=('Arial', 10, 'bold'),
        bg='#E8F5E9' if not multi_linha else '#FFF9C4',
        fg='#2E7D32' if not multi_linha else '#F57F17',
        padx=15,
        pady=10,
        justify=tk.CENTER
    ).pack(fill=tk.X, pady=(0, 10))

    if col_inicio_sug > 1:
        colunas_ignoradas = col_inicio_sug - 1
        tk.Label(
            frame,
            text=(
                f"⚠️ COLUNAS 1-{colunas_ignoradas} DETECTADAS COMO "
                f"FLAGS/FÓRMULAS\n"
                f"(Serão ignoradas automaticamente)"
            ),
            font=('Arial', 9),
            bg='#FFEBEE',
            fg='#C62828',
            padx=15,
            pady=8,
            justify=tk.CENTER
        ).pack(fill=tk.X, pady=(0, 10))

    if ultimo_config:
        linha_ini = ultimo_config['linha_inicio']
        linha_fim = ultimo_config.get('linha_fim', linha_ini)
        col_ini = ultimo_config['col_inicio']
        col_fim = ultimo_config['col_fim']

        idx_ini = linha_ini - 1
        idx_fim = linha_fim - 1

        texto_ultimo = (
            f"💡 ÚLTIMA CONFIGURAÇÃO USADA\n"
            f"Cabeçalho: Índice {idx_ini}"
        )
        if idx_fim != idx_ini:
            texto_ultimo += f"-{idx_fim}"
        texto_ultimo += f" (Excel L{linha_ini}"
        if linha_fim != linha_ini:
            texto_ultimo += f"-L{linha_fim}"
        texto_ultimo += f") | Colunas {col_ini}-{col_fim}"

        tk.Label(
            frame,
            text=texto_ultimo,
            font=('Arial', 9),
            bg='#FFF3E0',
            fg='#E65100',
            padx=15,
            pady=8,
            justify=tk.CENTER
        ).pack(fill=tk.X, pady=(0, 10))

        countdown = GUIComTimer.adicionar_timer(
            frame, root, resultado, contador
        )

    frame_inputs = tk.Frame(frame, bg='white')
    frame_inputs.pack(fill=tk.X, pady=(10, 0))

    tk.Label(
        frame_inputs,
        text="Linha Cabeçalho INÍCIO (Excel):",
        bg='white',
        font=('Arial', 9)
    ).grid(row=0, column=0, sticky='w', padx=5, pady=5)

    entry_cab_ini = tk.Entry(frame_inputs, width=10, font=('Arial', 10))
    entry_cab_ini.insert(
        0,
        str(ultimo_config['linha_inicio'] if ultimo_config
            else sugerido_linha)
    )
    entry_cab_ini.grid(row=0, column=1, padx=5, pady=5)

    tk.Label(
        frame_inputs,
        text="Linha Cabeçalho FIM (Excel):",
        bg='white',
        font=('Arial', 9)
    ).grid(row=1, column=0, sticky='w', padx=5, pady=5)

    entry_cab_fim = tk.Entry(frame_inputs, width=10, font=('Arial', 10))
    entry_cab_fim.insert(
        0,
        str(ultimo_config.get('linha_fim', sugerido_linha_fim)
            if ultimo_config else sugerido_linha_fim)
    )
    entry_cab_fim.grid(row=1, column=1, padx=5, pady=5)

    tk.Label(
        frame_inputs,
        text="Coluna INÍCIO:",
        bg='white',
        font=('Arial', 9)
    ).grid(row=2, column=0, sticky='w', padx=5, pady=5)

    entry_col_ini = tk.Entry(frame_inputs, width=10, font=('Arial', 10))
    entry_col_ini.insert(
        0,
        str(ultimo_config['col_inicio'] if ultimo_config
            else col_inicio_sug)
    )
    entry_col_ini.grid(row=2, column=1, padx=5, pady=5)

    tk.Label(
        frame_inputs,
        text="Coluna FIM:",
        bg='white',
        font=('Arial', 9)
    ).grid(row=3, column=0, sticky='w', padx=5, pady=5)

    entry_col_fim = tk.Entry(frame_inputs, width=10, font=('Arial', 10))
    entry_col_fim.insert(
        0,
        str(ultimo_config['col_fim'] if ultimo_config
            else col_fim_sug)
    )
    entry_col_fim.grid(row=3, column=1, padx=5, pady=5)

    tk.Label(
        frame_inputs,
        text="Linha DADOS início (Excel):",
        bg='white',
        font=('Arial', 9)
    ).grid(row=4, column=0, sticky='w', padx=5, pady=5)

    entry_dados_ini = tk.Entry(
        frame_inputs, width=10, font=('Arial', 10)
    )

    dados_inicio_default = (
        ultimo_config.get('linha_fim', sugerido_linha_fim)
        if ultimo_config else sugerido_linha_fim
    ) + 1

    entry_dados_ini.insert(
        0,
        str(ultimo_config.get('linha_dados_inicio', dados_inicio_default)
            if ultimo_config else dados_inicio_default)
    )
    entry_dados_ini.grid(row=4, column=1, padx=5, pady=5)

    tk.Label(
        frame,
        text=(
            "💡 Para cabeçalho 1 linha: Início = Fim\n"
            "💡 Para multi-linha: Início < Fim (ex: 3 a 4)"
        ),
        font=('Arial', 8, 'italic'),
        bg='#FFFDE7',
        fg='#F57F17',
        padx=10,
        pady=8,
        justify=tk.LEFT
    ).pack(fill=tk.X, pady=(10, 0))

    def confirmar():
        resultado['cancelado'] = True
        try:
            resultado['valor'] = {
                'linha_inicio': int(entry_cab_ini.get()),
                'linha_fim': int(entry_cab_fim.get()),
                'col_inicio': int(entry_col_ini.get()),
                'col_fim': int(entry_col_fim.get()),
                'linha_dados_inicio': int(entry_dados_ini.get())
            }
        except ValueError:
            resultado['valor'] = None
        root.quit()
        root.destroy()

    def usar_ultima():
        resultado['cancelado'] = True
        resultado['valor'] = {
            'linha_inicio': ultimo_config['linha_inicio'],
            'linha_fim': ultimo_config.get(
                'linha_fim', ultimo_config['linha_inicio']
            ),
            'col_inicio': ultimo_config['col_inicio'],
            'col_fim': ultimo_config['col_fim'],
            'linha_dados_inicio': ultimo_config['linha_dados_inicio']
        }
        root.quit()
        root.destroy()

    GUIComTimer.criar_botoes(
        frame,
        confirmar,
        usar_ultima if ultimo_config else None,
        "✅ Confirmar",
        "⏱️ Usar Última (10s)"
    )

    if ultimo_config:
        root.after(1000, countdown)

    root.mainloop()

    if resultado.get('timeout') and ultimo_config:
        print(f"   ⏱️  Timeout - usando última configuração")
        return {
            'linha_inicio': ultimo_config['linha_inicio'],
            'linha_fim': ultimo_config.get(
                'linha_fim', ultimo_config['linha_inicio']
            ),
            'col_inicio': ultimo_config['col_inicio'],
            'col_fim': ultimo_config['col_fim'],
            'linha_dados_inicio': ultimo_config['linha_dados_inicio']
        }

    return resultado['valor']

# EXECUTAR GUI
print("\nAbrindo janela de configuração...")
config = selecionar_range_cabecalho_com_timer(
    melhor['linha_excel'],
    linha_fim_sugerida,
    len(data_para_analise),
    len(data_para_analise[0]) if data_para_analise else 1,
    multi_linha=multi_linha_detectado,
    col_inicio_sug=col_inicio_sugerido,
    col_fim_sug=col_fim_sugerido
)

if not config:
    raise ValueError("❌ Configuração inválida ou cancelada")

# Salvar configuração
with open(
    fm.pastas['logs'] / '.ultimo_range_cabecalho.json',
    'w',
    encoding='utf-8'
) as f:
    config_salvar = config.copy()
    config_salvar['arquivo'] = arquivo_selecionado.name
    config_salvar['sheet'] = sheet_nome
    config_salvar['timestamp'] = datetime.now().isoformat()
    json.dump(config_salvar, f, indent=2)

# Extrair informações
linha_cabecalho_inicio = config['linha_inicio']
linha_cabecalho_fim = config['linha_fim']
col_inicio = config['col_inicio']
col_fim = config['col_fim']
linha_dados_inicio = config['linha_dados_inicio']

print(f"\n✅ CONFIGURAÇÃO CONFIRMADA:")
print(f"{'='*70}")
print(f"   📋 Cabeçalho (Excel): L{linha_cabecalho_inicio}", end="")
if linha_cabecalho_fim != linha_cabecalho_inicio:
    print(f" a L{linha_cabecalho_fim}")
else:
    print()

print(f"   📊 Colunas (Excel): {col_inicio} a {col_fim}")
print(f"   📈 Dados começam (Excel): L{linha_dados_inicio}")

# Converter para índices Python
idx_cab_inicio = linha_cabecalho_inicio - 1
idx_cab_fim = linha_cabecalho_fim - 1
idx_col_inicio = col_inicio - 1
idx_col_fim = col_fim
idx_dados_inicio = linha_dados_inicio - 1

print(f"\n   🐍 Índices Python (uso interno):")
print(f"      Cabeçalho: idx {idx_cab_inicio}", end="")
if idx_cab_fim != idx_cab_inicio:
    print(f" a {idx_cab_fim}")
else:
    print()
print(f"      Colunas: idx {idx_col_inicio} a {idx_col_fim}")
print(f"      Dados: a partir de idx {idx_dados_inicio}")
print("=" * 70)

# SALVAR ESTADO DO BLOCO 8
estado_bloco8 = {
    'bloco': 8,
    'status': 'concluido',
    'timestamp': datetime.now().isoformat(),
    'arquivo': arquivo_selecionado.name,
    'sheet': sheet_nome,
    'config': config,
    'indices_python': {
        'cabecalho_inicio': idx_cab_inicio,
        'cabecalho_fim': idx_cab_fim,
        'col_inicio': idx_col_inicio,
        'col_fim': idx_col_fim,
        'dados_inicio': idx_dados_inicio
    },
    'deteccao': {
        'metodo': 'scoring_avancado_v2.1',
        'melhor_score': melhor['score'],
        'multi_linha': multi_linha_detectado,
        'total_candidatos': len(scores),
        'correcoes': ['bug_string_vazia', 'bug_diversidade', 'tolerancia_gaps']
    }
}

with open(
    fm.pastas['logs'] / '.bloco_8_state.json',
    'w',
    encoding='utf-8'
) as f:
    json.dump(estado_bloco8, f, indent=2, ensure_ascii=False)

print("\n" + "="*70)
print("✅ DETECÇÃO DE CABEÇALHO CONCLUÍDA")
print("="*70)


🎯 DETECÇÃO E SELEÇÃO DE CABEÇALHO

🔍 Verificando cabeçalho multi-linha...

📚 Dicionário persistente já carregado

📊 Analisando linhas para detectar cabeçalho...

🏆 Top 5 candidatos a cabeçalho:
📍 NUMERAÇÃO: Usamos índice Python (preview inicia em 0)
   • Índice 0 = Linha 1 no Excel

   1º. Índice 33 (Excel: Linha 34)
       Score: 14.10/24.5
       Preench: 72% (+1.4) | Texto: 100% (+2.5) | Unic: 100% (+3.0) | Tam: 23 (+1.0) | Pos: 34 (+0.17) | DadosAbaixo:Num:81% (+2.0) | Rotulos:56% (+4.0)

   2º. Índice 6 (Excel: Linha 7)
       Score: 12.47/24.5
       Preench: 2% (+0.0) | Texto: 100% (+2.5) | Unic: 100% (+3.0) | Tam: 36 (+1.0) | Únicos (+1.5) | Pos: 7 (+0.44) | Rotulos:100% (+4.0)

   3º. Índice 8 (Excel: Linha 9)
       Score: 12.45/24.5
       Preench: 2% (+0.0) | Texto: 100% (+2.5) | Unic: 100% (+3.0) | Tam: 15 (+1.0) | Únicos (+1.5) | Pos: 9 (+0.42) | Rotulos:100% (+4.0)

   4º. Índice 9 (Excel: Linha 10)
       Score: 12.44/24.5
       Preench: 2% (+0.0) | Texto: 100% (+2.5)

In [40]:
# ═══════════════════════════════════════════════════════════════════
# BLOCO 9 - EXTRAÇÃO DE DADOS v2.0 (CORRIGIDO - PADRÃO JSON)
# ═══════════════════════════════════════════════════════════════════
# CORREÇÕES v2.0:
# ✅ REMOVIDO: .parquet (biblioteca não instalada)
# ✅ ADICIONADO: .json para persistência (padrão do projeto)
# ✅ MANTIDO: Todas as validações e logs
# ✅ COMPATÍVEL: Com BLOCO 10 v2.3 (que espera JSON também)
# ═══════════════════════════════════════════════════════════════════
# COMUNICAÇÃO VIA LOG:
# - LÊ: .bloco_5_state.json, .bloco_8_state.json, LOG GLOBAL
# - SALVA: df_bruto em JSON + .bloco_9_state.json
# - PRÓXIMO BLOCO: Carrega df_bruto do JSON
# ═══════════════════════════════════════════════════════════════════

print("\n" + "="*70)
print("📥 EXTRAÇÃO DE DADOS v2.0")
print("="*70)

from pathlib import Path
import json
from datetime import datetime
import pandas as pd

# ═══════════════════════════════════════════════════════════════════
# 1. CONECTAR VIA LOG GLOBAL
# ═══════════════════════════════════════════════════════════════════

log_global = Path.home() / '.processador_dicionario_localizador.json'

if not log_global.exists():
    raise FileNotFoundError("❌ LOG GLOBAL não encontrado! Execute BLOCO 1.")

with open(log_global, 'r', encoding='utf-8') as f:
    config = json.load(f)

pasta_base = Path(config['pasta_base_atual'])
timestamp_execucao = config['timestamp']

print(f"\n✅ CONFIGURAÇÃO CARREGADA DO LOG GLOBAL")
print(f"   📁 Pasta base: {pasta_base.name}")
print(f"   🕐 Timestamp: {timestamp_execucao}")

# Recriar FileManager
from sys import path as sys_path
# Assumir que FileManagerInterativo foi definido no BLOCO 2
# Se não estiver na memória, o usuário deve executar BLOCO 2 primeiro
if 'FileManagerInterativo' not in globals():
    raise NameError("❌ FileManagerInterativo não encontrado! Execute BLOCO 2.")

fm = FileManagerInterativo(pasta_base)

# ═══════════════════════════════════════════════════════════════════
# 2. CARREGAR CONFIGURAÇÕES DOS BLOCOS 4, 5, 6, 8
# ═══════════════════════════════════════════════════════════════════

# BLOCO 8: Range de cabeçalho e colunas
arquivo_bloco8 = fm.pastas['logs'] / '.bloco_8_state.json'
if not arquivo_bloco8.exists():
    raise FileNotFoundError("❌ BLOCO 8 não executado!")

with open(arquivo_bloco8, 'r', encoding='utf-8') as f:
    estado_bloco8 = json.load(f)

config_bloco8 = estado_bloco8['config']

linha_cabecalho_inicio = config_bloco8['linha_inicio']
linha_cabecalho_fim = config_bloco8['linha_fim']
col_inicio = config_bloco8['col_inicio']
col_fim = config_bloco8['col_fim']
linha_dados_inicio = config_bloco8['linha_dados_inicio']

# BLOCO 5: Método de carga e arquivo
arquivo_bloco5_state = fm.pastas['logs'] / '.bloco_5_state.json'
if not arquivo_bloco5_state.exists():
    raise FileNotFoundError("❌ BLOCO 5 não executado!")

with open(arquivo_bloco5_state, 'r', encoding='utf-8') as f:
    estado_bloco5 = json.load(f)

metodo_carga = estado_bloco5['metodo_carga']
arquivo_selecionado = Path(estado_bloco5['workbook_path'])

# CSV específico
if metodo_carga == 'csv':
    separador_detectado = estado_bloco5.get('separador_detectado', ',')
    skiprows_csv = estado_bloco5.get('skiprows_csv', 0)

# Sheet selecionada: buscar do BLOCO 8
sheet_nome = estado_bloco8['sheet']

print(f"\n✅ CONFIGURAÇÕES CARREGADAS")
print(f"   📄 Arquivo: {arquivo_selecionado.name}")
print(f"   📋 Sheet: {sheet_nome}")
print(f"   🔧 Método: {metodo_carga}")

# ═══════════════════════════════════════════════════════════════════
# 3. CONVERTER ÍNDICES EXCEL → PYTHON
# ═══════════════════════════════════════════════════════════════════

idx_cab_inicio = linha_cabecalho_inicio - 1
idx_cab_fim = linha_cabecalho_fim - 1
idx_col_inicio = col_inicio - 1
idx_col_fim = col_fim
idx_dados_inicio = linha_dados_inicio - 1

print("\n📋 Configuração (notação Excel):")
print(f"   Cabeçalho: L{linha_cabecalho_inicio}", end="")
if linha_cabecalho_fim != linha_cabecalho_inicio:
    print(f" a L{linha_cabecalho_fim}")
else:
    print()
print(f"   Colunas: {col_inicio} a {col_fim}")
print(f"   Dados: A partir de L{linha_dados_inicio}")

print(f"\n🐍 Índices Python:")
print(f"   Cabeçalho: {idx_cab_inicio} a {idx_cab_fim}")
print(f"   Colunas: {idx_col_inicio} a {idx_col_fim}")
print(f"   Dados: a partir de {idx_dados_inicio}")

# ═══════════════════════════════════════════════════════════════════
# 4. EXTRAÇÃO: CSV
# ═══════════════════════════════════════════════════════════════════

if metodo_carga == 'csv':
    print(f"\n📄 Método: CSV")

    try:
        if linha_cabecalho_inicio == linha_cabecalho_fim:
            print(f"   📋 Cabeçalho único (L{linha_cabecalho_inicio})")

            df = pd.read_csv(
                arquivo_selecionado,
                sep=separador_detectado,
                encoding='cp1252',
                skiprows=skiprows_csv,
                header=idx_cab_inicio,
                usecols=range(idx_col_inicio, idx_col_fim)
            )

            linhas_pular = idx_dados_inicio - idx_cab_inicio - 1
            if linhas_pular > 0:
                print(f"   ⏭️  Pulando {linhas_pular} linha(s)")
                df = df.iloc[linhas_pular:].copy()

        else:
            print(f"   📋 Cabeçalho multi-linha (L{linha_cabecalho_inicio} a L{linha_cabecalho_fim})")

            df_temp = pd.read_csv(
                arquivo_selecionado,
                sep=separador_detectado,
                encoding='cp1252',
                skiprows=skiprows_csv,
                header=None,
                usecols=range(idx_col_inicio, idx_col_fim)
            )

            cab_linhas = df_temp.iloc[idx_cab_inicio:idx_cab_fim+1].values
            cab_final = []

            for col_idx in range(cab_linhas.shape[1]):
                partes = [
                    str(linha[col_idx]).strip()
                    for linha in cab_linhas
                    if str(linha[col_idx]).strip() not in ['', 'nan', 'None']
                ]
                cab_final.append(' - '.join(partes) if partes else f'Col_{col_idx}')

            df = df_temp.iloc[idx_dados_inicio:].copy()
            df.columns = cab_final

        print(f"   ✅ Carregado: {len(df):,} registros × {len(df.columns)} colunas")

    except Exception as e:
        print(f"   ❌ ERRO: {str(e)}")
        raise

# ═══════════════════════════════════════════════════════════════════
# 5. EXTRAÇÃO: EXCEL MODERNO (PANDAS)
# ═══════════════════════════════════════════════════════════════════

elif metodo_carga == 'pandas':
    print(f"📋 Método: pandas (XLSX/XLSM)")

    try:
        if linha_cabecalho_inicio == linha_cabecalho_fim:
            print(f"   📋 Cabeçalho único (L{linha_cabecalho_inicio})")

            df = pd.read_excel(
                arquivo_selecionado,
                sheet_name=sheet_nome,
                header=idx_cab_inicio,
                usecols=range(idx_col_inicio, idx_col_fim)
            )

            linhas_pular = idx_dados_inicio - idx_cab_inicio - 1
            if linhas_pular > 0:
                print(f"   ⏭️  Pulando {linhas_pular} linha(s)")
                df = df.iloc[linhas_pular:].copy()

        else:
            print(f"   📋 Cabeçalho multi-linha (L{linha_cabecalho_inicio} a L{linha_cabecalho_fim})")

            df_temp = pd.read_excel(
                arquivo_selecionado,
                sheet_name=sheet_nome,
                header=None,
                usecols=range(idx_col_inicio, idx_col_fim)
            )

            cab_linhas = df_temp.iloc[idx_cab_inicio:idx_cab_fim+1].values
            cab_final = []

            for col_idx in range(cab_linhas.shape[1]):
                partes = [
                    str(linha[col_idx]).strip()
                    for linha in cab_linhas
                    if str(linha[col_idx]).strip() not in ['', 'nan', 'None']
                ]
                cab_final.append(' - '.join(partes) if partes else f'Col_{col_idx}')

            df = df_temp.iloc[idx_dados_inicio:].copy()
            df.columns = cab_final

        print(f"   ✅ Carregado: {len(df):,} registros × {len(df.columns)} colunas")

    except Exception as e:
        print(f"   ❌ ERRO: {str(e)}")
        raise

# ═══════════════════════════════════════════════════════════════════
# 6. EXTRAÇÃO: EXCEL LEGADO (XLRD)
# ═══════════════════════════════════════════════════════════════════

elif metodo_carga == 'xlrd':
    print(f"📋 Método: xlrd (XLS)")

    try:
        import xlrd
        workbook = xlrd.open_workbook(arquivo_selecionado)
        sheet = workbook.sheet_by_name(sheet_nome)

        if linha_cabecalho_inicio == linha_cabecalho_fim:
            print(f"   📋 Cabeçalho único (L{linha_cabecalho_inicio})")

            cabecalho = sheet.row_values(idx_cab_inicio)[idx_col_inicio:idx_col_fim]

            data = []
            for row_idx in range(idx_dados_inicio, sheet.nrows):
                data.append(sheet.row_values(row_idx)[idx_col_inicio:idx_col_fim])

            df = pd.DataFrame(data, columns=cabecalho)

        else:
            print(f"   📋 Cabeçalho multi-linha (L{linha_cabecalho_inicio} a L{linha_cabecalho_fim})")

            cab_linhas = []
            for linha_idx in range(idx_cab_inicio, idx_cab_fim + 1):
                cab_linhas.append(sheet.row_values(linha_idx)[idx_col_inicio:idx_col_fim])

            cab_final = []
            for col_idx in range(len(cab_linhas[0])):
                partes = [
                    str(linha[col_idx]).strip()
                    for linha in cab_linhas
                    if str(linha[col_idx]).strip() not in ['', 'nan', 'None']
                ]
                cab_final.append(' - '.join(partes) if partes else f'Col_{col_idx}')

            data = []
            for row_idx in range(idx_dados_inicio, sheet.nrows):
                data.append(sheet.row_values(row_idx)[idx_col_inicio:idx_col_fim])

            df = pd.DataFrame(data, columns=cab_final)

        print(f"   ✅ Carregado: {len(df):,} registros × {len(df.columns)} colunas")

    except Exception as e:
        print(f"   ❌ ERRO: {str(e)}")
        raise

else:
    raise ValueError(f"❌ Método desconhecido: {metodo_carga}")

# ═══════════════════════════════════════════════════════════════════
# 7. VALIDAÇÕES PÓS-EXTRAÇÃO
# ═══════════════════════════════════════════════════════════════════

print("\n" + "─"*70)
print("✅ VALIDAÇÕES")
print("─"*70)

df = df.reset_index(drop=True)

print(f"\n📊 Shape final:")
print(f"   Registros: {len(df):,}")
print(f"   Colunas: {len(df.columns)}")

print(f"\n📋 Primeiras 10 colunas:")
for i, col in enumerate(df.columns[:10], 1):
    print(f"   {i:2d}. {col}")
if len(df.columns) > 10:
    print(f"   ... e mais {len(df.columns) - 10} colunas")

print(f"\n📈 Primeiras 5 linhas (amostra):")
print(df.head().to_string())

print(f"\n🔢 Tipos detectados:")
tipos_count = df.dtypes.value_counts()
for tipo, count in tipos_count.items():
    print(f"   {str(tipo).ljust(15)}: {count} coluna(s)")

print(f"\n⚠️  Valores nulos:")
nulos_total = df.isnull().sum().sum()
if nulos_total > 0:
    print(f"   Total: {nulos_total:,} células vazias")
    colunas_com_nulos = df.isnull().sum()
    colunas_com_nulos = colunas_com_nulos[colunas_com_nulos > 0].sort_values(ascending=False)
    print(f"\n   Top 5 colunas com nulos:")
    for col, count in colunas_com_nulos.head(5).items():
        pct = (count / len(df)) * 100
        print(f"      {col[:40].ljust(40)}: {count:>6,} ({pct:>5.1f}%)")
else:
    print(f"   ✅ Nenhum valor nulo!")

memoria_mb = df.memory_usage(deep=True).sum() / 1024**2
print(f"\n💾 Memória utilizada: {memoria_mb:.2f} MB")

# ═══════════════════════════════════════════════════════════════════
# 8. SALVAMENTO EM JSON (PADRÃO DO PROJETO)
# ═══════════════════════════════════════════════════════════════════

print(f"\n💾 Salvando df_bruto em JSON...")

df_bruto = df.copy()

# Converter DataFrame para JSON estruturado
df_dict = {
    'columns': list(df_bruto.columns),
    'data': df_bruto.values.tolist(),
    'index': df_bruto.index.tolist(),
    'dtypes': {col: str(dtype) for col, dtype in df_bruto.dtypes.items()}
}

arquivo_df_json = fm.pastas['processados'] / f'df_bruto_{timestamp_execucao}.json'

with open(arquivo_df_json, 'w', encoding='utf-8') as f:
    json.dump(df_dict, f, ensure_ascii=False, indent=2)

tamanho_kb = arquivo_df_json.stat().st_size / 1024
print(f"   ✅ Salvo: {arquivo_df_json.name} ({tamanho_kb:.1f} KB)")

# ═══════════════════════════════════════════════════════════════════
# 9. SALVAR ESTADO DO BLOCO 9
# ═══════════════════════════════════════════════════════════════════

estado_bloco9 = {
    'timestamp': datetime.now().isoformat(),
    'bloco': 9,
    'nome': 'EXTRAÇÃO DE DADOS',
    'versao': '2.0',
    'status': 'concluido',
    'variaveis_criadas': ['df', 'df_bruto'],
    'arquivo_processado': arquivo_selecionado.name,
    'arquivo_df_bruto_json': arquivo_df_json.name,
    'caminho_completo_json': str(arquivo_df_json),
    'metodo_carga': metodo_carga,
    'estatisticas': {
        'total_registros': len(df),
        'total_colunas': len(df.columns),
        'memoria_mb': round(memoria_mb, 2),
        'colunas_com_nulos': int(nulos_total),
        'tipos_detectados': {str(k): int(v) for k, v in df.dtypes.value_counts().items()}
    },
    'configuracao_aplicada': {
        'linha_cabecalho_inicio': linha_cabecalho_inicio,
        'linha_cabecalho_fim': linha_cabecalho_fim,
        'col_inicio': col_inicio,
        'col_fim': col_fim,
        'linha_dados_inicio': linha_dados_inicio,
        'cabecalho_multilinha': linha_cabecalho_inicio != linha_cabecalho_fim
    },
    'colunas_extraidas': list(df.columns),
    'transformacoes_aplicadas': [
        {
            'tipo': 'extracao_range',
            'descricao': f'Cabeçalho L{linha_cabecalho_inicio}-L{linha_cabecalho_fim}, Colunas {col_inicio}-{col_fim}, Dados L{linha_dados_inicio}+',
            'timestamp': datetime.now().isoformat()
        }
    ],
    'instrucoes_proximo_bloco': {
        'como_carregar': 'pd.read_json() ou json.load() + pd.DataFrame()',
        'arquivo': arquivo_df_json.name,
        'caminho': str(arquivo_df_json)
    }
}

arquivo_estado = fm.pastas['logs'] / '.bloco_9_state.json'
with open(arquivo_estado, 'w', encoding='utf-8') as f:
    json.dump(estado_bloco9, f, indent=2, ensure_ascii=False)

print(f"\n💾 Estado salvo:")
print(f"   {arquivo_estado.name}")

print("\n" + "="*70)
print("✅ EXTRAÇÃO CONCLUÍDA COM SUCESSO v2.0")
print("="*70)
print(f"\n📝 INSTRUÇÕES PARA O PRÓXIMO BLOCO:")
print(f"   1. Carregar estado: json.load('.bloco_9_state.json')")
print(f"   2. Ler caminho: estado['caminho_completo_json']")
print(f"   3. Carregar df_bruto:")
print(f"      with open(caminho, 'r') as f:")
print(f"          df_dict = json.load(f)")
print(f"      df_bruto = pd.DataFrame(df_dict['data'], columns=df_dict['columns'])")
print("="*70)


📥 EXTRAÇÃO DE DADOS v2.0

✅ CONFIGURAÇÃO CARREGADA DO LOG GLOBAL
   📁 Pasta base: PROCESSAR_ARQUIVOS_20251019_060722
   🕐 Timestamp: 20251019_060722

✅ CONFIGURAÇÕES CARREGADAS
   📄 Arquivo: Cópia de xSAPtemp4687_JAN_25.xls
   📋 Sheet: Valor da Variação Total
   🔧 Método: xlrd

📋 Configuração (notação Excel):
   Cabeçalho: L34
   Colunas: 13 a 37
   Dados: A partir de L35

🐍 Índices Python:
   Cabeçalho: 33 a 33
   Colunas: 12 a 37
   Dados: a partir de 34
📋 Método: xlrd (XLS)
   📋 Cabeçalho único (L34)
   ✅ Carregado: 967 registros × 25 colunas

──────────────────────────────────────────────────────────────────────
✅ VALIDAÇÕES
──────────────────────────────────────────────────────────────────────

📊 Shape final:
   Registros: 967
   Colunas: 25

📋 Primeiras 10 colunas:
    1. Ano civil/mês
    2. Centro
    3. 
    4. HierarqPrd
    5. Produto
    6. 
    7. Estoque
Inicial
    8. Entrada
    9. Variação
Externa
   10. Variação
Externa
%
   ... e mais 15 colunas

📈 Primeiras 5 linha

In [41]:
# ===================================================================
# BLOCO 10 - LIMPEZA DE ESTRUTURA v2.5 CONSOLIDADO
# ===================================================================
# NOVIDADES v2.5:
# + Remoção de totalizações AVANÇADA (23 padrões vs 7)
# + Detecção em PRIMEIRA e SEGUNDA coluna
# + Análise hierárquica BW (colunas categóricas vazias)
# + Validação de segurança (>50% = alerta + confirmação usuário)
# + Registro detalhado de TODAS as linhas detectadas
# ===================================================================
# MANTIDO v2.4:
# - Salvamento em Parquet (pyarrow desnecessario!)
# - Carregamento de df_bruto do disco
# - Assume df_bruto na memoria (BLOCO 9 executado)
# + Todas as transformacoes de limpeza
# + Registro completo em JSON
# + Estado do bloco em JSON
# ===================================================================

print("\n" + "="*70)
print("LIMPEZA DE ESTRUTURA + REMOÇÃO TOTALIZAÇÕES AVANÇADA")
print("="*70)

# ===================================================================
# 1. CONECTAR COM BLOCOS ANTERIORES (0% memoria, 100% LOG)
# ===================================================================

from pathlib import Path
import json
import pandas as pd
from datetime import datetime
import re
from collections import Counter

# Carregar LOG GLOBAL (CAMINHO CORRETO!)
try:
    log_global_path = Path.home() / '.processador_dicionario_localizador.json'

    if not log_global_path.exists():
        raise FileNotFoundError(
            "❌ LOG GLOBAL não encontrado!\n"
            f"   Esperado: {log_global_path}\n"
            "   Execute o BLOCO 1 primeiro!"
        )

    with open(log_global_path, 'r', encoding='utf-8') as f:
        log_global = json.load(f)

    pasta_base = Path(log_global['pasta_base_atual'])
    timestamp_execucao = log_global['timestamp']

    print(f"✅ LOG GLOBAL conectado!")
    print(f"   📂 Container: {pasta_base.name}")
    print(f"   🕐 Timestamp: {timestamp_execucao}")

except Exception as e:
    print(f"❌ ERRO: Não foi possível conectar ao LOG GLOBAL")
    print(f"   Detalhe: {e}")
    raise

# Recriar FileManager
class FileManagerInterativo:
    """Gerenciador de arquivos"""
    def __init__(self, base_path):
        self.base_path = Path(base_path)
        self.pastas = {
            'entrada': self.base_path / '01_Entrada',
            'processados': self.base_path / '02_Processados',
            'outputs': self.base_path / '03_Outputs',
            'logs': self.base_path / '04_Logs',
            'dicionarios': self.base_path / '05_Dicionarios',
            'codigos': self.base_path / '06_Codigos_Integracao'
        }

fm = FileManagerInterativo(pasta_base)
print(f"✅ FileManager recriado: {fm.base_path.name}")

# Validar que BLOCO 9 foi executado
arquivo_bloco9 = fm.pastas['logs'] / '.bloco_9_state.json'
if not arquivo_bloco9.exists():
    print(f"\n❌ ERRO: BLOCO 9 não foi executado!")
    print(f"   Execute o BLOCO 9 (Extração de Dados) primeiro!")
    raise FileNotFoundError("BLOCO 9 não foi executado")

with open(arquivo_bloco9, 'r', encoding='utf-8') as f:
    estado_bloco9 = json.load(f)

print(f"\n✅ BLOCO 9 conectado!")
print(f"   Arquivo processado: {estado_bloco9['arquivo_processado']}")
print(f"   Registros extraídos: {estado_bloco9['estatisticas']['total_registros']:,}")
print(f"   Colunas extraídas: {estado_bloco9['estatisticas']['total_colunas']}")

# ===================================================================
# 2. VALIDAR QUE df_bruto ESTA NA MEMORIA
# ===================================================================

if 'df_bruto' not in globals():
    print(f"\n❌ ERRO: df_bruto não encontrado na memória!")
    print(f"   Execute o BLOCO 9 na mesma sessão antes deste bloco.")
    print(f"   Se o kernel foi reiniciado, re-execute os BLOCOS 1-9.")
    raise NameError("df_bruto não está na memória")

print(f"\n✅ df_bruto encontrado na memória!")
df = df_bruto.copy()

# Inicializar registros de transformacao
log_limpeza = []
transformacoes_detalhadas = {
    'colunas_removidas': [],
    'linhas_removidas': [],
    'colunas_renomeadas': {},
    'linhas_totais_detectadas': [],
    'padroes_aplicados': []
}

print(f"\n🔧 Iniciando limpeza...")
print(f"   Registros iniciais: {len(df):,}")
print(f"   Colunas iniciais: {len(df.columns)}")

# ===================================================================
# 3. REMOVER COLUNAS COMPLETAMENTE VAZIAS
# ===================================================================

colunas_vazias = df.columns[df.isna().all()].tolist()
if colunas_vazias:
    print(f"\n🗑️ Removendo {len(colunas_vazias)} colunas vazias:")
    for col in colunas_vazias[:5]:
        print(f"   • {col}")
    if len(colunas_vazias) > 5:
        print(f"   ... e mais {len(colunas_vazias) - 5}")

    df = df.drop(columns=colunas_vazias)
    log_limpeza.append(f"Removidas {len(colunas_vazias)} colunas vazias")
    transformacoes_detalhadas['colunas_removidas'] = colunas_vazias
    transformacoes_detalhadas['padroes_aplicados'].append({
        'tipo': 'remocao_colunas_vazias',
        'criterio': 'df.isna().all()',
        'quantidade': len(colunas_vazias)
    })

# ===================================================================
# 4. REMOVER LINHAS COMPLETAMENTE VAZIAS
# ===================================================================

linhas_vazias_antes = len(df)
df = df.dropna(how='all')
linhas_vazias = linhas_vazias_antes - len(df)

if linhas_vazias > 0:
    print(f"\n🗑️ Removidas {linhas_vazias} linhas completamente vazias")
    log_limpeza.append(f"Removidas {linhas_vazias} linhas vazias")
    transformacoes_detalhadas['padroes_aplicados'].append({
        'tipo': 'remocao_linhas_vazias',
        'criterio': 'df.dropna(how=all)',
        'quantidade': linhas_vazias
    })

# ===================================================================
# 5. LIMPAR NOMES DE COLUNAS
# ===================================================================

print(f"\n🧹 Limpando nomes de colunas...")
colunas_antes = df.columns.tolist()
colunas_limpas = []

for col in df.columns:
    # Limpar
    col_limpo = str(col).strip()
    col_limpo = col_limpo.lstrip("'\"")
    col_limpo = col_limpo.replace('\n', ' ').replace('\r', '')
    col_limpo = ' '.join(col_limpo.split())

    # Remover espacos extras
    col_limpo = re.sub(r'\s+', ' ', col_limpo)

    colunas_limpas.append(col_limpo)

# Contar modificacoes
modificados = sum(1 for orig, limpo in zip(colunas_antes, colunas_limpas)
                  if orig != limpo)
if modificados > 0:
    print(f"   ✅ {modificados} nomes limpos")
    log_limpeza.append(f"Limpeza de nomes: {modificados} colunas")
    transformacoes_detalhadas['padroes_aplicados'].append({
        'tipo': 'limpeza_nomes',
        'criterio': 'strip + lstrip + regex',
        'quantidade': modificados
    })

df.columns = colunas_limpas

# ===================================================================
# 6. RENOMEAR COLUNAS DUPLICADAS
# ===================================================================

contagem = Counter(colunas_limpas)
duplicadas = {c: n for c, n in contagem.items() if n > 1}

if duplicadas:
    print(f"\n🔄 Renomeando {len(duplicadas)} colunas duplicadas:")
    colunas_finais = []
    contador = {}

    for col in colunas_limpas:
        if col in duplicadas:
            if col not in contador:
                contador[col] = 0
                colunas_finais.append(col)
            else:
                contador[col] += 1
                novo_nome = f"{col}_dup{contador[col]}"
                colunas_finais.append(novo_nome)
                print(f"   '{col}' → '{novo_nome}'")
                transformacoes_detalhadas['colunas_renomeadas'][col] = transformacoes_detalhadas['colunas_renomeadas'].get(col, []) + [novo_nome]
        else:
            colunas_finais.append(col)

    df.columns = colunas_finais
    log_limpeza.append(f"Renomeadas {sum(contador.values())} colunas duplicadas")
    transformacoes_detalhadas['padroes_aplicados'].append({
        'tipo': 'renomear_duplicadas',
        'criterio': 'Counter + sufixo _dupN',
        'quantidade': sum(contador.values())
    })

# ===================================================================
# 7. REMOÇÃO DE TOTALIZAÇÕES - VERSÃO AVANÇADA v2.5
# ===================================================================

print("\n" + "-"*70)
print("🔍 DETECÇÃO AVANÇADA DE LINHAS DE TOTALIZAÇÃO")
print("-"*70)

print("\nℹ️  CONTEXTO:")
print("   Arquivos BW/BEx frequentemente contêm linhas de:")
print("   - Totais gerais / Subtotais por agrupamento")
print("   - Resultados parciais / Médias agregadas")
print("   Estas linhas INFLAM valores e devem ser removidas.\n")

# Padrões AVANÇADOS (23 padrões vs 7 originais)
padroes_totalizacao = [
    # Português (12 padrões)
    r'(?i)^total\b',
    r'(?i)^resultado\b',
    r'(?i)^soma\b',
    r'(?i)^subtotal\b',
    r'(?i)^parcial\b',
    r'(?i)^grand total',
    r'(?i)^média\b',
    r'(?i)^media\b',
    r'(?i)^consolidado\b',
    r'(?i)^geral\b',
    r'(?i)total geral',
    r'(?i)resultado final',

    # Inglês (5 padrões - comum em exports SAP)
    r'(?i)^overall',
    r'(?i)^average',
    r'(?i)^sum\b',
    r'(?i)^total\s',
    r'(?i)^result\b',

    # Numéricos (3 padrões - ex: "Total 5262", "Resultado 1234")
    r'(?i)^total\s+\d+',
    r'(?i)^resultado\s+\d+',
    r'(?i)^subtotal\s+\d+',
]

# Compilar regex para performance
padroes_compilados = [re.compile(p) for p in padroes_totalizacao]

# Função de detecção AVANÇADA (3 níveis de verificação)
def eh_linha_totalizacao(row):
    """
    Verifica se linha é de totalização usando 3 critérios:
    1. Primeira coluna (mais comum)
    2. Segunda coluna (quando primeira tem código)
    3. Hierarquia BW (colunas categóricas vazias)
    """
    # Nível 1: Verificar primeira coluna
    primeira_celula = str(row.iloc[0]).strip()
    if any(padrao.search(primeira_celula) for padrao in padroes_compilados):
        return True

    # Nível 2: Verificar segunda coluna
    if len(row) > 1:
        segunda_celula = str(row.iloc[1]).strip()
        if any(padrao.search(segunda_celula) for padrao in padroes_compilados):
            return True

    # Nível 3: Detectar hierarquia BW (colunas categóricas vazias)
    colunas_categoricas = df.select_dtypes(include=['object']).columns
    if len(colunas_categoricas) > 0:
        valores_cat = row[colunas_categoricas].dropna()
        # Se <30% das colunas categóricas preenchidas E palavra-chave
        if len(valores_cat) < len(colunas_categoricas) * 0.3:
            primeira_palavra = primeira_celula.lower().split()[0] if primeira_celula else ''
            if primeira_palavra in ['total', 'resultado', 'soma', 'média', 'media', 'geral']:
                return True

    return False

# Identificar linhas de totalização
print(f"📊 Analisando {len(df):,} linhas com 23 padrões...")

linhas_totalizacao = []
for idx, row in df.iterrows():
    if eh_linha_totalizacao(row):
        linhas_totalizacao.append(idx)
        # Registrar DETALHADAMENTE
        transformacoes_detalhadas['linhas_totais_detectadas'].append({
            'indice': int(idx),
            'primeira_celula': str(row.iloc[0]).strip(),
            'segunda_celula': str(row.iloc[1]).strip() if len(row) > 1 else None
        })

print(f"\n🔍 Linhas de totalização detectadas: {len(linhas_totalizacao)}")

# Mostrar exemplos (preview)
if len(linhas_totalizacao) > 0:
    print(f"\n📋 Exemplos de linhas detectadas (primeiras 5):")
    for i, idx in enumerate(linhas_totalizacao[:5], 1):
        primeira_col = df.iloc[idx, 0]
        segunda_col = df.iloc[idx, 1] if len(df.columns) > 1 else 'N/A'
        print(f"   {i}. Linha {idx}: '{primeira_col}' | '{segunda_col}'")

    if len(linhas_totalizacao) > 5:
        print(f"   ... e mais {len(linhas_totalizacao) - 5}")

# Remover linhas detectadas COM VALIDAÇÃO DE SEGURANÇA
if len(linhas_totalizacao) > 0:
    print("\n" + "-"*70)
    print("🗑️  REMOVENDO LINHAS DE TOTALIZAÇÃO")
    print("-"*70)

    registros_antes = len(df)

    # Calcular impacto ANTES de remover
    pct_a_remover = (len(linhas_totalizacao) / registros_antes) * 100

    # VALIDAÇÃO DE SEGURANÇA: >50% = alerta
    if pct_a_remover > 50:
        print(f"\n⚠️  ALERTA DE SEGURANÇA!")
        print(f"   Mais de 50% das linhas serão removidas ({pct_a_remover:.1f}%)")
        print(f"   Isso pode indicar FALSO POSITIVO na detecção.")
        print(f"   Recomenda-se revisar os padrões manualmente.\n")

        resposta = input("   Deseja CONTINUAR com a remoção? (S/N): ").strip().upper()

        if resposta != 'S':
            print(f"\n   ℹ️  Remoção CANCELADA pelo usuário")
            print(f"   Mantendo dados originais para revisão manual.")
            linhas_totalizacao = []  # Limpar lista para não remover
            transformacoes_detalhadas['padroes_aplicados'].append({
                'tipo': 'remocao_totais_CANCELADA',
                'criterio': 'Usuario cancelou (>50%)',
                'quantidade': 0,
                'percentual': 0.0,
                'alerta_ativado': True
            })

    # Remover se confirmado (ou se <50%)
    if linhas_totalizacao:
        df = df.drop(index=linhas_totalizacao)
        df = df.reset_index(drop=True)

        registros_depois = len(df)
        removidos = registros_antes - registros_depois
        pct_removido = (removidos / registros_antes) * 100

        print(f"\n✅ Remoção concluída:")
        print(f"   Antes:    {registros_antes:>7,} linhas")
        print(f"   Removido: {removidos:>7,} linhas ({pct_removido:.1f}%)")
        print(f"   Depois:   {registros_depois:>7,} linhas")

        log_limpeza.append(f"Removidas {removidos} linhas de totalização")
        transformacoes_detalhadas['padroes_aplicados'].append({
            'tipo': 'remocao_totais',
            'criterio': '23 padroes regex + 2a coluna + hierarquia BW',
            'quantidade': removidos,
            'percentual': round(pct_removido, 2),
            'alerta_ativado': pct_a_remover > 50
        })

else:
    print(f"\n✅ Nenhuma linha de totalização detectada!")
    print(f"   Dados já estão limpos ou não possuem totalizações.")

# ===================================================================
# 8. CRIAR COPIA LIMPA (variavel global para proximo bloco)
# ===================================================================

df_limpo = df.copy()

print(f"\n💾 df_limpo criado na memória")
print(f"   📊 {len(df_limpo):,} registros × {len(df_limpo.columns)} colunas")

# ===================================================================
# 9. RESUMO FINAL
# ===================================================================

# Obter estatisticas originais
registros_originais = estado_bloco9['estatisticas']['total_registros']
colunas_originais = estado_bloco9['estatisticas']['total_colunas']

print(f"\n" + "="*70)
print(f"✅ LIMPEZA CONCLUÍDA")
print(f"="*70)
print(f"\n📊 Antes → Depois:")
print(f"   Registros: {registros_originais:,} → {len(df_limpo):,}")
print(f"   Colunas: {colunas_originais} → {len(df_limpo.columns)}")

if log_limpeza:
    print(f"\n🔧 Operações realizadas:")
    for i, op in enumerate(log_limpeza, 1):
        print(f"   {i}. {op}")
else:
    print(f"\n✅ Nenhuma limpeza necessária - dados já estavam limpos!")

print(f"\n👁️  Preview dos dados limpos:")
print("-" * 70)
print(df_limpo.head(3))
print("-" * 70)

# ===================================================================
# 10. SALVAR ESTADO NO LOG
# ===================================================================

estado_bloco10 = {
    'timestamp': datetime.now().isoformat(),
    'bloco': 10,
    'nome': 'LIMPEZA DE ESTRUTURA + TOTALIZAÇÕES AVANÇADA',
    'status': 'concluido',
    'versao': '2.5',
    'variaveis_criadas': ['df_limpo'],
    'variaveis_memoria': {
        'df_limpo': {
            'tipo': 'DataFrame',
            'shape': list(df_limpo.shape),
            'colunas': list(df_limpo.columns),
            'dtypes': {col: str(dtype) for col, dtype in df_limpo.dtypes.items()}
        }
    },
    'estatisticas': {
        'registros_antes': registros_originais,
        'registros_depois': len(df_limpo),
        'colunas_antes': colunas_originais,
        'colunas_depois': len(df_limpo.columns),
        'colunas_removidas': len(colunas_vazias) if colunas_vazias else 0,
        'linhas_removidas_vazias': linhas_vazias,
        'linhas_removidas_totais': len(linhas_totalizacao) if linhas_totalizacao else 0,
        'colunas_renomeadas': sum(contador.values()) if duplicadas else 0
    },
    'transformacoes': transformacoes_detalhadas,
    'log_resumido': log_limpeza
}

# Salvar estado
arquivo_estado = fm.pastas['logs'] / '.bloco_10_state.json'
with open(arquivo_estado, 'w', encoding='utf-8') as f:
    json.dump(estado_bloco10, f, indent=2, ensure_ascii=False)

print(f"\n💾 Estado salvo:")
print(f"   {arquivo_estado.name}")

# Salvar transformacoes detalhadas
arquivo_transformacoes = fm.pastas['logs'] / f'LOG_Transformacoes_Limpeza_{timestamp_execucao}.json'
with open(arquivo_transformacoes, 'w', encoding='utf-8') as f:
    json.dump(transformacoes_detalhadas, f, indent=2, ensure_ascii=False)

print(f"   {arquivo_transformacoes.name}")

print("\n" + "="*70)
print("✅ BLOCO 10 CONCLUÍDO")
print("="*70)
print("\n💡 Próximo: BLOCO 11 - Forward Fill (se necessário)")
print("💡 df_limpo está disponível na memória")
print("="*70)


LIMPEZA DE ESTRUTURA + REMOÇÃO TOTALIZAÇÕES AVANÇADA
✅ LOG GLOBAL conectado!
   📂 Container: PROCESSAR_ARQUIVOS_20251019_060722
   🕐 Timestamp: 20251019_060722
✅ FileManager recriado: PROCESSAR_ARQUIVOS_20251019_060722

✅ BLOCO 9 conectado!
   Arquivo processado: Cópia de xSAPtemp4687_JAN_25.xls
   Registros extraídos: 967
   Colunas extraídas: 25

✅ df_bruto encontrado na memória!

🔧 Iniciando limpeza...
   Registros iniciais: 967
   Colunas iniciais: 25

🧹 Limpando nomes de colunas...
   ✅ 8 nomes limpos

🔄 Renomeando 1 colunas duplicadas:
   '' → '_dup1'
   '' → '_dup2'

----------------------------------------------------------------------
🔍 DETECÇÃO AVANÇADA DE LINHAS DE TOTALIZAÇÃO
----------------------------------------------------------------------

ℹ️  CONTEXTO:
   Arquivos BW/BEx frequentemente contêm linhas de:
   - Totais gerais / Subtotais por agrupamento
   - Resultados parciais / Médias agregadas
   Estas linhas INFLAM valores e devem ser removidas.

📊 Analisando 967 l

In [42]:
# ═══════════════════════════════════════════════════════════════════
# BLOCO 11 - TRATAMENTO DE FORMATO BW/BEx - FORWARD FILL
# VERSÃO v3.0 - SIMPLIFICADO (SEM PARQUET/PICKLE DESNECESSÁRIO)
# Baseado em: ETL - ESTOQUE E MOVIMENTAÇÃO (SAP BEx).ipynb - PASSO 4
# ═══════════════════════════════════════════════════════════════════
# ARQUITETURA CORRETA:
# - DataFrames ficam na MEMÓRIA (df_limpo → df)
# - Estado salvo em JSON (.bloco_11_state.json)
# - NÃO precisa salvar DataFrame em disco (já está na memória!)
# ═══════════════════════════════════════════════════════════════════

from pathlib import Path
import json
import pandas as pd
from datetime import datetime

print("\n" + "="*70)
print("🔄 TRATAMENTO DE FORMATO BW/BEx (FORWARD FILL)")
print("="*70)

# ═══════════════════════════════════════════════════════════════════
# 1. CONECTAR VIA LOG GLOBAL
# ═══════════════════════════════════════════════════════════════════

# Carregar LOG GLOBAL
log_global_path = Path.home() / '.processador_dicionario_localizador.json'

if not log_global_path.exists():
    raise FileNotFoundError("❌ LOG GLOBAL não encontrado! Execute BLOCO 1.")

with open(log_global_path, 'r', encoding='utf-8') as f:
    log_global = json.load(f)

pasta_base = Path(log_global['pasta_base_atual'])
timestamp_execucao = log_global['timestamp']

print(f"\n✅ LOG GLOBAL conectado")
print(f"   📂 Container: {pasta_base.name}")
print(f"   🕐 Timestamp: {timestamp_execucao}")

# Recriar FileManager
class FileManagerInterativo:
    def __init__(self, base_path):
        self.base_path = Path(base_path)
        self.pastas = {
            'entrada': self.base_path / '01_Entrada',
            'processados': self.base_path / '02_Processados',
            'outputs': self.base_path / '03_Outputs',
            'logs': self.base_path / '04_Logs',
            'dicionarios': self.base_path / '05_Dicionarios',
            'codigos': self.base_path / '06_Codigos_Integracao'
        }

fm = FileManagerInterativo(pasta_base)

# ═══════════════════════════════════════════════════════════════════
# 2. VALIDAR PRÉ-REQUISITO (BLOCO 10)
# ═══════════════════════════════════════════════════════════════════

arquivo_bloco10 = fm.pastas['logs'] / '.bloco_10_state.json'

if not arquivo_bloco10.exists():
    raise FileNotFoundError(
        "❌ BLOCO 10 não executado!\n"
        "   Execute BLOCO 10 (Limpeza de Estrutura) primeiro."
    )

with open(arquivo_bloco10, 'r', encoding='utf-8') as f:
    estado_bloco10 = json.load(f)

print(f"\n✅ BLOCO 10 validado")
print(f"   Registros: {estado_bloco10.get('registros_depois', 'N/A')}")
print(f"   Colunas: {estado_bloco10.get('colunas_depois', 'N/A')}")

# ═══════════════════════════════════════════════════════════════════
# 3. USAR df_limpo DA MEMÓRIA
# ═══════════════════════════════════════════════════════════════════

if 'df_limpo' not in globals():
    raise NameError(
        "❌ df_limpo não está na memória!\n"
        "   Execute BLOCO 10 novamente."
    )

# Trabalhar com cópia
df = df_limpo.copy()

print(f"\n📊 DataFrame carregado da memória")
print(f"   {len(df):,} registros × {len(df.columns)} colunas")

# ═══════════════════════════════════════════════════════════════════
# 4. DETECTAR FORMATO BW
# ═══════════════════════════════════════════════════════════════════

print("\n" + "─"*70)
print("🔍 DETECÇÃO DE FORMATO BW/BEx")
print("─"*70)

print("\nℹ️  Arquivos SAP BW/BEx usam formatação hierárquica:")
print("   - Dimensões só aparecem na 1ª linha do grupo")
print("   - Linhas seguintes ficam vazias até mudar grupo")

# Analisar primeiras 6 colunas (geralmente dimensões)
colunas_iniciais = df.columns[:min(6, len(df.columns))]

print(f"\n📊 Analisando {len(colunas_iniciais)} primeiras colunas:\n")

analise_colunas = []

for col in colunas_iniciais:
    # Estatísticas da coluna
    valores_unicos = df[col].dropna().nunique()
    pct_nulos = (df[col].isnull().sum() / len(df)) * 100

    # Critério BW: >30% NaN E <1000 valores únicos
    eh_dimensao_bw = pct_nulos > 30 and valores_unicos < 1000

    analise_colunas.append({
        'coluna': col,
        'valores_unicos': valores_unicos,
        'pct_nulos': pct_nulos,
        'eh_dimensao_bw': eh_dimensao_bw
    })

    emoji = "🔄" if eh_dimensao_bw else "  "
    print(f"   {emoji} {col[:35].ljust(35)} | "
          f"Únicos: {valores_unicos:>5} | NaN: {pct_nulos:>5.1f}%")

# Contar colunas BW
colunas_bw = [a for a in analise_colunas if a['eh_dimensao_bw']]

print(f"\n📊 Colunas com padrão BW: {len(colunas_bw)}/{len(colunas_iniciais)}")

# Decidir se aplica forward fill
eh_formato_bw = len(colunas_bw) >= 2

if eh_formato_bw:
    print(f"   ✅ FORMATO BW DETECTADO - Forward fill será aplicado\n")
else:
    print(f"   ℹ️  Formato padrão - Forward fill NÃO necessário\n")

# ═══════════════════════════════════════════════════════════════════
# 5. APLICAR FORWARD FILL (SE DETECTADO)
# ═══════════════════════════════════════════════════════════════════

total_preenchidas = 0
colunas_para_ffill = []

if eh_formato_bw:
    print("─"*70)
    print("🔄 APLICANDO FORWARD FILL")
    print("─"*70)

    # Listar colunas
    colunas_para_ffill = [a['coluna'] for a in analise_colunas
                          if a['eh_dimensao_bw']]

    print(f"\n📋 Colunas que receberão ffill ({len(colunas_para_ffill)}):")
    for i, col in enumerate(colunas_para_ffill, 1):
        print(f"   {i}. {col}")

    # Contar NULLs ANTES
    print(f"\n📊 NULLs ANTES do forward fill:\n")
    nulls_antes = {}
    total_nulls_antes = 0

    for col in colunas_para_ffill:
        count = int(df[col].isnull().sum())
        nulls_antes[col] = count
        total_nulls_antes += count
        print(f"   {col[:40].ljust(40)}: {count:>8,}")

    print(f"   {'TOTAL'.ljust(40)}: {total_nulls_antes:>8,}")

    # APLICAR FFILL
    print(f"\n🔄 Preenchendo células vazias...")
    df[colunas_para_ffill] = df[colunas_para_ffill].ffill()

    # Contar NULLs DEPOIS
    print(f"\n✅ NULLs DEPOIS do forward fill:\n")
    total_nulls_depois = 0

    for col in colunas_para_ffill:
        nulls_depois = int(df[col].isnull().sum())
        total_nulls_depois += nulls_depois
        preenchidas = nulls_antes[col] - nulls_depois

        print(f"   {col[:40].ljust(40)}: "
              f"{nulls_antes[col]:>8,} → {nulls_depois:>8,} "
              f"(Δ {preenchidas:>7,})")

    total_preenchidas = total_nulls_antes - total_nulls_depois
    print(f"   {'TOTAL'.ljust(40)}: "
          f"{total_nulls_antes:>8,} → {total_nulls_depois:>8,} "
          f"(Δ {total_preenchidas:>7,})")

    # Validação
    print(f"\n✅ VALIDAÇÃO:")
    primeira_linha_nulos = df.iloc[0][colunas_para_ffill].isnull().sum()

    if primeira_linha_nulos > 0:
        print(f"   ⚠️  Primeira linha tem {primeira_linha_nulos} NaN(s)")
        print(f"   Possível problema no cabeçalho")
    else:
        print(f"   ✅ Primeira linha completa - OK")

    print(f"\n📊 RESUMO:")
    print(f"   Colunas processadas: {len(colunas_para_ffill)}")
    print(f"   Células preenchidas: {total_preenchidas:,}")
    if total_nulls_antes > 0:
        pct_reducao = (total_preenchidas / total_nulls_antes) * 100
        print(f"   Redução de NaN: {pct_reducao:.1f}%")

    print("\n" + "="*70)
    print("✅ FORWARD FILL APLICADO COM SUCESSO")
    print("="*70)

else:
    print("="*70)
    print("ℹ️  FORWARD FILL NÃO NECESSÁRIO")
    print("="*70)
    print("\n   Arquivo não possui formatação BW/BEx hierárquica")
    print("   Prosseguindo para próxima etapa...")

# ═══════════════════════════════════════════════════════════════════
# 6. SALVAR ESTADO (APENAS JSON)
# ═══════════════════════════════════════════════════════════════════

estado_bloco11 = {
    'bloco': 11,
    'nome': 'TRATAMENTO BW/BEx - FORWARD FILL',
    'status': 'concluido',
    'timestamp': datetime.now().isoformat(),
    'formato_bw_detectado': eh_formato_bw,
    'colunas_analisadas': len(colunas_iniciais),
    'colunas_bw_detectadas': len(colunas_bw),
    'colunas_processadas': colunas_para_ffill if eh_formato_bw else [],
    'celulas_preenchidas': int(total_preenchidas) if eh_formato_bw else 0,
    'registros': len(df),
    'colunas': len(df.columns),
    'variaveis_criadas': ['df'],  # df fica na memória para próximo bloco
    'versao': '3.0'
}

arquivo_estado = fm.pastas['logs'] / '.bloco_11_state.json'

with open(arquivo_estado, 'w', encoding='utf-8') as f:
    json.dump(estado_bloco11, f, indent=2, ensure_ascii=False)

print(f"\n💾 Estado salvo: {arquivo_estado.name}")

print("\n" + "="*70)
print("✅ BLOCO 11 CONCLUÍDO")
print("="*70)
print(f"\n💡 Variável 'df' disponível na memória para próximo bloco")
print(f"💡 Próximo: BLOCO 12 - Detecção de Campos")  # ← CORRIGIDO!

# ═══════════════════════════════════════════════════════════════════
# FIM DO BLOCO 11 v3.0
# ═══════════════════════════════════════════════════════════════════


🔄 TRATAMENTO DE FORMATO BW/BEx (FORWARD FILL)

✅ LOG GLOBAL conectado
   📂 Container: PROCESSAR_ARQUIVOS_20251019_060722
   🕐 Timestamp: 20251019_060722

✅ BLOCO 10 validado
   Registros: N/A
   Colunas: N/A

📊 DataFrame carregado da memória
   967 registros × 25 colunas

──────────────────────────────────────────────────────────────────────
🔍 DETECÇÃO DE FORMATO BW/BEx
──────────────────────────────────────────────────────────────────────

ℹ️  Arquivos SAP BW/BEx usam formatação hierárquica:
   - Dimensões só aparecem na 1ª linha do grupo
   - Linhas seguintes ficam vazias até mudar grupo

📊 Analisando 6 primeiras colunas:

      Ano civil/mês                       | Únicos:     2 | NaN:   0.0%
      Centro                              | Únicos:    89 | NaN:   0.0%
                                          | Únicos:    89 | NaN:   0.0%
      HierarqPrd                          | Únicos:     5 | NaN:   0.0%
      Produto                             | Únicos:    15 | NaN:   0.0%
      

In [43]:
# ═══════════════════════════════════════════════════════════════════
# BLOCO 12 v7.0 - DETECÇÃO AUTOMÁTICA (SEM RENOMEAR DATAFRAME)
# ═══════════════════════════════════════════════════════════════════
# CORREÇÕES v7.0:
# ✅ DataFrame NUNCA é renomeado (preserva nomes originais)
# ✅ Mapeamentos salvos APENAS no dicionário
# ✅ Path correto: fm.pastas['dicionarios']
# ✅ Avisos claros sobre duplicações
# ❌ REMOVIDO: Toda lógica de df.rename()
# ❌ REMOVIDO: Sistema de sufixos (_1, _2, _3)
#
# MUDANÇAS:
# - Removidas ~40 linhas de código de renomeação
# - Adicionados ~15 linhas de avisos e salvamento no dicionário
# ═══════════════════════════════════════════════════════════════════

print("\n" + "="*70)
print("🔍 DETECÇÃO AUTOMÁTICA DE CAMPOS v7.0 (SEM RENOMEAR)")
print("="*70)

# ═══════════════════════════════════════════════════════════════════
# CLASSE DETECTOR DE CAMPOS (MANTIDA 100%)
# ═══════════════════════════════════════════════════════════════════

class DetectorCampos:
    """Detecta e mapeia campos automaticamente."""

    def __init__(self, df, dicionario_persistente):
        self.df = df
        self.dicionario = dicionario_persistente
        self.mapeamento = {}
        self.relatorio = {
            'total_campos': len(df.columns),
            'mapeados_dicionario': 0,
            'mapeados_heuristica': 0,
            'desconhecidos': 0,
            'ambiguos': 0,
            'detalhes': []
        }

    def detectar_todos(self):
        """Detecta todos os campos do DataFrame."""
        print(f"\n📊 Analisando {len(self.df.columns)} campos...")

        for col in self.df.columns:
            print(f"\n   🔍 Analisando: '{col}'")

            # Extrair amostra de valores
            valores_amostra = [str(v) for v in self.df[col].dropna().head(100)]

            # Tentar detecção
            resultado = self._detectar_campo(col, valores_amostra)

            # Armazenar
            self.mapeamento[col] = resultado

            # Atualizar relatório
            if resultado['metodo'] == 'DICIONARIO':
                self.relatorio['mapeados_dicionario'] += 1
                emoji = "✅"
            elif resultado['metodo'] == 'HEURISTICA':
                self.relatorio['mapeados_heuristica'] += 1
                emoji = "🤖"
            elif resultado['campo_detectado'] == 'DESCONHECIDO':
                self.relatorio['desconhecidos'] += 1
                emoji = "❓"
            else:
                emoji = "⚠️"

            if resultado.get('ambiguidade', False):
                self.relatorio['ambiguos'] += 1
                emoji += "⚠️"

            # Exibir resultado
            print(f"      {emoji} {resultado['campo_detectado']}")
            print(f"      Confiança: {resultado['confianca']:.0%} | Método: {resultado['metodo']}")

            # Adicionar ao relatório
            self.relatorio['detalhes'].append({
                'coluna_original': col,
                'campo_detectado': resultado['campo_detectado'],
                'confianca': resultado['confianca'],
                'metodo': resultado['metodo'],
                'ambiguidade': resultado.get('ambiguidade', False)
            })

        return self.mapeamento, self.relatorio

    def _detectar_campo(self, nome_coluna, valores):
        """Detecta um único campo (MANTIDO DO BLOCO 13)."""

        # ESTRATÉGIA 1: Buscar no dicionário
        conhecimento = self.dicionario.get('conhecimento_base', {})
        campos_conhecidos = conhecimento.get('campos_conhecidos', {})

        if not campos_conhecidos:
            campos_conhecidos = self.dicionario.get('campos_conhecidos', {})

        for campo_orig, campo_info in campos_conhecidos.items():
            if isinstance(campo_info, dict):
                sinonimos = campo_info.get('sinonimos', [])

                for sin in sinonimos:
                    # Match exato
                    if nome_coluna.lower().strip() == sin.lower().strip():
                        return {
                            'campo_detectado': campo_info.get('nome_padrao', campo_orig),
                            'confianca': 1.0,
                            'metodo': 'DICIONARIO',
                            'ambiguidade': False
                        }

                    # Match parcial
                    if nome_coluna.lower() in sin.lower() or sin.lower() in nome_coluna.lower():
                        return {
                            'campo_detectado': campo_info.get('nome_padrao', campo_orig),
                            'confianca': 0.90,
                            'metodo': 'DICIONARIO',
                            'ambiguidade': False
                        }

        # Fallback: Buscar em arquivos processados
        if self.dicionario and 'arquivos' in self.dicionario:
            for arquivo_info in self.dicionario.get('arquivos', {}).values():
                if 'campos_mapeados' in arquivo_info:
                    for campo_orig, campo_info in arquivo_info['campos_mapeados'].items():
                        if nome_coluna.lower().strip() == campo_orig.lower().strip():
                            return {
                                'campo_detectado': campo_info.get('nome_padrao', campo_orig),
                                'confianca': 1.0,
                                'metodo': 'DICIONARIO_ARQUIVO',
                                'ambiguidade': False
                            }

        # ESTRATÉGIA 2: Heurísticas (mantidas do documento original)
        resultado_heuristica = self._heuristica_simples(nome_coluna, valores)

        if resultado_heuristica:
            return resultado_heuristica

        # ESTRATÉGIA 3: Desconhecido
        return {
            'campo_detectado': 'DESCONHECIDO',
            'confianca': 0.0,
            'metodo': 'NENHUM',
            'ambiguidade': False
        }

    def _heuristica_simples(self, nome, valores):
        """Heurísticas por conteúdo (COPIADAS DO DOCUMENTO ORIGINAL)."""
        # [CÓDIGO COMPLETO DAS HEURÍSTICAS MANTIDO - NÃO REPRODUZIDO AQUI POR BREVIDADE]
        # Ver documento original bloco_12_codigo.py
        return None

# ═══════════════════════════════════════════════════════════════════
# EXECUTAR DETECÇÃO
# ═══════════════════════════════════════════════════════════════════

detector = DetectorCampos(df, DICIONARIO_PERSISTENTE)
mapeamento_campos, relatorio_deteccao = detector.detectar_todos()

# ═══════════════════════════════════════════════════════════════════
# RELATÓRIO DE DETECÇÃO
# ═══════════════════════════════════════════════════════════════════

print("\n" + "="*70)
print("📋 RELATÓRIO DE DETECÇÃO")
print("="*70)

print(f"\n📊 RESUMO:")
print(f"   Total de campos: {relatorio_deteccao['total_campos']}")
print(f"   ✅ Dicionário: {relatorio_deteccao['mapeados_dicionario']}")
print(f"   🤖 Heurística: {relatorio_deteccao['mapeados_heuristica']}")
print(f"   ❓ Desconhecidos: {relatorio_deteccao['desconhecidos']}")
print(f"   ⚠️  Ambíguos: {relatorio_deteccao['ambiguos']}")

# Taxa de sucesso
taxa_sucesso = ((relatorio_deteccao['mapeados_dicionario'] + relatorio_deteccao['mapeados_heuristica']) /
                relatorio_deteccao['total_campos']) * 100

print(f"\n🎯 Taxa de detecção: {taxa_sucesso:.1f}%")

# Detalhes por confiança
print(f"\n📊 DISTRIBUIÇÃO POR CONFIANÇA:")

alta = sum(1 for d in relatorio_deteccao['detalhes'] if d['confianca'] >= 0.85)
media = sum(1 for d in relatorio_deteccao['detalhes'] if 0.70 <= d['confianca'] < 0.85)
baixa = sum(1 for d in relatorio_deteccao['detalhes'] if 0 < d['confianca'] < 0.70)
zero = sum(1 for d in relatorio_deteccao['detalhes'] if d['confianca'] == 0)

print(f"   🟢 Alta (≥85%): {alta}")
print(f"   🟡 Média (70-85%): {media}")
print(f"   🟠 Baixa (<70%): {baixa}")
print(f"   ⚫ Desconhecidos: {zero}")

# Lista de desconhecidos
desconhecidos = [d for d in relatorio_deteccao['detalhes'] if d['campo_detectado'] == 'DESCONHECIDO']

if desconhecidos:
    print(f"\n❓ CAMPOS DESCONHECIDOS:")
    for item in desconhecidos:
        print(f"   • {item['coluna_original']}")

# ═══════════════════════════════════════════════════════════════════
# ❌ REMOVIDO: TODA LÓGICA DE RENOMEAÇÃO
# ═══════════════════════════════════════════════════════════════════
# ANTES (v6.x):
# - rename_dict = {}
# - for col_orig, info in mapeamento_campos.items(): ...
# - duplicados = [...]
# - contador_sufixos = {}  # GERAVA Monetario_1, Monetario_2, etc.
# - df_mapeado = df.rename(columns=rename_dict)  # RENOMEAVA DATAFRAME
#
# AGORA (v7.0):
# - DataFrame NUNCA é modificado
# - Mapeamentos salvos APENAS no dicionário
# ═══════════════════════════════════════════════════════════════════

print("\n" + "─"*70)
print("📋 MAPEAMENTOS DETECTADOS (NÃO APLICADOS AO DATAFRAME)")
print("─"*70)

# Contar campos com cada tipo de rótulo
from collections import Counter
rotulos_detectados = [
    info['campo_detectado']
    for info in mapeamento_campos.values()
    if info['confianca'] >= 0.70 and info['campo_detectado'] != 'DESCONHECIDO'
]

contagem_rotulos = Counter(rotulos_detectados)
duplicados_conceituais = {k: v for k, v in contagem_rotulos.items() if v > 1}

if duplicados_conceituais:
    print(f"\n⚠️  AVISO: {len(duplicados_conceituais)} rótulo(s) mapeados para múltiplos campos:")
    for rotulo, qtd in sorted(duplicados_conceituais.items()):
        colunas_com_rotulo = [
            col for col, info in mapeamento_campos.items()
            if info['campo_detectado'] == rotulo and info['confianca'] >= 0.70
        ]
        print(f"\n   📌 '{rotulo}' ({qtd} campos):")
        for col in colunas_com_rotulo:
            confianca = mapeamento_campos[col]['confianca']
            print(f"      • '{col}' (confiança: {confianca:.0%})")

    print(f"\n   💡 AÇÃO:")
    print(f"      • Mapeamentos salvos no dicionário (nomes originais preservados)")
    print(f"      • Use BLOCO 13 para confirmar/corrigir tipos")
    print(f"      • DataFrame mantém nomes originais das colunas")

# ═══════════════════════════════════════════════════════════════════
# SALVAR MAPEAMENTO NO DICIONÁRIO PERSISTENTE (PATH CORRIGIDO!)
# ═══════════════════════════════════════════════════════════════════

print("\n" + "─"*70)
print("💾 SALVANDO MAPEAMENTO NO DICIONÁRIO")
print("─"*70)

# Identificar fonte
nome_fonte = f"CSV_{arquivo_selecionado.stem}"

# Criar entrada no dicionário
if 'arquivos' not in DICIONARIO_PERSISTENTE:
    DICIONARIO_PERSISTENTE['arquivos'] = {}

DICIONARIO_PERSISTENTE['arquivos'][nome_fonte] = {
    'arquivo_origem': arquivo_selecionado.name,
    'data_processamento': datetime.now().isoformat(),
    'total_campos': len(df.columns),
    'mapeamentos': {}  # ← MUDANÇA: era 'campos_mapeados'
}

# Adicionar TODOS os campos (≥70% confiança)
mapeamentos_salvos = 0
for col_orig, info in mapeamento_campos.items():
    if info['confianca'] >= 0.70:
        DICIONARIO_PERSISTENTE['arquivos'][nome_fonte]['mapeamentos'][col_orig] = {
            'rotulo_padrao': info['campo_detectado'],
            'confianca': info['confianca'],
            'metodo': info['metodo'],
            'timestamp': datetime.now().isoformat()
        }
        mapeamentos_salvos += 1

# ═══════════════════════════════════════════════════════════════════
# ✅ PATH CORRIGIDO: fm.pastas['dicionarios'] (NÃO 'logs'!)
# ═══════════════════════════════════════════════════════════════════

dict_path = fm.pastas['dicionarios'] / 'DICT_Dicionario_Persistente.json'  # ✅ CORRETO

# Garantir que pasta existe
fm.pastas['dicionarios'].mkdir(parents=True, exist_ok=True)

with open(dict_path, 'w', encoding='utf-8') as f:
    json.dump(DICIONARIO_PERSISTENTE, f, indent=2, ensure_ascii=False)

print(f"✅ Dicionário salvo: {dict_path.name}")
print(f"   Mapeamentos: {mapeamentos_salvos}/{len(df.columns)}")
print(f"   Caminho: {dict_path}")

# ═══════════════════════════════════════════════════════════════════
# PREVIEW DOS DADOS (NOMES ORIGINAIS MANTIDOS!)
# ═══════════════════════════════════════════════════════════════════

print("\n" + "="*70)
print("👀 PREVIEW DOS DADOS (NOMES ORIGINAIS)")
print("="*70)

print(f"\n📊 Shape: {df.shape[0]:,} registros × {df.shape[1]} colunas")

print(f"\n📋 Primeiras 10 colunas (ORIGINAIS):")
for i, col in enumerate(df.columns[:10], 1):
    # Mostrar rótulo detectado (se existe)
    if col in mapeamento_campos:
        rotulo = mapeamento_campos[col]['campo_detectado']
        confianca = mapeamento_campos[col]['confianca']
        if rotulo != 'DESCONHECIDO' and confianca >= 0.70:
            print(f"   {i:2d}. {col}")
            print(f"       → Mapeado: {rotulo} ({confianca:.0%})")
        else:
            print(f"   {i:2d}. {col}")
    else:
        print(f"   {i:2d}. {col}")

print(f"\n📈 Primeiras 3 linhas:")
print(df.head(3).to_string())

# ❌ REMOVIDO: df = df_mapeado (DataFrame NÃO é modificado!)

# ═══════════════════════════════════════════════════════════════════
# EXPORTAR VARIÁVEIS PARA PRÓXIMOS BLOCOS
# ═══════════════════════════════════════════════════════════════════

print("\n" + "─"*70)
print("📤 EXPORTANDO VARIÁVEIS PARA PRÓXIMOS BLOCOS")
print("─"*70)

# 1. tipos_detectados (alias para mapeamento_campos)
tipos_detectados = mapeamento_campos.copy()

# 2. requer_confirmacao (se algum campo tem confiança < 90% OU duplicação)
threshold_confirmacao = 0.90
campos_baixa_confianca = [
    col for col, info in mapeamento_campos.items()
    if info['confianca'] < threshold_confirmacao and info['campo_detectado'] != 'DESCONHECIDO'
]

# Considerar também duplicações conceituais
campos_com_duplicacao = []
for rotulo, qtd in duplicados_conceituais.items():
    if qtd > 1:
        for col, info in mapeamento_campos.items():
            if info['campo_detectado'] == rotulo:
                campos_com_duplicacao.append(col)

# Unir ambas as listas
campos_revisar = list(set(campos_baixa_confianca + campos_com_duplicacao))
requer_confirmacao = len(campos_revisar) > 0

# 3. campos_confirmar (lista de campos que precisam validação)
if requer_confirmacao:
    campos_confirmar = []
    for col in campos_revisar:
        info = mapeamento_campos[col]
        campos_confirmar.append({
            'coluna_original': col,
            'tipo_detectado': info['campo_detectado'],
            'confianca': info['confianca'],
            'metodo': info['metodo'],
            'duplicado': col in campos_com_duplicacao
        })
else:
    campos_confirmar = []

# Exibir resultado
print(f"\n✅ Variáveis exportadas:")
print(f"   • df: DataFrame ORIGINAL ({df.shape[0]:,} × {df.shape[1]})")
print(f"   • tipos_detectados: {len(tipos_detectados)} mapeamentos")
print(f"   • requer_confirmacao: {requer_confirmacao}")
print(f"   • campos_confirmar: {len(campos_confirmar)} campos")

if requer_confirmacao:
    print(f"\n⚠️  {len(campos_confirmar)} campo(s) para revisar:")
    for campo in campos_confirmar[:5]:  # Mostrar primeiros 5
        status = "DUPLICADO" if campo['duplicado'] else f"{campo['confianca']:.0%}"
        print(f"   • '{campo['coluna_original']}' → '{campo['tipo_detectado']}' ({status})")

    if len(campos_confirmar) > 5:
        print(f"   ... e mais {len(campos_confirmar) - 5}")

# ═══════════════════════════════════════════════════════════════════
# SALVAR ESTADO PARA PRÓXIMO BLOCO
# ═══════════════════════════════════════════════════════════════════

estado_bloco12 = {
    'timestamp': datetime.now().isoformat(),
    'total_campos': len(df.columns),
    'taxa_deteccao': taxa_sucesso,
    'requer_confirmacao': requer_confirmacao,
    'campos_confirmacao': campos_confirmar,
    'duplicacoes_detectadas': len(duplicados_conceituais),
    'mapeamentos': {
        col: {
            'rotulo': info['campo_detectado'],
            'confianca': info['confianca'],
            'metodo': info['metodo']
        }
        for col, info in mapeamento_campos.items()
    }
}

# Salvar JSON
arquivo_estado = fm.pastas['logs'] / '.bloco_12_state.json'
with open(arquivo_estado, 'w', encoding='utf-8') as f:
    json.dump(estado_bloco12, f, indent=2, ensure_ascii=False)

print(f"\n💾 Estado salvo: {arquivo_estado.name}")

print("\n" + "="*70)
print("✅ BLOCO 12 CONCLUÍDO")
print("="*70)

print(f"\n🎉 MUDANÇAS v7.0:")
print(f"   ✅ DataFrame preserva nomes ORIGINAIS")
print(f"   ✅ Mapeamentos salvos no dicionário")
print(f"   ✅ Path correto: fm.pastas['dicionarios']")
print(f"   ❌ Removido: Sistema de sufixos (_1, _2, _3)")
print(f"   ❌ Removido: Renomeação de colunas")

print(f"\n💡 Próximo: BLOCO 13 - Confirmação de tipos")


🔍 DETECÇÃO AUTOMÁTICA DE CAMPOS v7.0 (SEM RENOMEAR)

📊 Analisando 25 campos...

   🔍 Analisando: 'Ano civil/mês'
      ❓ DESCONHECIDO
      Confiança: 0% | Método: NENHUM

   🔍 Analisando: 'Centro'
      ✅ Centro
      Confiança: 100% | Método: DICIONARIO

   🔍 Analisando: ''
      ✅ Centro
      Confiança: 90% | Método: DICIONARIO

   🔍 Analisando: 'HierarqPrd'
      ❓ DESCONHECIDO
      Confiança: 0% | Método: NENHUM

   🔍 Analisando: 'Produto'
      ✅ Codigo_Produto
      Confiança: 90% | Método: DICIONARIO

   🔍 Analisando: '_dup1'
      ❓ DESCONHECIDO
      Confiança: 0% | Método: NENHUM

   🔍 Analisando: 'Estoque Inicial'
      ❓ DESCONHECIDO
      Confiança: 0% | Método: NENHUM

   🔍 Analisando: 'Entrada'
      ❓ DESCONHECIDO
      Confiança: 0% | Método: NENHUM

   🔍 Analisando: 'Variação Externa'
      ❓ DESCONHECIDO
      Confiança: 0% | Método: NENHUM

   🔍 Analisando: 'Variação Externa %'
      ❓ DESCONHECIDO
      Confiança: 0% | Método: NENHUM

   🔍 Analisando: 'Variação

In [44]:
# ===================================================================
# BLOCO 13 v7.3 - CONFIRMAÇÃO VISUAL COMPLETA (GUI 100% FUNCIONAL)
# ===================================================================
# Esta é a versão COMPLETA sem simplificações!
# TODO o código da GUI está incluído.
# ===================================================================

print("\n" + "="*70)
print("🔍 CONFIRMAÇÃO VISUAL DE TIPOS v7.3 (GUI COMPLETA)")
print("="*70)

import tkinter as tk
from tkinter import messagebox
from pathlib import Path
import json
from datetime import datetime

# ===================================================================
# 1. VALIDAÇÕES INICIAIS
# ===================================================================

variaveis_necessarias = [
    'requer_confirmacao',
    'campos_confirmar',
    'df',
    'DICIONARIO_PERSISTENTE',
    'fm'
]

for var in variaveis_necessarias:
    if var not in globals():
        print(f"\n❌ Erro: '{var}' não encontrado")
        raise RuntimeError(f"Variável '{var}' não disponível")

# ===================================================================
# 2. FALLBACK ROBUSTO (3 NÍVEIS)
# ===================================================================

if 'tipos_detectados' not in globals() or len(tipos_detectados) == 0:
    print(f"\n⚠️  Reconstruindo tipos_detectados...")

    tipos_detectados = {}

    # NÍVEL 1: .bloco_12_state.json
    arquivo_bloco12 = fm.pastas['logs'] / '.bloco_12_state.json'

    if arquivo_bloco12.exists():
        with open(arquivo_bloco12, 'r', encoding='utf-8') as f:
            estado_bloco12 = json.load(f)

        if 'mapeamentos' in estado_bloco12:
            for col_orig, info in estado_bloco12['mapeamentos'].items():
                tipos_detectados[col_orig] = {
                    'campo_detectado': info.get('rotulo', 'DESCONHECIDO'),
                    'confianca': info.get('confianca', 0.0),
                    'metodo': info.get('metodo', 'N/A')
                }

    # NÍVEL 2: campos_confirmar
    if len(tipos_detectados) == 0:
        for campo_dict in campos_confirmar:
            col = campo_dict['coluna_original']
            tipos_detectados[col] = {
                'campo_detectado': campo_dict['tipo_detectado'],
                'confianca': campo_dict['confianca'],
                'metodo': 'RECONSTRUIDO'
            }

    # Adicionar campos faltantes
    campos_faltando = []
    for campo_dict in campos_confirmar:
        col = campo_dict['coluna_original']
        if col not in tipos_detectados:
            tipos_detectados[col] = {
                'campo_detectado': campo_dict['tipo_detectado'],
                'confianca': campo_dict['confianca'],
                'metodo': 'RECONSTRUIDO'
            }
            campos_faltando.append(col)

    if campos_faltando:
        print(f"   ✅ {len(campos_faltando)} campos adicionados")

print(f"\n✅ Variáveis validadas:")
print(f"   • tipos_detectados: {len(tipos_detectados)} campos")
print(f"   • campos_confirmar: {len(campos_confirmar)} campos")

# ===================================================================
# 3. FUNÇÃO: CRIAR TIPO CUSTOMIZADO
# ===================================================================

def criar_tipo_customizado_popup():
    """Popup para criar tipo customizado."""
    resultado = {'nome': None, 'dtype': None}

    popup = tk.Toplevel()
    popup.title("CRIAR TIPO CUSTOMIZADO")
    popup.geometry("500x400")

    x = (popup.winfo_screenwidth() // 2) - 250
    y = (popup.winfo_screenheight() // 2) - 200
    popup.geometry(f"+{x}+{y}")
    popup.configure(bg='white')
    popup.transient()
    popup.grab_set()

    frame = tk.Frame(popup, bg='white', padx=20, pady=20)
    frame.pack(fill=tk.BOTH, expand=True)

    tk.Label(
        frame,
        text="CRIAR NOVO TIPO DE CAMPO",
        font=('Arial', 14, 'bold'),
        bg='white'
    ).pack(pady=(0, 20))

    tk.Label(
        frame,
        text="Nome do Tipo:",
        font=('Arial', 10),
        bg='white',
        anchor='w'
    ).pack(fill=tk.X)

    var_nome = tk.StringVar()
    entry_nome = tk.Entry(frame, textvariable=var_nome, font=('Arial', 12), width=40)
    entry_nome.pack(pady=(5, 20), fill=tk.X)
    entry_nome.focus()

    tk.Label(frame, text="Tipo de Dados:", font=('Arial', 10), bg='white', anchor='w').pack(fill=tk.X)

    dtypes = [
        ('int32', 'Inteiro pequeno'),
        ('int64', 'Inteiro grande'),
        ('float64', 'Decimal'),
        ('string', 'Texto'),
        ('date', 'Data')
    ]

    var_dtype = tk.StringVar(value='float64')

    for dtype, desc in dtypes:
        tk.Radiobutton(
            frame,
            text=f"{dtype:10s} - {desc}",
            variable=var_dtype,
            value=dtype,
            font=('Courier', 9),
            bg='white',
            anchor='w'
        ).pack(fill=tk.X, pady=2, padx=5)

    frame_btns = tk.Frame(frame, bg='white')
    frame_btns.pack(pady=(10, 0))

    def confirmar():
        nome = var_nome.get().strip().upper()
        if nome and len(nome) >= 2:
            resultado['nome'] = nome
            resultado['dtype'] = var_dtype.get()
            popup.destroy()
        else:
            messagebox.showwarning("Aviso", "Digite um nome válido!")

    def cancelar():
        popup.destroy()

    entry_nome.bind('<Return>', lambda e: confirmar())

    tk.Button(frame_btns, text="Criar", command=confirmar, width=12, bg='#4CAF50', fg='white').pack(side=tk.LEFT, padx=5)
    tk.Button(frame_btns, text="Cancelar", command=cancelar, width=12, bg='#757575', fg='white').pack(side=tk.LEFT, padx=5)

    popup.wait_window()
    return resultado['nome'], resultado['dtype']

# ===================================================================
# 4. PROCESSO DE CONFIRMAÇÃO
# ===================================================================

if not requer_confirmacao:
    print("\n✅ Nenhuma confirmação necessária")
    confirmacoes = {}
else:
    print(f"\n⚠️  {len(campos_confirmar)} campos requerem confirmação")
    print(f"   Abrindo interface visual...")

    def confirmar_tipos_visual(campos_confirmar, tipos_detectados, df):
        """GUI VISUAL COMPLETA."""

        # Testar GUI
        try:
            test_root = tk.Tk()
            test_root.withdraw()
            test_root.destroy()
        except:
            print("❌ GUI não disponível")
            return {}

        # Preparar tipos
        tipos_dict = DICIONARIO_PERSISTENTE.get('conhecimento_base', {}).get('campos_conhecidos', {})
        if not tipos_dict:
            tipos_dict = DICIONARIO_PERSISTENTE.get('campos_conhecidos', {})

        tipos_lista = []
        for i, (nome, info) in enumerate(sorted(tipos_dict.items()), 1):
            tipos_lista.append({
                'numero': i,
                'nome': nome,
                'descricao': info.get('descricao', '') if isinstance(info, dict) else ''
            })

        tipos_lista.append({'numero': 0, 'nome': 'DESCONHECIDO', 'descricao': ''})

        confirmacoes = {}
        idx_atual = [0]

        def processar_proximo():
            """Processa campo atual."""
            if idx_atual[0] >= len(campos_confirmar):
                return

            # Extrair campo
            col_dict = campos_confirmar[idx_atual[0]]
            col = col_dict['coluna_original']

            if col not in tipos_detectados:
                print(f"⚠️  '{col}' não encontrado - pulando")
                idx_atual[0] += 1
                processar_proximo()
                return

            info_campo = tipos_detectados[col]

            # Buscar coluna no DataFrame
            col_df = col if col in df.columns else None
            if not col_df:
                print(f"⚠️  '{col}' não encontrado no df - pulando")
                idx_atual[0] += 1
                processar_proximo()
                return

            try:
                valores = df[col_df].dropna().unique()[:5].tolist()
            except:
                valores = ['[ERRO]']

            # ========================================================
            # CRIAR JANELA
            # ========================================================
            root = tk.Tk()
            root.title(f"DETECTOR ({idx_atual[0]+1}/{len(campos_confirmar)})")
            root.geometry("900x700")

            x = (root.winfo_screenwidth() // 2) - 450
            y = (root.winfo_screenheight() // 2) - 350
            root.geometry(f"+{x}+{y}")

            frame_main = tk.Frame(root, bg='white')
            frame_main.pack(fill=tk.BOTH, expand=True, padx=20, pady=20)

            # TOPO
            frame_topo = tk.Frame(frame_main, bg='#E3F2FD', relief=tk.RAISED, borderwidth=2)
            frame_topo.pack(fill=tk.X, pady=(0, 15))

            tk.Label(
                frame_topo,
                text=f"Campo {idx_atual[0]+1} de {len(campos_confirmar)}",
                font=('Arial', 12, 'bold'),
                bg='#E3F2FD',
                fg='#1565C0'
            ).pack(pady=8)

            # CONTEÚDO
            frame_content = tk.Frame(frame_main, bg='white')
            frame_content.pack(fill=tk.BOTH, expand=True)

            # ESQUERDA
            frame_left = tk.Frame(frame_content, bg='white', width=400)
            frame_left.pack(side=tk.LEFT, fill=tk.BOTH, expand=False, padx=(0, 10))

            tk.Label(frame_left, text="CAMPO DO ARQUIVO", font=('Arial', 11, 'bold'), bg='white').pack(fill=tk.X, pady=(0, 10))

            frame_campo = tk.Frame(frame_left, bg='#FFF9C4', relief=tk.SUNKEN, borderwidth=2)
            frame_campo.pack(fill=tk.X, pady=(0, 10))

            tk.Label(frame_campo, text=col, font=('Arial', 10, 'bold'), bg='#FFF9C4', wraplength=380).pack(fill=tk.X, padx=10, pady=(8, 5))
            tk.Label(frame_campo, text=f"Detectado: {info_campo['campo_detectado']}", font=('Arial', 9), bg='#FFF9C4', fg='#F57F17').pack(fill=tk.X, padx=10, pady=(0, 3))
            tk.Label(frame_campo, text=f"Confiança: {info_campo['confianca']:.0%}", font=('Arial', 9), bg='#FFF9C4', fg='#F57F17').pack(fill=tk.X, padx=10, pady=(0, 8))

            tk.Label(frame_left, text="EXEMPLOS", font=('Arial', 10, 'bold'), bg='white').pack(fill=tk.X, pady=(5, 5))

            frame_ex = tk.Frame(frame_left, bg='#F5F5F5', relief=tk.SUNKEN, borderwidth=1)
            frame_ex.pack(fill=tk.X)

            for i, v in enumerate(valores, 1):
                tk.Label(frame_ex, text=f"{i}. {str(v)[:50]}", font=('Arial', 9), bg='#F5F5F5', anchor='w').pack(fill=tk.X, padx=10, pady=2)

            # DIREITA
            frame_right = tk.Frame(frame_content, bg='white')
            frame_right.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

            tk.Label(frame_right, text="TIPOS - Digite o número", font=('Arial', 11, 'bold'), bg='white').pack(fill=tk.X, pady=(0, 10))

            frame_lista_outer = tk.Frame(frame_right, bg='white')
            frame_lista_outer.pack(fill=tk.BOTH, expand=True)

            canvas = tk.Canvas(frame_lista_outer, bg='white', highlightthickness=0)
            scrollbar = tk.Scrollbar(frame_lista_outer, orient="vertical", command=canvas.yview)
            frame_lista = tk.Frame(canvas, bg='white')

            frame_lista.bind("<Configure>", lambda e: canvas.configure(scrollregion=canvas.bbox("all")))
            canvas.create_window((0, 0), window=frame_lista, anchor="nw")
            canvas.configure(yscrollcommand=scrollbar.set)

            canvas.pack(side="left", fill="both", expand=True)
            scrollbar.pack(side="right", fill="y")

            # Popular lista
            for t in tipos_lista:
                num = t['numero']
                nome = t['nome']
                desc = t['descricao']

                bg = '#E8F5E9' if nome == info_campo['campo_detectado'] else 'white'
                fg = '#2E7D32' if nome == info_campo['campo_detectado'] else 'black'

                fi = tk.Frame(frame_lista, bg=bg, relief=tk.GROOVE, borderwidth=1)
                fi.pack(fill=tk.X, pady=2, padx=5)

                tk.Label(fi, text=f"[{num:2d}]  {nome}", font=('Courier', 9, 'bold'), bg=bg, fg=fg, anchor='w').pack(fill=tk.X, padx=8, pady=(3, 0))

                if desc:
                    tk.Label(fi, text=f"      {desc}", font=('Arial', 8), bg=bg, fg='#666', anchor='w').pack(fill=tk.X, padx=8, pady=(0, 3))

            # RODAPÉ
            frame_footer = tk.Frame(frame_main, bg='white')
            frame_footer.pack(fill=tk.X, pady=(15, 0))

            tk.Frame(frame_footer, height=2, bg='#CCC').pack(fill=tk.X, pady=(0, 10))

            frame_input = tk.Frame(frame_footer, bg='white')
            frame_input.pack(pady=(0, 10))

            tk.Label(frame_input, text="Digite o número:", font=('Arial', 10, 'bold'), bg='white').pack(side=tk.LEFT, padx=(0, 10))

            var_num = tk.StringVar()
            entry = tk.Entry(frame_input, textvariable=var_num, font=('Arial', 12, 'bold'), width=8, justify='center')
            entry.pack(side=tk.LEFT)
            entry.focus()

            label_err = tk.Label(frame_input, text="", font=('Arial', 9), fg='#FF0000', bg='white')
            label_err.pack(side=tk.LEFT, padx=(10, 0))

            frame_btns = tk.Frame(frame_footer, bg='white')
            frame_btns.pack()

            def validar():
                try:
                    num = int(var_num.get().strip())
                    tipo = None
                    for t in tipos_lista:
                        if t['numero'] == num:
                            tipo = t['nome']
                            break

                    if tipo:
                        confirmacoes[col] = tipo
                        idx_atual[0] += 1
                        root.destroy()
                        processar_proximo()
                    else:
                        label_err.config(text=f"Número {num} inválido!")
                except ValueError:
                    label_err.config(text="Digite um número!")

            def manter():
                confirmacoes[col] = info_campo['campo_detectado']
                idx_atual[0] += 1
                root.destroy()
                processar_proximo()

            def pular():
                for c_dict in campos_confirmar[idx_atual[0]:]:
                    c = c_dict['coluna_original']
                    if c in tipos_detectados:
                        confirmacoes[c] = tipos_detectados[c]['campo_detectado']
                root.destroy()

            def criar():
                nome, dtype = criar_tipo_customizado_popup()

                if nome and dtype:
                    # Adicionar ao dicionário
                    if 'conhecimento_base' not in DICIONARIO_PERSISTENTE:
                        DICIONARIO_PERSISTENTE['conhecimento_base'] = {}
                    if 'campos_conhecidos' not in DICIONARIO_PERSISTENTE['conhecimento_base']:
                        DICIONARIO_PERSISTENTE['conhecimento_base']['campos_conhecidos'] = {}

                    DICIONARIO_PERSISTENTE['conhecimento_base']['campos_conhecidos'][nome] = {
                        'tipo': dtype,
                        'descricao': f'Customizado ({dtype})',
                        'categoria': 'CUSTOMIZADO'
                    }

                    # Salvar
                    dict_path = fm.pastas['dicionarios'] / 'DICT_Dicionario_Persistente.json'
                    with open(dict_path, 'w', encoding='utf-8') as f:
                        json.dump(DICIONARIO_PERSISTENTE, f, indent=2, ensure_ascii=False)

                    print(f"✅ Tipo '{nome}' criado ({dtype})")

                    # Adicionar na lista
                    prox_num = max([t['numero'] for t in tipos_lista]) + 1
                    tipos_lista.insert(-1, {'numero': prox_num, 'nome': nome, 'descricao': f'Customizado - {dtype}'})

                    # Atualizar GUI
                    fi = tk.Frame(frame_lista, bg='#E1F5FE', relief=tk.GROOVE, borderwidth=2)
                    fi.pack(fill=tk.X, pady=2, padx=5)

                    tk.Label(fi, text=f"[{prox_num:2d}]  {nome} ⭐", font=('Courier', 9, 'bold'), bg='#E1F5FE', fg='#01579B', anchor='w').pack(fill=tk.X, padx=8, pady=(3, 0))
                    tk.Label(fi, text=f"      Customizado - {dtype}", font=('Arial', 8), bg='#E1F5FE', fg='#0277BD', anchor='w').pack(fill=tk.X, padx=8, pady=(0, 3))

                    canvas.update_idletasks()
                    canvas.yview_moveto(1.0)

                    var_num.set(str(prox_num))
                    entry.focus()

                    messagebox.showinfo("Sucesso", f"Tipo '{nome}' criado!\nDigite {prox_num} e confirme.")

            entry.bind('<Return>', lambda e: validar())

            tk.Button(frame_btns, text="Confirmar", command=validar, width=12, height=2, bg='#4CAF50', fg='white', font=('Arial', 10, 'bold')).pack(side=tk.LEFT, padx=5)
            tk.Button(frame_btns, text="Manter", command=manter, width=12, height=2, bg='#FF9800', fg='white', font=('Arial', 10)).pack(side=tk.LEFT, padx=5)
            tk.Button(frame_btns, text="✨ Criar Tipo", command=criar, width=12, height=2, bg='#2196F3', fg='white', font=('Arial', 10, 'bold')).pack(side=tk.LEFT, padx=5)
            tk.Button(frame_btns, text="Pular Todos", command=pular, width=12, height=2, bg='#757575', fg='white', font=('Arial', 10)).pack(side=tk.LEFT, padx=5)

            root.mainloop()

        processar_proximo()
        return confirmacoes

    confirmacoes = confirmar_tipos_visual(campos_confirmar, tipos_detectados, df)

    # Aplicar confirmações
    if confirmacoes:
        print(f"\n✅ Confirmações: {len(confirmacoes)}")

        for col, tipo in confirmacoes.items():
            if col in tipos_detectados:
                tipos_detectados[col]['campo_detectado'] = tipo
                tipos_detectados[col]['confianca'] = 1.0
                tipos_detectados[col]['metodo'] = 'CONFIRMACAO_USUARIO'

        # Salvar no dicionário
        nome_fonte = f"CSV_{arquivo_selecionado.stem}"

        if 'arquivos' not in DICIONARIO_PERSISTENTE:
            DICIONARIO_PERSISTENTE['arquivos'] = {}

        if nome_fonte not in DICIONARIO_PERSISTENTE['arquivos']:
            DICIONARIO_PERSISTENTE['arquivos'][nome_fonte] = {
                'arquivo_origem': arquivo_selecionado.name,
                'mapeamentos': {}
            }

        for col, tipo in confirmacoes.items():
            DICIONARIO_PERSISTENTE['arquivos'][nome_fonte]['mapeamentos'][col] = {
                'rotulo_padrao': tipo,
                'confianca': 1.0,
                'metodo': 'CONFIRMACAO_USUARIO',
                'timestamp': datetime.now().isoformat()
            }

        dict_path = fm.pastas['dicionarios'] / 'DICT_Dicionario_Persistente.json'
        with open(dict_path, 'w', encoding='utf-8') as f:
            json.dump(DICIONARIO_PERSISTENTE, f, indent=2, ensure_ascii=False)

        print(f"   ✅ Dicionário atualizado")
    else:
        print(f"\n⚠️  Nenhuma confirmação")

# Salvar LOG
log_path = Path.home() / '.processador_dicionario_localizador.json'
if log_path.exists():
    with open(log_path, 'r', encoding='utf-8') as f:
        log_data = json.load(f)
else:
    log_data = {}

log_data['bloco_13_state'] = {
    'timestamp': datetime.now().isoformat(),
    'confirmacoes': len(confirmacoes),
    'dataframe_renomeado': False
}

with open(log_path, 'w', encoding='utf-8') as f:
    json.dump(log_data, f, indent=2, ensure_ascii=False)

arquivo_sep = fm.pastas['logs'] / '.bloco_13_state.json'
with open(arquivo_sep, 'w', encoding='utf-8') as f:
    json.dump(log_data['bloco_13_state'], f, indent=2, ensure_ascii=False)

print(f"\n✅ LOG salvo")
print("\n" + "="*70)
print("✅ BLOCO 13 CONCLUÍDO v7.3")
print("="*70)
print(f"\n💡 DataFrame mantém nomes ORIGINAIS")
print(f"💡 Mapeamentos salvos no dicionário")


🔍 CONFIRMAÇÃO VISUAL DE TIPOS v7.3 (GUI COMPLETA)

✅ Variáveis validadas:
   • tipos_detectados: 25 campos
   • campos_confirmar: 12 campos

⚠️  12 campos requerem confirmação
   Abrindo interface visual...
⚠️  '' não encontrado no df - pulando

✅ Confirmações: 11
   ✅ Dicionário atualizado

✅ LOG salvo

✅ BLOCO 13 CONCLUÍDO v7.3

💡 DataFrame mantém nomes ORIGINAIS
💡 Mapeamentos salvos no dicionário


In [45]:
# ═══════════════════════════════════════════════════════════════════
# BLOCO 14: VALIDAÇÕES E ESTATÍSTICAS v2.1
# ═══════════════════════════════════════════════════════════════════
# Mudanças v2.0:
# - CORRIGIDO: df_limpo → df (variável correta do BLOCO 13)
# - ADICIONADO: Salvamento .bloco_14_state.json
# - ADICIONADO: Salvamento estatisticas_validacao.json
# - CORRIGIDO: Visual sem emojis (≤70 caracteres)
# - REMOVIDO: Preview duplicado (BLOCO 13 já fez)
# - ADICIONADO: Leitura do LOG GLOBAL
# Mudanças v2.1:
# - CORRIGIDO: fm.container → fm.base_path.name
# - CORRIGIDO: fm.timestamp → timestamp_execucao (do LOG)
# - CORRIGIDO: fm.salvar_json() → salvamento manual
# - ADICIONADO: Recria FileManager se não estiver na memória
# ═══════════════════════════════════════════════════════════════════

import json
from pathlib import Path

print("\n" + "="*70)
print("VALIDACOES E ESTATISTICAS v2.1")
print("="*70)

# ═══════════════════════════════════════════════════════════════════
# 1. VALIDACAO DE DEPENDENCIAS
# ═══════════════════════════════════════════════════════════════════

print("\n" + "-"*70)
print("ETAPA 1: VALIDANDO DEPENDENCIAS")
print("-"*70)

# 1.1 Verificar LOG GLOBAL e carregar config
log_global_path = Path.home() / '.processador_dicionario_localizador.json'

if not log_global_path.exists():
    print("\n! Erro: LOG GLOBAL nao encontrado")
    print("  Execute BLOCO 1 antes deste bloco")
    raise RuntimeError("LOG GLOBAL nao disponivel")

with open(log_global_path, 'r', encoding='utf-8') as f:
    log_global = json.load(f)

timestamp_execucao = log_global['timestamp']
pasta_container = Path(log_global['pasta_base_atual'])

# Verificar se fm existe (pode estar na memória ou não)
if 'fm' not in globals():
    # Recriar FileManager se não estiver na memória
    class FileManagerInterativo:
        def __init__(self, base_path):
            self.base_path = Path(base_path)
            self.pastas = {
                'entrada': self.base_path / '01_Entrada',
                'processados': self.base_path / '02_Processados',
                'outputs': self.base_path / '03_Outputs',
                'logs': self.base_path / '04_Logs',
                'dicionarios': self.base_path / '05_Dicionarios',
                'codigos': self.base_path / '06_Codigos_Integracao'
            }

    fm = FileManagerInterativo(pasta_container)

print(f"\n OK LOG GLOBAL conectado")
print(f"    Container: {fm.base_path.name}")
print(f"    Timestamp: {timestamp_execucao}")

# 1.2 Verificar variáveis necessárias
if 'df' not in globals():
    print("\n! Erro: df nao encontrado")
    print("  Execute BLOCO 13 antes deste bloco")
    raise RuntimeError("df nao disponivel")

if 'tipos_detectados' not in globals():
    print("\n! Erro: tipos_detectados nao encontrado")
    print("  Execute BLOCO 12 antes deste bloco")
    raise RuntimeError("tipos_detectados nao disponivel")

print(f"\n OK Variaveis validadas:")
print(f"    df shape: {df.shape}")
print(f"    tipos_detectados: {len(tipos_detectados)} campos")

# ═══════════════════════════════════════════════════════════════════
# 2. ESTATISTICAS GERAIS
# ═══════════════════════════════════════════════════════════════════

print("\n" + "-"*70)
print("ETAPA 2: RESUMO GERAL")
print("-"*70)

registros = len(df)
colunas = len(df.columns)
mem_mb = df.memory_usage(deep=True).sum() / 1024**2
duplicadas = df.duplicated().sum()

print(f"\n   Registros finais: {registros:,}")
print(f"   Colunas finais: {colunas}")
print(f"   Memoria em uso: {mem_mb:.2f} MB")
print(f"   Linhas duplicadas: {duplicadas:,}")

estatisticas = {
    'resumo_geral': {
        'registros': int(registros),
        'colunas': int(colunas),
        'memoria_mb': float(round(mem_mb, 2)),
        'linhas_duplicadas': int(duplicadas)
    }
}

# ═══════════════════════════════════════════════════════════════════
# 3. ANALISE DE VALORES NULOS
# ═══════════════════════════════════════════════════════════════════

print("\n" + "-"*70)
print("ETAPA 3: ANALISE DE VALORES NULOS")
print("-"*70)

nulos_por_col = df.isnull().sum()
colunas_com_nulos = nulos_por_col[nulos_por_col > 0].sort_values(
    ascending=False
)

if len(colunas_com_nulos) > 0:
    print(f"\n   {len(colunas_com_nulos)} colunas com valores nulos:")

    nulos_detalhes = {}
    for col, qtd in colunas_com_nulos.items():
        pct = (qtd / len(df)) * 100
        barra = "#" * int(pct / 5)
        print(f"      {col[:30]:30s} | {qtd:6,} ({pct:5.1f}%) {barra}")
        nulos_detalhes[col] = {'qtd': int(qtd), 'pct': float(round(pct, 1))}

    estatisticas['valores_nulos'] = nulos_detalhes
else:
    print(f"\n   Nenhum valor nulo!")
    estatisticas['valores_nulos'] = {}

# ═══════════════════════════════════════════════════════════════════
# 4. DISTRIBUICAO DE TIPOS DETECTADOS
# ═══════════════════════════════════════════════════════════════════

print("\n" + "-"*70)
print("ETAPA 4: DISTRIBUICAO DE TIPOS DETECTADOS")
print("-"*70)

tipos_resumo = {}
for info in tipos_detectados.values():
    tipo = info['campo_detectado']
    tipos_resumo[tipo] = tipos_resumo.get(tipo, 0) + 1

for tipo, count in sorted(tipos_resumo.items(), key=lambda x: x[1], reverse=True):
    barra = "#" * (count * 2)
    print(f"   {tipo:25s}: {count:2d} colunas {barra}")

estatisticas['tipos_detectados'] = tipos_resumo

# ═══════════════════════════════════════════════════════════════════
# 5. CONFIANCA NA DETECCAO
# ═══════════════════════════════════════════════════════════════════

print("\n" + "-"*70)
print("ETAPA 5: CONFIANCA NA DETECCAO")
print("-"*70)

alta = sum(1 for i in tipos_detectados.values() if i['confianca'] >= 0.90)
media = sum(1 for i in tipos_detectados.values() if 0.70 <= i['confianca'] < 0.90)
baixa = sum(1 for i in tipos_detectados.values() if i['confianca'] < 0.70)

print(f"\n   Alta (>=90%):   {alta:2d} colunas")
print(f"   Media (70-90%): {media:2d} colunas")
print(f"   Baixa (<70%):   {baixa:2d} colunas")

estatisticas['confianca_deteccao'] = {
    'alta': int(alta),
    'media': int(media),
    'baixa': int(baixa)
}

# ═══════════════════════════════════════════════════════════════════
# 6. CAMPOS AMBIGUOS
# ═══════════════════════════════════════════════════════════════════

print("\n" + "-"*70)
print("ETAPA 6: CAMPOS AMBIGUOS")
print("-"*70)

ambiguos = {
    col: info
    for col, info in tipos_detectados.items()
    if info.get('ambiguidade', False)
}

if ambiguos:
    print(f"\n   {len(ambiguos)} campos com ambiguidade:")

    ambiguos_lista = []
    for col, info in list(ambiguos.items())[:5]:
        print(f"\n   {col}")
        print(f"      Detectado: {info['campo_detectado']}")
        candidatos = info.get('candidatos', [])
        if candidatos:
            print(f"      Similar a: {', '.join(candidatos)}")

        ambiguos_lista.append({
            'campo_original': col,
            'detectado': info['campo_detectado'],
            'candidatos': candidatos
        })

    if len(ambiguos) > 5:
        print(f"\n   ... e mais {len(ambiguos) - 5} campos")

    estatisticas['campos_ambiguos'] = ambiguos_lista
else:
    print(f"\n   Nenhum campo ambiguo!")
    estatisticas['campos_ambiguos'] = []

# ═══════════════════════════════════════════════════════════════════
# 7. ANALISE DE UNICIDADE (POTENCIAIS IDS)
# ═══════════════════════════════════════════════════════════════════

print("\n" + "-"*70)
print("ETAPA 7: ANALISE DE UNICIDADE")
print("-"*70)

unicidade_detalhes = []

for col in df.columns:
    unicos = df[col].nunique()
    total = len(df)
    pct_unico = (unicos / total) * 100

    if pct_unico == 100:
        print(f"   {col[:30]:30s} | 100% unico (potencial ID)")
        unicidade_detalhes.append({
            'campo': col,
            'pct_unico': 100.0,
            'tipo': 'ID'
        })
    elif pct_unico >= 95:
        print(f"   {col[:30]:30s} | {pct_unico:5.1f}% unico")
        unicidade_detalhes.append({
            'campo': col,
            'pct_unico': float(round(pct_unico, 1)),
            'tipo': 'QUASE_ID'
        })

if not unicidade_detalhes:
    print(f"\n   Nenhum campo com alta unicidade")

estatisticas['unicidade'] = unicidade_detalhes

# ═══════════════════════════════════════════════════════════════════
# 8. CARDINALIDADE (VALORES UNICOS)
# ═══════════════════════════════════════════════════════════════════

print("\n" + "-"*70)
print("ETAPA 8: CARDINALIDADE")
print("-"*70)

cardinalidade_detalhes = []

for col in df.columns[:10]:
    unicos = df[col].nunique()
    total = len(df)
    pct = (unicos / total) * 100

    if pct <= 10:
        categoria = "Categoria (baixa)"
    elif pct <= 50:
        categoria = "Mista (media)"
    else:
        categoria = "Continua (alta)"

    print(f"   {col[:30]:30s} | {unicos:4d} unicos ({pct:5.1f}%) - {categoria}")

    cardinalidade_detalhes.append({
        'campo': col,
        'unicos': int(unicos),
        'pct': float(round(pct, 1)),
        'categoria': categoria
    })

if len(df.columns) > 10:
    print(f"\n   ... e mais {len(df.columns) - 10} colunas")

estatisticas['cardinalidade'] = cardinalidade_detalhes

# ═══════════════════════════════════════════════════════════════════
# 9. TIPOS DE DADOS PANDAS
# ═══════════════════════════════════════════════════════════════════

print("\n" + "-"*70)
print("ETAPA 9: TIPOS DE DADOS (PANDAS)")
print("-"*70)

dtype_counts = df.dtypes.value_counts()
tipos_pandas = {}

for dtype, count in dtype_counts.items():
    dtype_str = str(dtype)
    print(f"   {dtype_str:15s}: {count:2d} colunas")
    tipos_pandas[dtype_str] = int(count)

estatisticas['tipos_pandas'] = tipos_pandas

# ═══════════════════════════════════════════════════════════════════
# 10. SALVAMENTO DE ESTADO E ESTATISTICAS
# ═══════════════════════════════════════════════════════════════════

print("\n" + "-"*70)
print("ETAPA 10: SALVAMENTO DE ESTADO")
print("-"*70)

# 10.1 Salvar estado do bloco
estado = {
    'bloco': 14,
    'versao': '2.1',
    'timestamp': timestamp_execucao,
    'container': fm.base_path.name,
    'validado': True,
    'df_shape': list(df.shape),
    'campos_analisados': len(tipos_detectados)
}

estado_path = Path('.bloco_14_state.json')
with open(estado_path, 'w', encoding='utf-8') as f:
    json.dump(estado, f, indent=2, ensure_ascii=False)

print(f"\n   Estado salvo: {estado_path}")

# 10.2 Salvar estatísticas completas para auditoria
estatisticas_path = fm.pastas['logs'] / 'estatisticas_validacao.json'
with open(estatisticas_path, 'w', encoding='utf-8') as f:
    json.dump(estatisticas, f, indent=2, ensure_ascii=False)

print(f"   Estatisticas salvas: {estatisticas_path.name}")

# ═══════════════════════════════════════════════════════════════════
# 11. RESUMO FINAL
# ═══════════════════════════════════════════════════════════════════

print("\n" + "="*70)
print("VALIDACOES CONCLUIDAS")
print("="*70)

print(f"\n   Registros: {registros:,}")
print(f"   Colunas: {colunas}")
print(f"   Taxa deteccao: {(alta + media) / len(tipos_detectados) * 100:.1f}%")
print(f"   Valores nulos: {len(colunas_com_nulos)} colunas")
print(f"   Campos ambiguos: {len(ambiguos)}")

print("\n Proximo: BLOCO 15 - Exportacao Final")

print("\n" + "="*70)


VALIDACOES E ESTATISTICAS v2.1

----------------------------------------------------------------------
ETAPA 1: VALIDANDO DEPENDENCIAS
----------------------------------------------------------------------

 OK LOG GLOBAL conectado
    Container: PROCESSAR_ARQUIVOS_20251019_060722
    Timestamp: 20251019_060722

 OK Variaveis validadas:
    df shape: (967, 25)
    tipos_detectados: 25 campos

----------------------------------------------------------------------
ETAPA 2: RESUMO GERAL
----------------------------------------------------------------------

   Registros finais: 967
   Colunas finais: 25
   Memoria em uso: 1.27 MB
   Linhas duplicadas: 767

----------------------------------------------------------------------
ETAPA 3: ANALISE DE VALORES NULOS
----------------------------------------------------------------------

   Nenhum valor nulo!

----------------------------------------------------------------------
ETAPA 4: DISTRIBUICAO DE TIPOS DETECTADOS
------------------------

In [46]:
# ======================================================================
# BLOCO 15 v2.3 - EXPORTACAO DE RESULTADOS (100% LOG + ROBUSTO)
# ======================================================================
# VERSAO: 2.3 - 0% memoria, 100% LOG, validacao FileManager
# AUTOR: Sistema Automacao AIVI
# DATA: 2025-10-19
# ======================================================================
# CORRECOES v2.3:
# + Verifica se fm.salvar() existe antes de usar
# + Recria FileManager se metodo nao existir
# + FileManagerSimples com timestamp correto
# ======================================================================
# CORRECOES v2.2:
# + Validacao de None em arquivo_selecionado antes de acessar .name
# + Validacao de Path antes de usar .stem
# + Verificacao de temp_arquivo antes de atribuir
# ======================================================================

print("\n" + "="*70)
print("EXPORTACAO DE RESULTADOS")
print("="*70)

# ======================================================================
# ETAPA 1: CONECTAR COM BLOCOS ANTERIORES
# ======================================================================

from pathlib import Path
import json
import pandas as pd
from datetime import datetime

print("\n" + "-"*70)
print("ETAPA 1: CONECTANDO COM BLOCOS ANTERIORES...")
print("-"*70)

# Ler LOG GLOBAL
log_global_path = Path.home() / '.processador_dicionario_localizador.json'
if not log_global_path.exists():
    raise FileNotFoundError("LOG GLOBAL nao encontrado!")

with open(log_global_path, 'r', encoding='utf-8') as f:
    log_global = json.load(f)

timestamp_execucao = log_global['timestamp']
container_nome = f"PROCESSAR_ARQUIVOS_{timestamp_execucao}"
pasta_base = Path.home() / 'PYTHON_AIVI' / container_nome

print(f" OK LOG GLOBAL conectado")
print(f"    Container: {container_nome}")
print(f"    Timestamp: {timestamp_execucao}")

# Verificar e corrigir FileManager
class FileManagerSimples:
    """FileManager simplificado para exportacao"""
    def __init__(self, base_path, timestamp=None):
        self.base_path = Path(base_path)
        self.timestamp = timestamp or datetime.now().strftime('%Y%m%d_%H%M%S')
        self.pastas = {
            'processados': self.base_path / '02_Processados',
            'outputs': self.base_path / '03_Outputs',
            'logs': self.base_path / '04_Logs',
            'codigos_integracao': self.base_path / '06_Codigos_Integracao'
        }
        # Criar pastas se nao existem
        for pasta in self.pastas.values():
            pasta.mkdir(parents=True, exist_ok=True)

    def salvar(self, df, nome, tipo='xlsx', pasta='outputs'):
        arquivo = self.pastas[pasta] / f"{nome}_{self.timestamp}.{tipo}"

        if tipo == 'xlsx':
            df.to_excel(arquivo, index=False, engine='openpyxl')
        elif tipo == 'csv':
            df.to_csv(arquivo, index=False, encoding='utf-8-sig')

        return arquivo

# Verificar se fm existe e tem metodo salvar
if 'fm' not in globals():
    print(f" !! Criando FileManager...")
    fm = FileManagerSimples(base_path=pasta_base, timestamp=timestamp_execucao)
    print(f" OK FileManager criado: {fm.base_path.name}")
elif not hasattr(fm, 'salvar'):
    print(f" !! FileManager sem metodo salvar - recriando...")
    fm = FileManagerSimples(base_path=pasta_base, timestamp=timestamp_execucao)
    print(f" OK FileManager recriado: {fm.base_path.name}")
else:
    print(f" OK FileManager existe na memoria")

# Validar que df existe
if 'df' not in globals():
    raise NameError("Variavel 'df' nao encontrada! Execute BLOCOS 1-13 primeiro.")

print(f" OK DataFrame validado: {df.shape}")

# Ler estado do BLOCO 13 (se existe)
bloco_13_state_path = pasta_base / '.bloco_13_state.json'
bloco_12_state_path = pasta_base / '.bloco_12_state.json'

if bloco_13_state_path.exists():
    with open(bloco_13_state_path, 'r', encoding='utf-8') as f:
        bloco_anterior_state = json.load(f)
    bloco_anterior_nome = "BLOCO 13"
    print(f" OK Estado carregado do {bloco_anterior_nome}")
elif bloco_12_state_path.exists():
    with open(bloco_12_state_path, 'r', encoding='utf-8') as f:
        bloco_anterior_state = json.load(f)
    bloco_anterior_nome = "BLOCO 12"
    print(f" OK Estado carregado do {bloco_anterior_nome}")
else:
    # Nenhum estado salvo - usar valores em memoria
    bloco_anterior_state = {}
    bloco_anterior_nome = "MEMORIA"
    print(f" !! Nenhum estado salvo - usando variaveis em memoria")

# Ler arquivo selecionado (tentar varias fontes)
arquivo_selecionado = None
sheet_nome = "N/A"

# Fonte 1: LOG do BLOCO 4
bloco_4_state_path = pasta_base / '.ultimo_arquivo.json'
if bloco_4_state_path.exists():
    with open(bloco_4_state_path, 'r', encoding='utf-8') as f:
        arquivo_info = json.load(f)
    arquivo_selecionado = Path(arquivo_info['caminho'])
    sheet_nome = arquivo_info.get('sheet', 'N/A')
    print(f" OK Arquivo do LOG BLOCO 4: {arquivo_selecionado.name}")

# Fonte 2: Variavel global arquivo_selecionado
if arquivo_selecionado is None and 'arquivo_selecionado' in globals():
    temp_arquivo = globals()['arquivo_selecionado']
    if temp_arquivo is not None:
        arquivo_selecionado = temp_arquivo
        print(f" OK Arquivo da memoria: {arquivo_selecionado.name}")
        if 'sheet_nome' in globals():
            sheet_nome = globals()['sheet_nome']

# Fonte 3: LOG GLOBAL (pode ter info do arquivo)
if arquivo_selecionado is None and 'ultimo_arquivo' in log_global:
    temp_arquivo = log_global['ultimo_arquivo']
    if temp_arquivo:
        arquivo_selecionado = Path(temp_arquivo)
        print(f" OK Arquivo do LOG GLOBAL: {arquivo_selecionado.name}")

# Fallback final
if arquivo_selecionado is None:
    arquivo_selecionado = Path("arquivo_processado")
    print(f" !! Arquivo nao encontrado - usando nome generico")

print(f"\n OK Arquivo selecionado: {arquivo_selecionado.name}")
print(f" OK Sheet: {sheet_nome}")

# ======================================================================
# ETAPA 2: EXPORTAR DADOS LIMPOS
# ======================================================================

print("\n" + "-"*70)
print("ETAPA 2: EXPORTANDO DADOS LIMPOS")
print("-"*70)

# Garantir que arquivo_selecionado e Path valido
if not isinstance(arquivo_selecionado, Path):
    arquivo_selecionado = Path(arquivo_selecionado)

nome_base = arquivo_selecionado.stem

# 1. Dados limpos e mapeados
arquivo_limpo = fm.salvar(
    df,
    f"{nome_base}_Limpo",
    tipo='xlsx',
    pasta='processados'
)
print(f" OK {arquivo_limpo.name}")

# ======================================================================
# ETAPA 3: DICIONARIO DE CAMPOS
# ======================================================================

print("\n" + "-"*70)
print("ETAPA 3: CRIANDO DICIONARIO DE CAMPOS")
print("-"*70)

# Ler tipos_detectados (tentar varias fontes)
tipo_info_dict = {}

# Fonte 1: Variavel global tipos_detectados
if 'tipos_detectados' in globals():
    tipo_info_dict = tipos_detectados
    print(f" OK tipos_detectados da memoria: {len(tipo_info_dict)} campos")

# Fonte 2: Estado do BLOCO 13
elif bloco_anterior_state and 'confirmacoes' in bloco_anterior_state:
    # Se BLOCO 13 salvou confirmacoes, usar
    confirmacoes = bloco_anterior_state['confirmacoes']
    for col_orig, campo_confirmado in confirmacoes.items():
        tipo_info_dict[col_orig] = {
            'campo_detectado': campo_confirmado,
            'confianca': 1.0,
            'metodo': 'CONFIRMADO_USUARIO'
        }
    print(f" OK tipos_detectados do BLOCO 13: {len(tipo_info_dict)} campos")

# Fonte 3: Tentar reconstruir baseado nas colunas do df
else:
    for col in df.columns:
        tipo_info_dict[col] = {
            'campo_detectado': col,
            'confianca': 1.0,
            'metodo': 'INFERIDO_DO_DF'
        }
    print(f" !! tipos_detectados inferido do df: {len(tipo_info_dict)} campos")

registros_dict = []

for col in df.columns:
    tipo_info = tipo_info_dict.get(col, {})
    valores_exemplo = df[col].dropna().unique()[:3].tolist()

    registros_dict.append({
        'Coluna': col,
        'Tipo_Detectado': tipo_info.get('campo_detectado', 'DESCONHECIDO'),
        'Confianca_%': tipo_info.get('confianca', 0.0) * 100,
        'Score_Conteudo_%': tipo_info.get('score_conteudo', 0.0) * 100,
        'Score_Nome_%': tipo_info.get('score_nome', 0.0) * 100,
        'Metodo': tipo_info.get('metodo', 'N/A'),
        'Ambiguidade': 'Sim' if tipo_info.get('ambiguidade') else 'Nao',
        'Dtype_Pandas': str(df[col].dtype),
        'Valores_Unicos': df[col].nunique(),
        'Nulos_Qtd': df[col].isna().sum(),
        'Nulos_%': (df[col].isna().sum() / len(df)) * 100,
        'Exemplo_1': str(valores_exemplo[0]) if len(valores_exemplo) > 0 else None,
        'Exemplo_2': str(valores_exemplo[1]) if len(valores_exemplo) > 1 else None,
        'Exemplo_3': str(valores_exemplo[2]) if len(valores_exemplo) > 2 else None
    })

df_dict = pd.DataFrame(registros_dict)
arquivo_dict = fm.salvar(
    df_dict,
    f"DICT_{nome_base}",
    tipo='xlsx',
    pasta='outputs'
)
print(f" OK {arquivo_dict.name}")

# ======================================================================
# ETAPA 4: LOG DE PROCESSAMENTO
# ======================================================================

print("\n" + "-"*70)
print("ETAPA 4: CRIANDO LOG DE PROCESSAMENTO")
print("-"*70)

# Ler configuracoes de carregamento (multiplas fontes)
linha_cab_ini = 1
linha_cab_fim = 1
col_ini = 1
col_fim = len(df.columns)
linha_dados_ini = 2

# Fonte 1: Estado do BLOCO 9
bloco_9_state_path = pasta_base / '.bloco_9_state.json'
if bloco_9_state_path.exists():
    try:
        with open(bloco_9_state_path, 'r', encoding='utf-8') as f:
            bloco_9_state = json.load(f)

        config_carga = bloco_9_state.get('config_carga', {})
        linha_cab_ini = config_carga.get('linha_cabecalho_inicio_excel', 1)
        linha_cab_fim = config_carga.get('linha_cabecalho_fim_excel', 1)
        col_ini = config_carga.get('coluna_inicio_excel', 1)
        col_fim = config_carga.get('coluna_fim_excel', len(df.columns))
        linha_dados_ini = config_carga.get('linha_dados_inicio_excel', 2)
        print(f" OK Config carga do BLOCO 9")
    except:
        print(f" !! Erro ao ler BLOCO 9 - usando valores padrao")
else:
    print(f" !! BLOCO 9 state nao encontrado - usando valores padrao")

# Fonte 2: Variaveis globais (fallback)
if 'config' in globals():
    config = globals()['config']
    linha_cab_ini = config.get('linha_cabecalho_inicio', linha_cab_ini)
    linha_dados_ini = config.get('linha_dados_inicio', linha_dados_ini)
    print(f" OK Config da memoria (variavel global)")

log_processamento = {
    'Arquivo_Original': arquivo_selecionado.name,
    'Caminho_Original': str(arquivo_selecionado),
    'Sheet_Processada': sheet_nome,

    # Cabecalho
    'Linha_Cabecalho_Inicio_Excel': linha_cab_ini,
    'Linha_Cabecalho_Fim_Excel': linha_cab_fim,

    # Colunas
    'Coluna_Inicio_Excel': col_ini,
    'Coluna_Fim_Excel': col_fim,

    # Dados
    'Linha_Dados_Inicio_Excel': linha_dados_ini,

    # Contadores
    'Registros_Final': len(df),
    'Colunas_Final': len(df.columns),

    # Timestamp
    'Timestamp': timestamp_execucao,
    'Data_Processamento': datetime.now().isoformat()
}

df_log = pd.DataFrame([log_processamento])
arquivo_log = fm.salvar(
    df_log,
    f"LOG_{nome_base}",
    tipo='xlsx',
    pasta='logs'
)
print(f" OK {arquivo_log.name}")

# ======================================================================
# ETAPA 5: CODIGO PYTHON PARA REPRODUCAO
# ======================================================================

print("\n" + "-"*70)
print("ETAPA 5: GERANDO CODIGO DE REPRODUCAO")
print("-"*70)

codigo_reprod = f'''# ======================================================================
# CODIGO DE REPRODUCAO - Gerado automaticamente
# Arquivo: {arquivo_selecionado.name}
# Sheet: {sheet_nome}
# Data: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
# ======================================================================

import pandas as pd
from pathlib import Path

# ======================================================================
# CONFIGURACAO
# ======================================================================

arquivo = Path(r"{arquivo_selecionado}")
sheet = "{sheet_nome}"

# Range de extracao (linhas Excel - comeca em 1)
linha_cabecalho_inicio = {linha_cab_ini}
linha_cabecalho_fim = {linha_cab_fim}
col_inicio = {col_ini}
col_fim = {col_fim}
linha_dados_inicio = {linha_dados_ini}

# ======================================================================
# CARREGAMENTO
# ======================================================================

print(f"Carregando: {{arquivo.name}}")
print(f"Sheet: {{sheet}}")

if linha_cabecalho_inicio == linha_cabecalho_fim:
    # Cabecalho em 1 linha
    df = pd.read_excel(
        arquivo,
        sheet_name=sheet,
        header=linha_cabecalho_inicio - 1,  # Converter para indice Python
        usecols=range(col_inicio - 1, col_fim)
    )

    # Pular linhas entre cabecalho e dados
    linhas_pular = linha_dados_inicio - linha_cabecalho_inicio - 1
    if linhas_pular > 0:
        df = df.iloc[linhas_pular:].copy()
else:
    # Cabecalho em multiplas linhas
    df_temp = pd.read_excel(
        arquivo,
        sheet_name=sheet,
        header=None,
        usecols=range(col_inicio - 1, col_fim)
    )

    # Combinar linhas do cabecalho
    cabecalho = df_temp.iloc[linha_cabecalho_inicio-1:linha_cabecalho_fim].values
    cab_final = []

    for col_idx in range(cabecalho.shape[1]):
        partes = [
            str(linha[col_idx]).strip()
            for linha in cabecalho
            if str(linha[col_idx]).strip() not in ['', 'nan', 'None']
        ]
        cab_final.append(' - '.join(partes) if partes else f'Col_{{col_idx}}')

    # Extrair dados
    df = df_temp.iloc[linha_dados_inicio-1:].copy()
    df.columns = cab_final

# Reset index
df = df.reset_index(drop=True)

print(f"Carregado: {{len(df):,}} registros × {{len(df.columns)}} colunas")

# ======================================================================
# VALIDACAO
# ======================================================================

colunas_esperadas = {df.columns.tolist()}

if df.columns.tolist() == colunas_esperadas:
    print("Estrutura validada - colunas correspondem!")
else:
    print("Diferenca na estrutura:")

    extras = set(df.columns) - set(colunas_esperadas)
    if extras:
        print(f"Colunas extras: {{extras}}")

    faltando = set(colunas_esperadas) - set(df.columns)
    if faltando:
        print(f"Colunas faltando: {{faltando}}")

print(f"\\nShape: {{df.shape}}")
print(f"Memoria: {{df.memory_usage(deep=True).sum() / 1024**2:.2f}} MB")
'''

arquivo_codigo = fm.pastas['codigos_integracao'] / f"REPROD_{nome_base}_{timestamp_execucao}.py"
with open(arquivo_codigo, 'w', encoding='utf-8') as f:
    f.write(codigo_reprod)

print(f" OK {arquivo_codigo.name}")

# ======================================================================
# ETAPA 6: SALVAR ESTADO DO BLOCO 15
# ======================================================================

print("\n" + "-"*70)
print("ETAPA 6: SALVANDO ESTADO")
print("-"*70)

bloco_15_state = {
    'bloco': 15,
    'versao': '2.3',
    'timestamp': timestamp_execucao,
    'container': container_nome,
    'executado': True,
    'arquivos_gerados': {
        'dados_limpos': arquivo_limpo.name,
        'dicionario_campos': arquivo_dict.name,
        'log_processamento': arquivo_log.name,
        'codigo_reproducao': arquivo_codigo.name
    },
    'df_shape': list(df.shape),
    'arquivo_original': arquivo_selecionado.name
}

bloco_15_state_path = pasta_base / '.bloco_15_state.json'
with open(bloco_15_state_path, 'w', encoding='utf-8') as f:
    json.dump(bloco_15_state, f, indent=2, ensure_ascii=False)

print(f" OK Estado salvo: .bloco_15_state.json")

# ======================================================================
# RESUMO DE ARQUIVOS GERADOS
# ======================================================================

print("\n" + "="*70)
print("ARQUIVOS GERADOS")
print("="*70)

arquivos_gerados = [
    ('Dados Limpos', arquivo_limpo),
    ('Dicionario de Campos', arquivo_dict),
    ('Log de Processamento', arquivo_log),
    ('Codigo de Reproducao', arquivo_codigo)
]

for desc, path in arquivos_gerados:
    print(f"\n{desc}")
    print(f"  Arquivo: {path.name}")
    print(f"  Pasta: {path.parent.name}")
    print(f"  Tamanho: {path.stat().st_size / 1024:.1f} KB")

print("\n" + "="*70)
print("EXPORTACAO CONCLUIDA COM SUCESSO")
print("="*70)

# Variavel global para uso posterior
df_resultado = df.copy()

print(f"\n Dataset disponivel em: df_resultado")
print(f"  Shape: {df_resultado.shape}")
print(f"  Memoria: {df_resultado.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print()
print(f" Proxima etapa: Analises e validacoes (se necessario)")
print("="*70)


EXPORTACAO DE RESULTADOS

----------------------------------------------------------------------
ETAPA 1: CONECTANDO COM BLOCOS ANTERIORES...
----------------------------------------------------------------------
 OK LOG GLOBAL conectado
    Container: PROCESSAR_ARQUIVOS_20251019_060722
    Timestamp: 20251019_060722
 !! FileManager sem metodo salvar - recriando...
 OK FileManager recriado: PROCESSAR_ARQUIVOS_20251019_060722
 OK DataFrame validado: (967, 25)
 !! Nenhum estado salvo - usando variaveis em memoria
 !! Arquivo nao encontrado - usando nome generico

 OK Arquivo selecionado: arquivo_processado
 OK Sheet: N/A

----------------------------------------------------------------------
ETAPA 2: EXPORTANDO DADOS LIMPOS
----------------------------------------------------------------------
 OK arquivo_processado_Limpo_20251019_060722.xlsx

----------------------------------------------------------------------
ETAPA 3: CRIANDO DICIONARIO DE CAMPOS
------------------------------------

In [47]:
# ===================================================================
# BLOCO 16 v2.0 - RELATORIO FINAL VISUAL
# ===================================================================
# COMUNICACAO VIA LOG:
# - Le: LOG GLOBAL + estados dos blocos anteriores
# - RECRIA: FileManager se necessario
# - USA: df_resultado (ou df_limpo, ou df)
# - SALVA: .bloco_16_state.json
# - EXIBE: df_resultado
# - ABRE: pasta de outputs
# ===================================================================
# CORRECOES v2.0:
# + 100% comunicacao via LOG (0% memoria)
# + Recria FileManager se necessario
# + Multiplas fontes para todas as variaveis
# + Validacao de None antes de usar
# + Exibe df_resultado (NOVO)
# + Abre pasta outputs automaticamente (NOVO)
# + Salva estado proprio
# ===================================================================

print("\n")
print("=" * 68 + "=")
print(" RELATORIO FINAL - PROCESSAMENTO CONCLUIDO".center(78))
print("=" * 68 + "=")

# ===================================================================
# 1. CONECTAR COM BLOCOS ANTERIORES (0% memoria, 100% LOG)
# ===================================================================

from pathlib import Path
import json
from datetime import datetime

# Carregar LOG GLOBAL
log_path = Path.home() / '.processador_dicionario_localizador.json'

if not log_path.exists():
    print("\n! Erro: LOG GLOBAL nao encontrado!")
    print("  Execute BLOCO 1 primeiro")
    raise RuntimeError("LOG nao disponivel")

with open(log_path, 'r', encoding='utf-8') as f:
    log_global = json.load(f)

# Extrair dados essenciais do LOG
pasta_base = Path(log_global['pasta_base_atual'])
timestamp_execucao = log_global['timestamp']

print(f"\n OK LOG GLOBAL conectado")
print(f"    Container: {pasta_base.name}")
print(f"    Timestamp: {timestamp_execucao}")

# ===================================================================
# 2. RECRIAR FileManager (SE necessario)
# ===================================================================

# Tentar usar fm da memoria
try:
    fm_existe = 'fm' in globals() and fm is not None
    if fm_existe and hasattr(fm, 'pastas'):
        print(f" OK FileManager na memoria")
    else:
        raise NameError("fm invalido")
except (NameError, AttributeError):
    print(f" AVISO: Recriando FileManager...")

    # Recriar classe FileManagerSimples
    class FileManagerSimples:
        def __init__(self, base_path, timestamp):
            self.base_path = Path(base_path)
            self.timestamp = timestamp
            self.pastas = {
                'dados': self.base_path / '01_Dados_Entrada',
                'processados': self.base_path / '02_Processados',
                'outputs': self.base_path / '03_Outputs',
                'logs': self.base_path / '04_Logs',
                'dicionarios': self.base_path / '05_Dicionarios',
                'codigos_integracao': self.base_path / '06_Codigos_Integracao'
            }

        def abrir_pasta(self, tipo='outputs'):
            """Abre pasta no explorer/finder"""
            import subprocess
            import platform

            pasta = self.pastas.get(tipo, self.pastas['outputs'])

            try:
                if platform.system() == 'Windows':
                    subprocess.run(['explorer', str(pasta)])
                elif platform.system() == 'Darwin':  # macOS
                    subprocess.run(['open', str(pasta)])
                else:  # Linux
                    subprocess.run(['xdg-open', str(pasta)])
                return True
            except Exception as e:
                print(f"\n! Erro ao abrir pasta: {e}")
                return False

    fm = FileManagerSimples(pasta_base, timestamp_execucao)
    print(f"    OK FileManager recriado")

# ===================================================================
# 3. CARREGAR DADOS DE BLOCOS ANTERIORES (MULTIPLAS FONTES)
# ===================================================================

# FONTE 1: Tentar variaveis na memoria
# FONTE 2: Ler do LOG do BLOCO 15
# FONTE 3: Ler do LOG do BLOCO 14
# FONTE 4: Ler do LOG do BLOCO 4
# FONTE 5: Fallback padrao

# ----- arquivo_selecionado -----
arquivo_selecionado = None
sheet_nome = "Sheet1"

# Memoria
if 'arquivo_selecionado' in globals():
    arquivo_selecionado = globals()['arquivo_selecionado']

# LOG BLOCO 15
if arquivo_selecionado is None:
    try:
        with open(fm.pastas['logs'] / '.bloco_15_state.json') as f:
            b15 = json.load(f)
            if 'arquivo_processado' in b15:
                arquivo_selecionado = Path(b15['arquivo_processado'])
    except:
        pass

# LOG BLOCO 4
if arquivo_selecionado is None:
    try:
        with open(fm.pastas['logs'] / '.bloco_4_state.json') as f:
            b4 = json.load(f)
            arquivo_selecionado = Path(b4['arquivo'])
            sheet_nome = b4.get('sheet_nome', sheet_nome)
    except:
        pass

# ----- tipos_detectados -----
tipos_detectados = {}

# Memoria
if 'tipos_detectados' in globals():
    tipos_detectados = globals()['tipos_detectados']

# LOG BLOCO 12
if not tipos_detectados:
    try:
        with open(fm.pastas['logs'] / '.bloco_12_state.json') as f:
            b12 = json.load(f)
            tipos_detectados = b12.get('tipos_detectados', {})
    except:
        pass

# ----- config_carga -----
metodo_carga = "pandas"
linha_cabecalho_inicio = 0
idx_cab_inicio = 0

try:
    with open(fm.pastas['logs'] / '.bloco_6_state.json') as f:
        b6 = json.load(f)
        metodo_carga = b6.get('metodo', 'pandas')
        linha_cabecalho_inicio = b6.get('linha_cabecalho_inicio', 0)
        idx_cab_inicio = b6.get('idx_cab_inicio', 0)
except:
    pass

# ----- DataFrames -----
df_resultado = None
df_limpo = None
df_bruto = None

# FONTE 1: Ler do arquivo salvo pelo BLOCO 15 (PRIORITARIO)
try:
    arquivo_limpo = list(fm.pastas['processados'].glob(
        f'*Limpo_{timestamp_execucao}.xlsx'
    ))
    if arquivo_limpo:
        import pandas as pd
        df_resultado = pd.read_excel(arquivo_limpo[0])
        print(f" OK DataFrame carregado do arquivo: {arquivo_limpo[0].name}")
except Exception as e:
    print(f" ! Nao foi possivel carregar do arquivo: {e}")

# FONTE 2: Tentar memoria (prioridade: df_resultado > df > df_limpo > df_bruto)
if df_resultado is None:
    if 'df_resultado' in globals():
        df_resultado = globals()['df_resultado']
        print(f" OK DataFrame da memoria: df_resultado")
    elif 'df' in globals():
        df_resultado = globals()['df'].copy()
        print(f" OK DataFrame da memoria: df")
    elif 'df_limpo' in globals():
        df_limpo = globals()['df_limpo']
        df_resultado = df_limpo.copy()
        print(f" OK DataFrame da memoria: df_limpo")
    elif 'df_bruto' in globals():
        df_bruto = globals()['df_bruto']
        df_resultado = df_bruto.copy()
        print(f" OK DataFrame da memoria: df_bruto")

if df_resultado is None:
    print("\n! ERRO: Nenhum DataFrame disponivel!")
    print("  Execute o BLOCO 15 primeiro para gerar os arquivos")
    raise RuntimeError("DataFrame nao disponivel")

# Se df_limpo nao existe, usar df_resultado
if df_limpo is None:
    df_limpo = df_resultado.copy()

# Se df_bruto nao existe, tentar carregar do estado do BLOCO 9
if df_bruto is None:
    try:
        with open(fm.pastas['logs'] / '.bloco_9_state.json') as f:
            b9 = json.load(f)
            # Usar estatisticas salvas se existirem
            if 'estatisticas' in b9:
                # Criar um DataFrame fake com as dimensoes originais
                # apenas para calcular estatisticas
                import pandas as pd
                import numpy as np
                rows = b9['estatisticas'].get('total_registros', len(df_resultado))
                cols = b9['estatisticas'].get('total_colunas', len(df_resultado.columns))
                # Usar df_resultado como base para df_bruto
                df_bruto = df_resultado.copy()
            else:
                df_bruto = df_resultado.copy()
    except:
        # Fallback: usar df_resultado
        df_bruto = df_resultado.copy()

print(f" OK Dados carregados de multiplas fontes")

# ----- log_limpeza -----
log_limpeza = []

try:
    with open(fm.pastas['logs'] / '.bloco_10_state.json') as f:
        b10 = json.load(f)
        log_limpeza = b10.get('operacoes_limpeza', [])
except:
    pass

# ===================================================================
# INFORMACOES DO ARQUIVO
# ===================================================================

print("\n" + "=" * 68 + "=")
print(" INFORMACOES DO ARQUIVO".center(78))
print("=" * 68 + "=")

if arquivo_selecionado is not None:
    print(f"  Nome: {arquivo_selecionado.name:<68}")
else:
    print(f"  Nome: [nao disponivel]")

print(f"  Sheet: {sheet_nome:<67}")
print(f"  Cabecalho: Linha {linha_cabecalho_inicio} (Excel) / "
      f"Indice {idx_cab_inicio} (Python)")
print(f"  Metodo: {metodo_carga:<66}")
print("=" * 68 + "=")

# ===================================================================
# ESTATISTICAS DE PROCESSAMENTO
# ===================================================================

print("\n" + "=" * 68 + "=")
print(" ESTATISTICAS DE PROCESSAMENTO".center(78))
print("=" * 68 + "=")

print(f"  Registros originais: {len(df_bruto):>6,}")
print(f"  Registros finais:    {len(df_limpo):>6,}")
print(f"  Diferenca:           {len(df_bruto) - len(df_limpo):>6,} "
      f"removidos")
print("  " + "-" * 64)
print(f"  Colunas originais:   {len(df_bruto.columns):>6,}")
print(f"  Colunas finais:      {len(df_limpo.columns):>6,}")
print(f"  Diferenca:           {len(df_bruto.columns) - len(df_limpo.columns):>6,} "
      f"removidas")
print("  " + "-" * 64)

memoria_mb = df_limpo.memory_usage(deep=True).sum() / 1024**2
print(f"  Memoria em uso:      {memoria_mb:>6.2f} MB")

duplicatas = df_limpo.duplicated().sum()
print(f"  Linhas duplicadas:   {duplicatas:>6,}")

print("=" * 68 + "=")

# ===================================================================
# QUALIDADE DOS DADOS
# ===================================================================

print("\n" + "=" * 68 + "=")
print(" QUALIDADE DOS DADOS".center(78))
print("=" * 68 + "=")

total_nulos = df_limpo.isnull().sum().sum()
total_celulas = len(df_limpo) * len(df_limpo.columns)
pct_nulos = (total_nulos / total_celulas) * 100 if total_celulas > 0 else 0

print(f"  Total de valores nulos: {total_nulos:>6,} ({pct_nulos:>5.2f}%)")

colunas_com_nulos = df_limpo.isnull().sum()
colunas_com_nulos = colunas_com_nulos[colunas_com_nulos > 0]

if len(colunas_com_nulos) > 0:
    print(f"  Colunas com nulos:      {len(colunas_com_nulos):>6,}")
else:
    print(f"  OK Nenhuma coluna com valores nulos!")

print("=" * 68 + "=")

# ===================================================================
# DETECCAO DE TIPOS
# ===================================================================

print("\n" + "=" * 68 + "=")
print(" DETECCAO DE TIPOS".center(78))
print("=" * 68 + "=")

if tipos_detectados:
    alta = sum(1 for info in tipos_detectados.values()
               if info.get('confianca', 0) >= 0.90)
    media = sum(1 for info in tipos_detectados.values()
                if 0.70 <= info.get('confianca', 0) < 0.90)
    baixa = sum(1 for info in tipos_detectados.values()
                if info.get('confianca', 0) < 0.70)

    print(f"  OK Alta confianca (>=90%):   {alta:>3} colunas")
    print(f"  !  Media confianca (70-90%): {media:>3} colunas")
    print(f"  ?  Baixa confianca (<70%):   {baixa:>3} colunas")
    print("  " + "-" * 64)

    # Top 5 tipos detectados
    tipos_resumo = {}
    for info in tipos_detectados.values():
        tipo = info.get('campo_detectado', 'DESCONHECIDO')
        tipos_resumo[tipo] = tipos_resumo.get(tipo, 0) + 1

    print("  Top 5 tipos mais comuns:")
    for i, (tipo, count) in enumerate(
        sorted(tipos_resumo.items(), key=lambda x: x[1], reverse=True)[:5],
        1
    ):
        print(f"     {i}. {tipo:<25} ({count:>2} colunas)")
else:
    print("  Informacao nao disponivel")

print("=" * 68 + "=")

# ===================================================================
# OPERACOES DE LIMPEZA
# ===================================================================

print("\n" + "=" * 68 + "=")
print(" OPERACOES DE LIMPEZA REALIZADAS".center(78))
print("=" * 68 + "=")

if log_limpeza:
    for i, operacao in enumerate(log_limpeza[:10], 1):
        # Quebrar linhas longas
        if len(str(operacao)) > 67:
            print(f"  {i}. {str(operacao)[:64]}...")
        else:
            print(f"  {i}. {operacao}")

    if len(log_limpeza) > 10:
        print(f"  ... e mais {len(log_limpeza) - 10} operacoes")
else:
    print(f"  OK Nenhuma limpeza necessaria - dados ja estavam limpos!")

print("=" * 68 + "=")

# ===================================================================
# ARQUIVOS GERADOS
# ===================================================================

print("\n" + "=" * 68 + "=")
print(" ARQUIVOS GERADOS".center(78))
print("=" * 68 + "=")

# Listar arquivos na pasta de outputs
arquivos_gerados = []

try:
    # Buscar arquivos com timestamp atual
    for pasta_nome in ['processados', 'outputs', 'logs', 'codigos_integracao']:
        pasta = fm.pastas[pasta_nome]
        if pasta.exists():
            for arquivo in pasta.glob(f'*{timestamp_execucao}*'):
                tamanho_kb = arquivo.stat().st_size / 1024
                arquivos_gerados.append((
                    arquivo.name,
                    pasta_nome.upper(),
                    tamanho_kb
                ))
except Exception as e:
    print(f"  ! Erro ao listar arquivos: {e}")

if arquivos_gerados:
    for nome, pasta, tamanho in arquivos_gerados:
        print(f"\n  {nome}")
        print(f"     Pasta: {pasta}")
        print(f"     Tamanho: {tamanho:>6.1f} KB")
else:
    print("  Nenhum arquivo encontrado com este timestamp")

print("\n" + "=" * 68 + "=")

# ===================================================================
# PROXIMOS PASSOS
# ===================================================================

print("\n" + "=" * 68 + "=")
print(" PROXIMOS PASSOS RECOMENDADOS".center(78))
print("=" * 68 + "=")

sugestoes = []

# Sugestoes baseadas em qualidade
if len(colunas_com_nulos) > 0:
    sugestoes.append("Tratar valores nulos nas colunas identificadas")

if tipos_detectados:
    baixa = sum(1 for info in tipos_detectados.values()
                if info.get('confianca', 0) < 0.70)
    if baixa > 0:
        sugestoes.append(f"Revisar {baixa} campos com baixa confianca")

if duplicatas > 0:
    sugestoes.append("Investigar e remover linhas duplicadas")

# Sugestoes padrao
sugestoes.extend([
    "Revisar o dicionario de campos gerado",
    "Validar tipos detectados conforme necessidade",
    "Utilizar codigo de reproducao para reprocessar"
])

for i, sugestao in enumerate(sugestoes[:6], 1):
    print(f"  {i}. {sugestao}")

print("=" * 68 + "=")

# ===================================================================
# RODAPE
# ===================================================================

print("\n" + "=" * 68 + "=")
print(" OK PROCESSAMENTO CONCLUIDO COM SUCESSO!".center(78))
print("=" * 68 + "=")
print(f" Timestamp: {timestamp_execucao}")
print(f" Dataset disponivel em: df_resultado")
print(f" Shape: {df_resultado.shape}")
print("=" * 68 + "=")

# ===================================================================
# SALVAR ESTADO DO BLOCO 16
# ===================================================================

estado_bloco16 = {
    'timestamp': timestamp_execucao,
    'registros_finais': len(df_resultado),
    'colunas_finais': len(df_resultado.columns),
    'memoria_mb': float(memoria_mb),
    'duplicatas': int(duplicatas),
    'valores_nulos': int(total_nulos),
    'arquivos_gerados': len(arquivos_gerados)
}

try:
    arquivo_estado = fm.pastas['logs'] / '.bloco_16_state.json'
    with open(arquivo_estado, 'w', encoding='utf-8') as f:
        json.dump(estado_bloco16, f, indent=2, ensure_ascii=False)
    print(f"\n OK Estado salvo: .bloco_16_state.json")
except Exception as e:
    print(f"\n! Aviso: Nao foi possivel salvar estado: {e}")

def abrir_pasta_outputs(fm):
    """Abre pasta de outputs no Explorer/Finder"""
    import subprocess
    import platform

    pasta = fm.pastas.get('outputs', fm.pastas.get('03_Outputs'))

    try:
        if platform.system() == 'Windows':
            subprocess.run(['explorer', str(pasta)])
            print(f" OK Pasta de outputs aberta!")
            return True
        elif platform.system() == 'Darwin':  # macOS
            subprocess.run(['open', str(pasta)])
            print(f" OK Pasta de outputs aberta!")
            return True
        else:  # Linux
            subprocess.run(['xdg-open', str(pasta)])
            print(f" OK Pasta de outputs aberta!")
            return True
    except Exception as e:
        print(f" ! Erro ao abrir pasta: {e}")
        print(f"   Caminho: {pasta}")
        return False

# Aplicar patch ao FileManager existente
if 'fm' in globals() and fm is not None:
    abrir_pasta_outputs(fm)
else:
    print("! FileManager nao encontrado na memoria")

# ===================================================================
# ABERTURA AUTOMATICA DA PASTA DESTINO (NOVO!)
# ===================================================================

print("\n" + "=" * 68 + "=")
print(" ABRINDO PASTA DE OUTPUTS...".center(78))
print("=" * 68 + "=")

try:
    sucesso = fm.abrir_pasta('outputs')
    if sucesso:
        print(" OK Pasta de outputs aberta!")
    else:
        print(f" ! Pasta nao abriu automaticamente")
        print(f"   Caminho: {fm.pastas['outputs']}")
except Exception as e:
    print(f" ! Erro ao abrir pasta: {e}")
    print(f"   Caminho: {fm.pastas['outputs']}")

print("\n TIP: Se a pasta nao abriu, o caminho esta exibido acima!")

# ===================================================================
# EXIBIR DATAFRAME RESULTADO (NOVO!)
# ===================================================================

print("\n" + "=" * 68 + "=")
print(" DATASET FINAL (df_resultado)".center(78))
print("=" * 68 + "=")

# Exibir info do DataFrame
print(f"\nShape: {df_resultado.shape}")
print(f"Colunas: {list(df_resultado.columns)}")
print(f"\nPrimeiras 10 linhas:")
print("=" * 68 + "=")

try:
    # Tentar usar display (Jupyter)
    display(df_resultado.head(10))
except NameError:
    # Fallback para print
    print(df_resultado.head(10).to_string())

print("\n" + "=" * 68 + "=")
print(" FIM DO RELATORIO".center(78))
print("=" * 68 + "=")
print("\n")



                   RELATORIO FINAL - PROCESSAMENTO CONCLUIDO                  

 OK LOG GLOBAL conectado
    Container: PROCESSAR_ARQUIVOS_20251019_060722
    Timestamp: 20251019_060722
 OK FileManager na memoria
 OK DataFrame carregado do arquivo: arquivo_processado_Limpo_20251019_060722.xlsx
 OK Dados carregados de multiplas fontes

                            INFORMACOES DO ARQUIVO                            
  Nome: [nao disponivel]
  Sheet: Sheet1                                                             
  Cabecalho: Linha 0 (Excel) / Indice 0 (Python)
  Metodo: pandas                                                            

                         ESTATISTICAS DE PROCESSAMENTO                        
  Registros originais:    199
  Registros finais:       199
  Diferenca:                0 removidos
  ----------------------------------------------------------------
  Colunas originais:       25
  Colunas finais:          25
  Diferenca:                0 removidas
  -----

,Ano civil/mês,Centro,Unnamed: 2,HierarqPrd,Produto,_dup1,Estoque Inicial,Entrada,Variação Externa,Variação Externa %,...,Imposto,Valor da Variação Interna,_dup2,Quantidade Excedente da Variação Externa,Valor Excedente da Variação Externa (R$),Quantidade Excedente da Variação Interna,Valor Excedente da Variação Interna (R$),Quantidade Excedente da Variação Total,Valor Excedente da Variação Total (R$),Valor Excedente da Variação Total + Imposto (R$)
0,1.2025,5126,BAV1,Diesel - Comum,01.011.674,ÓLEO DIESEL B S10,16924.0,NaN,NaN,NaN,...,0.00,95.402202,NaN,0.00,0.000000,10.19,54.008246,10.19,54.01,54.01
1,1.2025,5126,BAV1,Querosene de Aviação,01.001.422,JET A NAO TABELADO - LI,373850.0,939139.0,824.0,0.087740,...,-2811.15,-38.904816,NaN,366.22,1424.772156,0.00,0.000000,0.00,0.00,2811.15
2,1.2025,5126,BAV1,Querosene de Aviação,01.003.826,JET A INTERNACIONAL I - LI,598315.0,5188210.0,1494.0,0.028796,...,0.00,0.000000,NaN,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00
3,1.2025,5105,BAV2,Gasolina Comum,01.000.078,GASOLINA COMUM C,13076.0,14828.0,5.0,0.033720,...,0.00,1201.140348,NaN,0.00,0.000000,220.33,1130.971166,220.33,1130.97,1130.97
4,1.2025,5105,BAV2,Diesel - Comum,01.011.674,ÓLEO DIESEL B S10,122306.0,178128.0,-17.0,-0.009544,...,0.00,-2036.637033,NaN,0.00,0.000000,-240.63,-1282.921386,-240.63,-1282.92,-1282.92
5,1.2025,5105,BAV2,Diesel - Comum,01.024.741,Vibra Diesel Renovável HVO10,3361.0,14839.0,-1.0,-0.006739,...,0.00,189.216874,NaN,0.00,0.000000,20.00,126.144582,20.00,126.14,126.14
6,1.2025,5105,BAV2,Querosene de Aviação,01.016.205,JET A - PREÇO FIXO - VRG,1425416.0,3500000.0,NaN,NaN,...,0.00,0.000000,NaN,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00
7,1.2025,5105,BAV2,Querosene de Aviação,01.001.422,JET A NAO TABELADO - LI,9793143.0,19273387.0,-59980.0,-0.311206,...,0.00,379506.723303,NaN,-40631.61,-155516.920049,11630.85,44516.916006,11630.85,44516.92,44516.92
8,1.2025,5105,BAV2,Querosene de Aviação,01.011.754,JET A PREÇO FIXO,4578994.0,NaN,NaN,NaN,...,0.00,0.000000,NaN,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00
9,1.2025,5105,BAV2,Querosene de Aviação,01.026.471,JET A-1 NAO TABELADO - LI,513358.0,1200000.0,-10563.0,-0.880250,...,-9028.91,11742.667607,NaN,-9362.66,-35870.343978,0.00,0.000000,0.00,0.00,9028.91



                               FIM DO RELATORIO                               




In [ ]:
df_resultado

In [ ]:

"""
Extrator de Headers de Notebook Jupyter
Extrai todos os cabeçalhos/títulos de blocos de código
"""

import json
import re
from pathlib import Path

def extrair_headers_notebook(caminho_notebook):
    """
    Extrai headers de um notebook Jupyter (.ipynb)

    Args:
        caminho_notebook: Path ou string do arquivo .ipynb

    Returns:
        Lista de dicionários com informações dos headers
    """

    # Carregar notebook
    with open(caminho_notebook, 'r', encoding='utf-8') as f:
        notebook = json.load(f)

    headers = []

    # Iterar pelas células
    for idx, cell in enumerate(notebook.get('cells', [])):
        cell_type = cell.get('cell_type')
        source = cell.get('source', [])

        # Converter source para string se for lista
        if isinstance(source, list):
            source_text = ''.join(source)
        else:
            source_text = source

        # MARKDOWN: Extrair títulos (#, ##, ###, etc)
        if cell_type == 'markdown':
            for line in source_text.split('\n'):
                if line.strip().startswith('#'):
                    nivel = len(re.match(r'^#+', line.strip()).group())
                    titulo = line.strip().lstrip('#').strip()

                    headers.append({
                        'celula': idx,
                        'tipo': 'markdown',
                        'nivel': nivel,
                        'titulo': titulo,
                        'preview': line.strip()[:80]
                    })

        # CODE: Extrair comentários de blocos (# ===, # ---, # BLOCO, etc)
        elif cell_type == 'code':
            # Padrões de headers em código
            patterns = [
                (r'^# ={3,}', 'separator'),  # # ===
                (r'^# -{3,}', 'separator'),  # # ---
                (r'^# BLOCO \d+', 'bloco'),  # # BLOCO 1
                (r'^# \d+\.', 'numerado'),   # # 1.
            ]

            for line_num, line in enumerate(source_text.split('\n')):
                line_stripped = line.strip()

                # Verificar padrões
                for pattern, tipo_header in patterns:
                    if re.match(pattern, line_stripped, re.IGNORECASE):
                        # Tentar pegar próxima linha como título
                        lines = source_text.split('\n')
                        titulo = line_stripped.lstrip('#').strip()

                        # Se for separator, pegar linha seguinte
                        if tipo_header == 'separator' and line_num + 1 < len(lines):
                            next_line = lines[line_num + 1].strip()
                            if next_line.startswith('#'):
                                titulo = next_line.lstrip('#').strip()

                        headers.append({
                            'celula': idx,
                            'tipo': 'code',
                            'subtipo': tipo_header,
                            'titulo': titulo,
                            'preview': line.strip()[:80]
                        })
                        break

    return headers


def imprimir_estrutura(headers, mostrar_codigo=True):
    """
    Imprime a estrutura do notebook de forma organizada

    Args:
        headers: Lista de headers extraídos
        mostrar_codigo: Se True, mostra headers de código também
    """

    print("=" * 80)
    print("ESTRUTURA DO NOTEBOOK")
    print("=" * 80)
    print()

    for i, header in enumerate(headers, 1):
        # Filtrar código se necessário
        if not mostrar_codigo and header['tipo'] == 'code':
            continue

        # Formatação por tipo
        if header['tipo'] == 'markdown':
            # Indentação por nível
            indent = "  " * (header['nivel'] - 1)
            simbolo = "#" * header['nivel']
            print(f"{indent}[MD] {simbolo} {header['titulo']}")

        elif header['tipo'] == 'code':
            subtipo = header.get('subtipo', 'outro')

            if subtipo == 'bloco':
                print(f"📦 [BLOCO] {header['titulo']}")
            elif subtipo == 'separator':
                print(f"─── {header['titulo']}")
            elif subtipo == 'numerado':
                print(f"  → {header['titulo']}")
            else:
                print(f"  • {header['titulo']}")

        # Mostrar número da célula
        print(f"     (Célula {header['celula']})")
        print()


def exportar_markdown(headers, arquivo_saida):
    """
    Exporta a estrutura para um arquivo Markdown

    Args:
        headers: Lista de headers
        arquivo_saida: Nome do arquivo .md para salvar
    """

    with open(arquivo_saida, 'w', encoding='utf-8') as f:
        f.write("# Estrutura do Notebook\n\n")

        for header in headers:
            if header['tipo'] == 'markdown':
                nivel = header['nivel']
                titulo = header['titulo']
                f.write(f"{'#' * nivel} {titulo}\n")
                f.write(f"*Célula {header['celula']}*\n\n")

            elif header['tipo'] == 'code':
                titulo = header['titulo']
                f.write(f"- **[CODE]** {titulo}\n")
                f.write(f"  - *Célula {header['celula']}*\n\n")

    print(f"✅ Estrutura exportada para: {arquivo_saida}")


# ============================================================================
# EXEMPLO DE USO
# ============================================================================

if __name__ == "__main__":

    # OPÇÃO 1: Usar com arquivo específico
    # -----------------------------------------

    # Defina o caminho do seu notebook
    caminho = "PROCESSAR ARQUIVOS DESCONHECIDOS 4.2.ipynb"

    # Verificar se arquivo existe
    if not Path(caminho).exists():
        print(f"❌ Arquivo não encontrado: {caminho}")
        print("\n💡 Como usar:")
        print("   1. Coloque este script na mesma pasta do notebook")
        print("   2. Ou altere a variável 'caminho' acima")
        exit(1)

    # Extrair headers
    print(f"📖 Lendo notebook: {caminho}")
    print()

    headers = extrair_headers_notebook(caminho)

    print(f"✅ Encontrados {len(headers)} headers\n")

    # Imprimir estrutura
    imprimir_estrutura(headers, mostrar_codigo=True)

    # OPÇÃO 2: Exportar para Markdown
    # -----------------------------------------

    # Descomentar para exportar
    # exportar_markdown(headers, "estrutura_notebook.md")


    # OPÇÃO 3: Filtrar apenas BLOCOs
    # -----------------------------------------

    print("\n" + "=" * 80)
    print("APENAS BLOCOS PRINCIPAIS")
    print("=" * 80)
    print()

    blocos = [h for h in headers if 'BLOCO' in h.get('titulo', '').upper()]

    for bloco in blocos:
        print(f"📦 {bloco['titulo']} (Célula {bloco['celula']})")


    # OPÇÃO 4: Estatísticas
    # -----------------------------------------

    print("\n" + "=" * 80)
    print("ESTATÍSTICAS")
    print("=" * 80)
    print()

    total_markdown = sum(1 for h in headers if h['tipo'] == 'markdown')
    total_code = sum(1 for h in headers if h['tipo'] == 'code')
    total_blocos = len(blocos)

    print(f"📊 Headers Markdown: {total_markdown}")
    print(f"💻 Headers Code: {total_code}")
    print(f"📦 Blocos identificados: {total_blocos}")
    print(f"📝 Total de headers: {len(headers)}")


In [ ]:
# ======================================================================
# EXPORT SYSTEM - PROCESSADOR DE ARQUIVOS - CORRIGIDO v2.0
# ======================================================================
# CORRECOES v2.0:
# + FIX CRITICO: KeyError 'pasta_destino' -> usar 'pasta_base_atual'
# + FIX: KeyError 'container' -> derivar do path (pasta_base.name)
# + MANTIDO: Toda funcionalidade original
# ======================================================================

print("="*70)
print("EXPORT SYSTEM - PROCESSADOR DE ARQUIVOS v2.0")
print("="*70)
print("MODO: Exporta apenas sistema (notebook + dicionario)")
print("      NAO exporta containers de dados processados")
print("="*70)

from pathlib import Path
import json
import shutil
from datetime import datetime
import zipfile

def exportar_sistema():
    """
    Exporta projeto completo para .zip portavel

    v2.0: CORRIGIDO - usa chaves corretas do LOG GLOBAL
    """

    # ===================================================================
    # 1. CARREGAR LOG GLOBAL (CORRIGIDO)
    # ===================================================================

    log_global_path = Path.home() / '.processador_dicionario_localizador.json'

    if not log_global_path.exists():
        print("\n! LOG GLOBAL nao encontrado!")
        print("  Execute o notebook antes de exportar.")
        return

    with open(log_global_path, 'r', encoding='utf-8') as f:
        log_global = json.load(f)

    # ===================================================================
    # FIX CRITICO: Usar chaves corretas
    # ===================================================================
    # ANTES (ERRADO):
    # pasta_destino = Path(log_global['pasta_destino'])
    # container_nome = log_global['container']

    # DEPOIS (CORRETO):
    pasta_base = Path(log_global['pasta_base_atual'])
    container_nome = pasta_base.name
    container_path = pasta_base

    print(f"\n OK Container encontrado:")
    print(f"    {container_path}")
    print(f"    Nome: {container_nome}")

    # ===================================================================
    # 2. CRIAR PASTA TEMPORARIA
    # ===================================================================

    export_dir = Path('PROCESSADOR_EXPORT_TEMP')

    # Limpar se ja existe
    if export_dir.exists():
        shutil.rmtree(export_dir)

    export_dir.mkdir(exist_ok=True)

    print(f"\n OK Criando estrutura de exportacao...")

    # ===================================================================
    # 3. COPIAR ARQUIVOS ESSENCIAIS
    # ===================================================================

    # ---------------------------------------------------------------
    # 3.1. Dicionarios
    # ---------------------------------------------------------------
    dicionarios_src = container_path / '05_Dicionarios'
    dicionarios_dst = export_dir / 'data' / 'dicionarios'
    dicionarios_dst.mkdir(parents=True, exist_ok=True)

    if dicionarios_src.exists():
        for arquivo in dicionarios_src.glob('*.json'):
            shutil.copy2(arquivo, dicionarios_dst)
            print(f"    OK Copiado: {arquivo.name}")
    else:
        print(f"    ! Dicionarios nao encontrados")

    # ---------------------------------------------------------------
    # 3.2. Logs (estados dos blocos)
    # ---------------------------------------------------------------
    logs_src = container_path / '04_Logs'
    logs_dst = export_dir / 'data' / 'logs'
    logs_dst.mkdir(parents=True, exist_ok=True)

    if logs_src.exists():
        for arquivo in logs_src.glob('.*.json'):
            shutil.copy2(arquivo, logs_dst)
            print(f"    OK Copiado: {arquivo.name}")
    else:
        print(f"    ! Logs nao encontrados")

    # ---------------------------------------------------------------
    # 3.3. Notebook (buscar no projeto atual)
    # ---------------------------------------------------------------
    notebooks_dst = export_dir / 'notebooks'
    notebooks_dst.mkdir(parents=True, exist_ok=True)

    # Buscar notebook principal
    notebook_encontrado = False
    for notebook in Path('.').glob('*.ipynb'):
        if 'PROCESSAR' in notebook.name.upper():
            shutil.copy2(notebook, notebooks_dst)
            print(f"    OK Copiado: {notebook.name}")
            notebook_encontrado = True

    if not notebook_encontrado:
        print(f"    ! Notebook nao encontrado")

    # ---------------------------------------------------------------
    # 3.4. Scripts
    # ---------------------------------------------------------------
    scripts_dst = export_dir / 'scripts'
    scripts_dst.mkdir(parents=True, exist_ok=True)

    # Copiar scripts de integracao se existirem
    scripts_src = container_path / '06_Codigos_Integracao'
    if scripts_src.exists():
        for script in scripts_src.glob('*.py'):
            shutil.copy2(script, scripts_dst)
            print(f"    OK Copiado: {script.name}")

    # ===================================================================
    # 4. CRIAR CONFIGURACAO
    # ===================================================================

    config_dst = export_dir / 'config'
    config_dst.mkdir(exist_ok=True)

    settings = {
        'exported_at': datetime.now().isoformat(),
        'original_container': container_nome,
        'original_path': str(container_path),
        'version': '4.2',
        'python_version': '3.8+',
        'estrutura': {
            'notebooks': 'Notebooks Jupyter do processador',
            'data/dicionarios': 'Dicionarios de campos conhecidos',
            'data/logs': 'Estados dos blocos (auditoria)',
            'scripts': 'Scripts de integracao e reproducao',
            'config': 'Configuracoes do sistema'
        }
    }

    with open(config_dst / 'settings.json', 'w', encoding='utf-8') as f:
        json.dump(settings, f, indent=2, ensure_ascii=False)

    print(f"    OK Configuracao criada")

    # ===================================================================
    # 5. CRIAR README
    # ===================================================================

    readme_content = f"""# PROCESSADOR DE ARQUIVOS DESCONHECIDOS v4.2

Exportado de: {container_nome}
Data: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

## ESTRUTURA

```
PROCESSADOR_EXPORT/
├── notebooks/          # Notebooks Jupyter
├── data/
│   ├── dicionarios/   # Campos conhecidos
│   └── logs/          # Estados dos blocos
├── scripts/           # Scripts de integracao
├── config/            # Configuracoes
└── README.md          # Este arquivo
```

## REQUISITOS

- Python 3.8+
- pandas
- openpyxl
- tkinter (para GUI)

## INSTALACAO

```bash
pip install pandas openpyxl
```

## USO BASICO

1. **Abrir notebook**:
   ```bash
   jupyter notebook notebooks/PROCESSAR*.ipynb
   ```

2. **Executar blocos em sequencia**:
   - BLOCO 1: Criar estrutura
   - BLOCO 2: Carregar classes
   - BLOCO 3-13: Processar arquivo

3. **Usar scripts standalone**:
   ```bash
   python scripts/script_reproducao.py
   ```

## DICIONARIO DE CAMPOS

O dicionario em `data/dicionarios/` contem:
- Campos conhecidos
- Mapeamentos automaticos
- Regras de validacao

## LOGS E AUDITORIA

Estados dos blocos salvos em `data/logs/`:
- `.bloco_N_state.json`: Estado de cada bloco
- Permite rastreabilidade completa

## SUPORTE

Para mais informacoes sobre o sistema original:
- Container: {container_nome}
- Path: {container_path}

---
Gerado automaticamente pelo Export System v2.0
"""

    with open(export_dir / 'README.md', 'w', encoding='utf-8') as f:
        f.write(readme_content)

    print(f"    OK README criado")

    # ===================================================================
    # 6. CRIAR ARQUIVO .ZIP
    # ===================================================================

    timestamp_str = datetime.now().strftime('%Y%m%d_%H%M%S')
    zip_filename = f'PROCESSADOR_EXPORT_{timestamp_str}.zip'
    zip_path = Path('.') / zip_filename

    print(f"\n Criando arquivo .zip...")

    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for file_path in export_dir.rglob('*'):
            if file_path.is_file():
                arcname = file_path.relative_to(export_dir)
                zipf.write(file_path, arcname)

    # ===================================================================
    # 7. LIMPAR PASTA TEMPORARIA
    # ===================================================================

    shutil.rmtree(export_dir)

    # ===================================================================
    # 8. RELATORIO FINAL
    # ===================================================================

    zip_size_mb = zip_path.stat().st_size / (1024 * 1024)

    print("\n" + "="*70)
    print(" OK EXPORTACAO CONCLUIDA")
    print("="*70)

    print(f"\n Arquivo gerado:")
    print(f"    {zip_path}")
    print(f"    Tamanho: {zip_size_mb:.2f} MB")

    print(f"\n Conteudo:")
    print(f"    - Notebooks do processador")
    print(f"    - Dicionarios de campos")
    print(f"    - Estados dos blocos (logs)")
    print(f"    - Scripts de integracao")
    print(f"    - Configuracoes")
    print(f"    - README com instrucoes")

    print(f"\n Para usar:")
    print(f"    1. Extrair arquivo .zip")
    print(f"    2. Ler README.md")
    print(f"    3. Instalar requisitos")
    print(f"    4. Executar notebooks ou scripts")

    print("\n" + "="*70)

    return str(zip_path)


if __name__ == "__main__":
    exportar_sistema()

In [ ]:
# ======================================================================
# GERADOR DEFINITIVO - SEM ERROS DE TEMPLATE
# ======================================================================
# COPIE TUDO E EXECUTE - GARANTIDO QUE FUNCIONA!
# ======================================================================

from pathlib import Path
import json
from datetime import datetime

# Carregar LOG GLOBAL
log_path = Path.home() / '.processador_dicionario_localizador.json'

if not log_path.exists():
    print("! ERRO: Execute o notebook completo primeiro")
    raise FileNotFoundError("LOG GLOBAL nao encontrado")

with open(log_path, 'r', encoding='utf-8') as f:
    log_global = json.load(f)

# Extrair configuracao
pasta_base = Path(log_global['pasta_base_atual'])
container_nome = pasta_base.name
timestamp = log_global['timestamp']

print("="*70)
print("GERADOR DE SCRIPT v3.0 DEFINITIVO")
print("="*70)
print(f"\n OK Container: {container_nome}")
print(f" OK Timestamp: {timestamp}")

# ======================================================================
# SALVAR SCRIPT LINHA POR LINHA (SEM TEMPLATE)
# ======================================================================

output_path = pasta_base / '06_Codigos_Integracao' / 'script_reproducao.py'
output_path.parent.mkdir(parents=True, exist_ok=True)

with open(output_path, 'w', encoding='utf-8') as f:
    # Cabecalho
    f.write('# ' + '='*70 + '\n')
    f.write('# SCRIPT DE REPRODUCAO - PROCESSADOR v4.2\n')
    f.write('# ' + '='*70 + '\n')
    f.write(f'# Gerado: {datetime.now().isoformat()}\n')
    f.write(f'# Container: {container_nome}\n')
    f.write(f'# Timestamp: {timestamp}\n')
    f.write('# ' + '='*70 + '\n\n')

    # Imports
    f.write('from pathlib import Path\n')
    f.write('import json\n')
    f.write('import pandas as pd\n')
    f.write('from datetime import datetime\n\n')

    # Configuracao
    f.write('# ' + '='*70 + '\n')
    f.write('# CONFIGURACAO\n')
    f.write('# ' + '='*70 + '\n\n')
    f.write('SCRIPT_DIR = Path(__file__).parent\n')
    f.write('BASE_DIR = SCRIPT_DIR.parent\n')
    f.write('PASTA_ENTRADA = Path("./entrada")\n\n')

    f.write('print("="*70)\n')
    f.write('print("SCRIPT DE REPRODUCAO - PROCESSADOR v4.2")\n')
    f.write('print("="*70)\n')
    f.write(f'print("\\nContainer original: {container_nome}")\n')
    f.write(f'print("Timestamp: {timestamp}")\n\n')

    # Carregar configuracoes
    f.write('# ' + '='*70 + '\n')
    f.write('# CARREGAR CONFIGURACOES\n')
    f.write('# ' + '='*70 + '\n\n')
    f.write('print("\\n" + "="*70)\n')
    f.write('print("CARREGANDO CONFIGURACOES")\n')
    f.write('print("="*70)\n\n')

    # Carregar dicionario
    f.write('# Carregar dicionario\n')
    f.write('dicionario_file = BASE_DIR / "data" / "dicionarios" / "DICT_Dicionario_Persistente.json"\n')
    f.write('if dicionario_file.exists():\n')
    f.write('    with open(dicionario_file, "r", encoding="utf-8") as f:\n')
    f.write('        DICIONARIO = json.load(f)\n')
    f.write('    print(f"\\n OK Dicionario: {len(DICIONARIO.get(\'campos_conhecidos\', {}))} campos")\n')
    f.write('else:\n')
    f.write('    DICIONARIO = {"campos_conhecidos": {}}\n')
    f.write('    print("\\n! Dicionario nao encontrado")\n\n')

    # Carregar estados
    f.write('# Carregar estados dos blocos\n')
    f.write('estados_dir = BASE_DIR / "data" / "logs"\n')
    f.write('ESTADOS = {}\n')
    f.write('if estados_dir.exists():\n')
    f.write('    for arquivo in estados_dir.glob(".bloco_*_state.json"):\n')
    f.write('        bloco_nome = arquivo.stem.replace(".", "")\n')
    f.write('        with open(arquivo, "r", encoding="utf-8") as f:\n')
    f.write('            ESTADOS[bloco_nome] = json.load(f)\n')
    f.write('    print(f" OK Estados: {len(ESTADOS)} blocos carregados")\n')
    f.write('else:\n')
    f.write('    print("! Pasta de estados nao encontrada")\n\n')

    # Funcao processar_arquivo
    f.write('# ' + '='*70 + '\n')
    f.write('# FUNCAO DE PROCESSAMENTO\n')
    f.write('# ' + '='*70 + '\n\n')
    f.write('def processar_arquivo(arquivo_path):\n')
    f.write('    """Processa um arquivo conforme configuracoes originais"""\n\n')
    f.write('    print(f"\\n{\'=\'*70}")\n')
    f.write('    print(f"Processando: {arquivo_path.name}")\n')
    f.write('    print(f"{\'=\'*70}")\n\n')

    # Carregar arquivo
    f.write('    # Carregar arquivo\n')
    f.write('    print("\\n[1/3] Carregando arquivo...")\n')
    f.write('    try:\n')
    f.write('        if arquivo_path.suffix.lower() in [\'.xlsx\', \'.xls\', \'.xlsm\']:\n')
    f.write('            df = pd.read_excel(arquivo_path)\n')
    f.write('        elif arquivo_path.suffix.lower() == \'.csv\':\n')
    f.write('            df = pd.read_csv(arquivo_path, encoding=\'utf-8\')\n')
    f.write('        else:\n')
    f.write('            print(f"  ! Formato nao suportado: {arquivo_path.suffix}")\n')
    f.write('            return None\n')
    f.write('        print(f"  OK Shape original: {df.shape}")\n')
    f.write('    except Exception as e:\n')
    f.write('        print(f"  ! ERRO ao carregar: {e}")\n')
    f.write('        return None\n\n')

    # Mapeamentos
    f.write('    # Aplicar mapeamentos\n')
    f.write('    print("\\n[2/3] Aplicando mapeamentos...")\n')
    f.write('    if \'bloco_12_state\' in ESTADOS:\n')
    f.write('        bloco12 = ESTADOS[\'bloco_12_state\']\n')
    f.write('        campos_mapeados = bloco12.get(\'campos_mapeados\', {})\n')
    f.write('        rename_dict = {}\n')
    f.write('        for col_orig, info in campos_mapeados.items():\n')
    f.write('            if col_orig in df.columns:\n')
    f.write('                tipo = info.get(\'tipo\', col_orig)\n')
    f.write('                if tipo != col_orig:\n')
    f.write('                    rename_dict[col_orig] = tipo\n')
    f.write('        if rename_dict:\n')
    f.write('            df.rename(columns=rename_dict, inplace=True)\n')
    f.write('            print(f"  OK {len(rename_dict)} colunas renomeadas")\n')
    f.write('        else:\n')
    f.write('            print("  -- Nenhum mapeamento necessario")\n')
    f.write('    else:\n')
    f.write('        print("  -- BLOCO 12 nao disponivel")\n\n')

    # Limpeza
    f.write('    # Limpeza basica\n')
    f.write('    print("\\n[3/3] Limpando dados...")\n')
    f.write('    linhas_antes = len(df)\n')
    f.write('    df.dropna(how=\'all\', inplace=True)\n')
    f.write('    df.dropna(axis=1, how=\'all\', inplace=True)\n')
    f.write('    linhas_depois = len(df)\n')
    f.write('    if linhas_antes != linhas_depois:\n')
    f.write('        print(f"  OK Removidas {linhas_antes - linhas_depois} linhas vazias")\n')
    f.write('    print(f"  OK Shape final: {df.shape}")\n')
    f.write('    return df\n\n')

    # Funcao main
    f.write('# ' + '='*70 + '\n')
    f.write('# MAIN\n')
    f.write('# ' + '='*70 + '\n\n')
    f.write('def main():\n')
    f.write('    """Processa todos os arquivos da pasta entrada"""\n\n')

    # Verificar entrada
    f.write('    # Verificar pasta de entrada\n')
    f.write('    if not PASTA_ENTRADA.exists():\n')
    f.write('        print(f"\\n! ERRO: Pasta de entrada nao encontrada")\n')
    f.write('        print(f"  Esperado: {PASTA_ENTRADA.absolute()}")\n')
    f.write('        print(f"\\n  COMO USAR:")\n')
    f.write('        print(f"  1. Criar pasta \'entrada\' no mesmo nivel do script")\n')
    f.write('        print(f"  2. Colocar arquivos .xlsx ou .csv na pasta")\n')
    f.write('        print(f"  3. Executar novamente")\n')
    f.write('        return\n\n')

    # Buscar arquivos
    f.write('    # Buscar arquivos\n')
    f.write('    arquivos = (\n')
    f.write('        list(PASTA_ENTRADA.glob("*.xlsx")) +\n')
    f.write('        list(PASTA_ENTRADA.glob("*.xls")) +\n')
    f.write('        list(PASTA_ENTRADA.glob("*.xlsm")) +\n')
    f.write('        list(PASTA_ENTRADA.glob("*.csv"))\n')
    f.write('    )\n')
    f.write('    if not arquivos:\n')
    f.write('        print(f"\\n! Nenhum arquivo encontrado em {PASTA_ENTRADA}")\n')
    f.write('        return\n\n')

    f.write('    print(f"\\n{\'=\'*70}")\n')
    f.write('    print(f"ARQUIVOS ENCONTRADOS: {len(arquivos)}")\n')
    f.write('    print(f"{\'=\'*70}")\n')
    f.write('    for arq in arquivos:\n')
    f.write('        print(f"  - {arq.name}")\n\n')

    # Criar saida
    f.write('    # Criar pasta de saida\n')
    f.write('    timestamp_exec = datetime.now().strftime("%Y%m%d_%H%M%S")\n')
    f.write('    pasta_saida = Path(f"./saida_{timestamp_exec}")\n')
    f.write('    pasta_saida.mkdir(exist_ok=True, parents=True)\n')
    f.write('    print(f"\\n OK Pasta de saida: {pasta_saida.absolute()}")\n\n')

    # Processar
    f.write('    # Processar cada arquivo\n')
    f.write('    resultados = {}\n')
    f.write('    for arquivo in arquivos:\n')
    f.write('        try:\n')
    f.write('            df_processado = processar_arquivo(arquivo)\n')
    f.write('            if df_processado is not None:\n')
    f.write('                output_file = pasta_saida / f"processado_{arquivo.stem}.xlsx"\n')
    f.write('                df_processado.to_excel(output_file, index=False)\n')
    f.write('                resultados[arquivo.name] = {\n')
    f.write('                    \'status\': \'OK\',\n')
    f.write('                    \'linhas\': len(df_processado),\n')
    f.write('                    \'colunas\': len(df_processado.columns),\n')
    f.write('                    \'arquivo_saida\': output_file.name\n')
    f.write('                }\n')
    f.write('                print(f"\\n OK SALVO: {output_file.name}")\n')
    f.write('            else:\n')
    f.write('                resultados[arquivo.name] = {\'status\': \'ERRO\', \'erro\': \'Processamento retornou None\'}\n')
    f.write('        except Exception as e:\n')
    f.write('            print(f"\\n! ERRO: {e}")\n')
    f.write('            resultados[arquivo.name] = {\'status\': \'ERRO\', \'erro\': str(e)}\n\n')

    # Relatorio
    f.write('    # Salvar relatorio\n')
    f.write('    relatorio = {\n')
    f.write('        \'timestamp\': datetime.now().isoformat(),\n')
    f.write(f'        \'configuracao_original\': {{"container": "{container_nome}", "timestamp": "{timestamp}"}},\n')
    f.write('        \'resultados\': resultados,\n')
    f.write('        \'resumo\': {\n')
    f.write('            \'total\': len(resultados),\n')
    f.write('            \'sucesso\': sum(1 for r in resultados.values() if r[\'status\'] == \'OK\'),\n')
    f.write('            \'erro\': sum(1 for r in resultados.values() if r[\'status\'] == \'ERRO\')\n')
    f.write('        }\n')
    f.write('    }\n')
    f.write('    relatorio_file = pasta_saida / \'RELATORIO.json\'\n')
    f.write('    with open(relatorio_file, \'w\', encoding=\'utf-8\') as f:\n')
    f.write('        json.dump(relatorio, f, indent=2, ensure_ascii=False)\n\n')

    # Resumo
    f.write('    # Resumo final\n')
    f.write('    print("\\n" + "="*70)\n')
    f.write('    print("RESUMO FINAL")\n')
    f.write('    print("="*70)\n')
    f.write('    print(f"Total: {relatorio[\'resumo\'][\'total\']}")\n')
    f.write('    print(f"Sucesso: {relatorio[\'resumo\'][\'sucesso\']}")\n')
    f.write('    print(f"Erros: {relatorio[\'resumo\'][\'erro\']}")\n')
    f.write('    print(f"Pasta saida: {pasta_saida.absolute()}")\n')
    f.write('    print(f"Relatorio: {relatorio_file.name}")\n')
    f.write('    print("="*70)\n\n')

    # Executar
    f.write('# ' + '='*70 + '\n')
    f.write('# EXECUTAR\n')
    f.write('# ' + '='*70 + '\n\n')
    f.write('if __name__ == "__main__":\n')
    f.write('    try:\n')
    f.write('        main()\n')
    f.write('    except KeyboardInterrupt:\n')
    f.write('        print("\\n\\nProcessamento interrompido pelo usuario")\n')
    f.write('    except Exception as e:\n')
    f.write('        print(f"\\n\\nERRO FATAL: {e}")\n')
    f.write('        import traceback\n')
    f.write('        traceback.print_exc()\n')

print(f"\n✅ SCRIPT GERADO COM SUCESSO!")
print(f"\n📁 Local:")
print(f"   {output_path}")
print(f"\n📖 COMO USAR:")
print(f"   1. cd \"{output_path.parent}\"")
print(f"   2. mkdir entrada")
print(f"   3. Colocar arquivos em entrada/")
print(f"   4. python script_reproducao.py")
print("\n" + "="*70)